# Highlight Sig Test values Streamlit Interface 

### **Imports**

In [1]:
# ============================================
# PART 1 — Imports & Basic Utilities
# ============================================
import pandas as pd
import numpy as np
import re
from typing import List, Tuple, Optional, Dict, Any, Set


### Read Input Data File (Raw Format)

In [ ]:
# >>> REPLACE any hard-coded path cell with this cell <<<
import os
from pathlib import Path
import pandas as pd

# 1) papermill may inject a variable called `input_file` if you pass params
uploaded_path = None
try:
    # if this notebook is executed with papermill and params include input_file
    uploaded_path = Path(input_file)  # NameError if not set
except NameError:
    uploaded_path = None

# 2) fallback: environment variable (app.py could set UPLOADED_FILE env)
if not uploaded_path:
    env_path = os.environ.get("UPLOADED_FILE")
    if env_path:
        uploaded_path = Path(env_path)

# 3) last resort: relative path inside project (useful for local testing)
if not uploaded_path:
    uploaded_path = Path("input_data") / "uploaded_file.xlsx"

# make sure we have a proper Path and it exists
if not uploaded_path or not uploaded_path.exists():
    raise FileNotFoundError(
        f"Uploaded file not found. Tried: "
        f"\n - papermill param `input_file` -> {locals().get('input_file', '<not set>')}"
        f"\n - env UPLOADED_FILE -> {os.environ.get('UPLOADED_FILE')}"
        f"\n - fallback -> {uploaded_path}"
    )

# now read via pandas:
df_raw = pd.read_excel(uploaded_path, header=None)
print("Reading uploaded file:", uploaded_path)
print("Raw shape:", df_raw.shape)
display(df_raw.head(10))


Raw shape: (124, 212)


,0,1,2,3,4,5,6,7,8,9,...,202,203,204,205,206,207,208,209,210,211
0,Table 1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Custom Table,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,Total,NaN,Sample,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,Total,NaN,Total,NaN,Local Youths,NaN,Non-Local Youths,NaN,...,I feel fearful for no apparent reason,NaN,I've been feeling optimistic about the future,NaN,I've been dealing well with my problems,NaN,I act without stopping to think,NaN,I enjoy new and exciting experiences even if t...,NaN
5,NaN,NaN,Count,Column N %,Count,Column N %,Count,Column N %,Count,Column N %,...,Count,Column N %,Count,Column N %,Count,Column N %,Count,Column N %,Count,Column N %
6,S5. Which category do you fall under?,Total,4805,1,4805,1,3005,1,302,1,...,472,1,3072,1,3379,1,488,1,2265,1
7,NaN,Secondary School/ IP Junior High Student,658,0.137,658,0.137,624,0.208,34,0.113,...,82,0.173,343,0.112,390,0.116,82,0.168,337,0.149
8,NaN,JC/MI/ IB/ IP Senior High Student,101,0.021,101,0.021,95,0.032,6,0.02,...,10,0.022,60,0.019,69,0.02,12,0.025,54,0.024
9,NaN,Polytechnic Student,305,0.063,305,0.063,296,0.098,9,0.03,...,47,0.1,175,0.057,204,0.06,35,0.072,183,0.081


### Clean Raw Data by Removing Footnotes

In [3]:
# CELL 2: Remove footnote rows before splitting into tables

def remove_footnotes(df_raw):
    """
    Remove rows that are footnotes (typically starting with letters like 'a.', 'b.', 'c.' 
    or containing statistical methodology text)
    """
    df_clean = df_raw.copy()
    rows_to_drop = []
    
    for idx, row in df_clean.iterrows():
        # Get first non-null value in the row
        first_val = None
        for val in row:
            if pd.notna(val) and str(val).strip() != '':
                first_val = str(val).strip()
                break
        
        if first_val:
            # Check if it's a footnote pattern
            footnote_patterns = [
                r'^[a-z]\.',  # starts with "a.", "b.", "c.", etc.
                r'^Results are based on',
                r'^This category is not used',
                r'^Tests are adjusted',
                r'^Cell counts',
                r'significance level',
                r'Bonferroni correction',
                r'column proportion',
            ]
            
            if any(re.search(pattern, first_val, re.IGNORECASE) for pattern in footnote_patterns):
                rows_to_drop.append(idx)
    
    df_clean = df_clean.drop(rows_to_drop).reset_index(drop=True)
    print(f"Removed {len(rows_to_drop)} footnote rows")
    return df_clean

# Clean the data
df_cleaned = remove_footnotes(df_raw)

print("Cleaned shape:", df_cleaned.shape)
display(df_cleaned.head(10))

Removed 19 footnote rows
Cleaned shape: (105, 212)


,0,1,2,3,4,5,6,7,8,9,...,202,203,204,205,206,207,208,209,210,211
0,Table 1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Custom Table,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,Total,NaN,Sample,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,Total,NaN,Total,NaN,Local Youths,NaN,Non-Local Youths,NaN,...,I feel fearful for no apparent reason,NaN,I've been feeling optimistic about the future,NaN,I've been dealing well with my problems,NaN,I act without stopping to think,NaN,I enjoy new and exciting experiences even if t...,NaN
5,NaN,NaN,Count,Column N %,Count,Column N %,Count,Column N %,Count,Column N %,...,Count,Column N %,Count,Column N %,Count,Column N %,Count,Column N %,Count,Column N %
6,S5. Which category do you fall under?,Total,4805,1,4805,1,3005,1,302,1,...,472,1,3072,1,3379,1,488,1,2265,1
7,NaN,Secondary School/ IP Junior High Student,658,0.137,658,0.137,624,0.208,34,0.113,...,82,0.173,343,0.112,390,0.116,82,0.168,337,0.149
8,NaN,JC/MI/ IB/ IP Senior High Student,101,0.021,101,0.021,95,0.032,6,0.02,...,10,0.022,60,0.019,69,0.02,12,0.025,54,0.024
9,NaN,Polytechnic Student,305,0.063,305,0.063,296,0.098,9,0.03,...,47,0.1,175,0.057,204,0.06,35,0.072,183,0.081


### Identify and Display Removed Footnote Rows

In [4]:
# Show what footnotes were removed
def show_footnotes(df_raw):
    """
    Display rows that match footnote patterns
    """
    footnote_rows = []
    
    for idx, row in df_raw.iterrows():
        # Get first non-null value in the row
        first_val = None
        for val in row:
            if pd.notna(val) and str(val).strip() != '':
                first_val = str(val).strip()
                break
        
        if first_val:
            # Check if it's a footnote pattern
            footnote_patterns = [
                r'^[a-z]\.',  # starts with "a.", "b.", "c.", etc.
                r'^Results are based on',
                r'^This category is not used',
                r'^Tests are adjusted',
                r'^Cell counts',
                r'significance level',
                r'Bonferroni correction',
                r'column proportion',
            ]
            
            if any(re.search(pattern, first_val, re.IGNORECASE) for pattern in footnote_patterns):
                footnote_rows.append((idx, first_val))
    
    print(f"Found {len(footnote_rows)} footnote rows in original data:")
    print("="*60)
    for idx, content in footnote_rows:
        print(f"Row {idx}: {content}")
    print("="*60)

show_footnotes(df_raw)

Found 19 footnote rows in original data:
Row 19: Comparisons of Column Proportionsc,d
Row 34: Results are based on two-sided tests with significance level .05. For each significant pair, the key of the category with the smaller column proportion appears under the category with the larger column proportion.
Row 35: a. This category is not used in comparisons because its column proportion is equal to zero or one.
Row 36: b. This category is not used in comparisons because the sum of case weights is less than two.
Row 37: c. Tests are adjusted for all pairwise comparisons within a row of each innermost subtable using the Bonferroni correction.
Row 38: d. Cell counts of some categories are not integers. They were rounded to the nearest integers before performing column proportions tests.
Row 51: Comparisons of Column Proportionsc,d
Row 58: Results are based on two-sided tests with significance level .05. For each significant pair, the key of the category with the smaller column proportion 

### **Detect Header Row**

In [5]:
def detect_header_row(df, min_frac_nonnull=0.15, max_scan_rows=40):
    ncols = df.shape[1]
    for i in range(min(max_scan_rows, df.shape[0])):
        nonnull = df.iloc[i].notna().sum()
        if nonnull / max(1, ncols) >= min_frac_nonnull:
            return i
    return None

header_row = detect_header_row(df_cleaned)
print("Detected header row:", header_row)

if header_row is not None:
    # Set header from cleaned dataframe, not from file
    df_header = df_cleaned.copy()
    df_header.columns = df_cleaned.iloc[header_row]
    df_header = df_header.drop(range(0, header_row + 1)).reset_index(drop=True)
else:
    df_header = df_cleaned.copy()

display(df_header.head(10))

Detected header row: 4


4,NaN,NaN,Total,NaN,Total,NaN,Local Youths,NaN,Non-Local Youths,NaN,...,I feel fearful for no apparent reason,NaN,I've been feeling optimistic about the future,NaN,I've been dealing well with my problems,NaN,I act without stopping to think,NaN,I enjoy new and exciting experiences even if they are unusual,NaN
0,NaN,NaN,Count,Column N %,Count,Column N %,Count,Column N %,Count,Column N %,...,Count,Column N %,Count,Column N %,Count,Column N %,Count,Column N %,Count,Column N %
1,S5. Which category do you fall under?,Total,4805,1,4805,1,3005,1,302,1,...,472,1,3072,1,3379,1,488,1,2265,1
2,NaN,Secondary School/ IP Junior High Student,658,0.137,658,0.137,624,0.208,34,0.113,...,82,0.173,343,0.112,390,0.116,82,0.168,337,0.149
3,NaN,JC/MI/ IB/ IP Senior High Student,101,0.021,101,0.021,95,0.032,6,0.02,...,10,0.022,60,0.019,69,0.02,12,0.025,54,0.024
4,NaN,Polytechnic Student,305,0.063,305,0.063,296,0.098,9,0.03,...,47,0.1,175,0.057,204,0.06,35,0.072,183,0.081
5,NaN,ITE Student,111,0.023,111,0.023,109,0.036,2,0.007,...,19,0.041,57,0.018,59,0.018,16,0.033,52,0.023
6,NaN,"Local University Student (i.e., NUS/NTU/ SMU/ ...",529,0.11,529,0.11,483,0.161,46,0.152,...,56,0.12,353,0.115,384,0.114,42,0.085,294,0.13
7,NaN,Private Education Institute Student,347,0.072,347,0.072,180,0.06,167,0.553,...,48,0.102,210,0.068,208,0.062,45,0.092,154,0.068
8,NaN,Full-time National Servicemen (NSF),164,0.034,164,0.034,164,0.055,0,0,...,24,0.05,102,0.033,112,0.033,23,0.048,97,0.043
9,NaN,Currently holding a full-time/ part-time job,2063,0.429,2063,0.429,1032,0.343,38,0.126,...,157,0.333,1403,0.457,1526,0.452,200,0.41,953,0.421


### **Split Into Tables**

In [6]:
def split_tables(df):
    df_norm = df.replace(r'^\s*$', np.nan, regex=True)
    blank_mask = df_norm.isna().all(axis=1)
    blank_rows = df_norm.index[blank_mask].tolist()

    boundaries = [-1] + blank_rows + [len(df_norm)]
    tables = []

    for a, b in zip(boundaries[:-1], boundaries[1:]):
        chunk = df_norm.iloc[a+1:b].dropna(how='all')
        if not chunk.empty and chunk.notna().sum(axis=1).ge(2).any():
            tables.append(chunk.reset_index(drop=True))

    return tables

tables = split_tables(df_cleaned)  # ← Changed from df_header

print("Number of tables found:", len(tables))
for i, t in enumerate(tables):
    print(f"\nTABLE {i}: shape {t.shape}")
    display(t.head(5))

Number of tables found: 6

TABLE 0: shape (16, 212)


,0,1,2,3,4,5,6,7,8,9,...,202,203,204,205,206,207,208,209,210,211
0,Custom Table,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,Total,NaN,Sample,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,Total,NaN,Total,NaN,Local Youths,NaN,Non-Local Youths,NaN,...,I feel fearful for no apparent reason,NaN,I've been feeling optimistic about the future,NaN,I've been dealing well with my problems,NaN,I act without stopping to think,NaN,I enjoy new and exciting experiences even if t...,NaN
3,NaN,NaN,Count,Column N %,Count,Column N %,Count,Column N %,Count,Column N %,...,Count,Column N %,Count,Column N %,Count,Column N %,Count,Column N %,Count,Column N %
4,S5. Which category do you fall under?,Total,4805,1,4805,1,3005,1,302,1,...,472,1,3072,1,3379,1,488,1,2265,1



TABLE 1: shape (14, 212)


,0,1,2,3,4,5,6,7,8,9,...,202,203,204,205,206,207,208,209,210,211
0,NaN,NaN,Total,Sample,NaN,NaN,NaN,S2. Age Group,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,Total,Total,Local Youths,Non-Local Youths,Adults,Total,13 – 14 years,15 – 19 years,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,(A),(A),(B),(C),(D),(A),(B),(C),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,S5. Which category do you fall under?,Secondary School/ IP Junior High Student,.,NaN,A C,NaN,.a,NaN,A C,A,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,JC/MI/ IB/ IP Senior High Student,.,NaN,A,NaN,.a,D,.a,A D,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN



TABLE 2: shape (8, 212)


...

,0,1,2,3,4,5,6,7,8,9,...,202,203,204,205,206,207,208,209,210,211
0,NaN,NaN,Total,S2. Age Group,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,Total,Total,30-34 years,35-39 years,40-44 years,45-49 years,50-54 years,55-59 years,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,(A),(A),(B),(C),(D),(E),(F),(G),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,B1_Stance,Conservative,.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,Neutral,.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN



TABLE 4: shape (26, 212)


,0,1,2,3,4,5,6,7,8,9,...,202,203,204,205,206,207,208,209,210,211
0,Custom Table,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,Total,NaN,S5. Education/ Status vs S6. In-Risk Youth,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,Total,NaN,Total,NaN,Sec,NaN,JC,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,Count,Column N %,Count,Column N %,Count,Column N %,Count,Column N %,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,$A1.Score,Total,3005.123,1,3005.123,1,613.395,1,93.03,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN



TABLE 5: shape (24, 212)


,0,1,2,3,4,5,6,7,8,9,...,202,203,204,205,206,207,208,209,210,211
0,NaN,NaN,Total,S5. Education/ Status vs S6. In-Risk Youth,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,Total,Total,Sec,JC,Poly,ITE,Local Uni,PEI,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,(A),(A),(B),(C),(D),(E),(F),(G),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,$A1.Score,Consuming drugs can be addictive.,.,B,NaN,NaN,B,NaN,B,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,Consuming drugs will harm one’s health.,.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
# Drop columns that are all NaN for every table and display them again
cleaned_tables = []

print("\nDropping all-NaN columns from each table...\n")

for i, t in enumerate(tables):
    t_clean = t.dropna(axis=1, how='all')   # remove columns where all values are NaN
    cleaned_tables.append(t_clean)

    print(f"\nCLEANED TABLE {i}: shape {t_clean.shape}")
    display(t_clean.head(5))



Dropping all-NaN columns from each table...


CLEANED TABLE 0: shape (16, 212)


,0,1,2,3,4,5,6,7,8,9,...,202,203,204,205,206,207,208,209,210,211
0,Custom Table,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,Total,NaN,Sample,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,Total,NaN,Total,NaN,Local Youths,NaN,Non-Local Youths,NaN,...,I feel fearful for no apparent reason,NaN,I've been feeling optimistic about the future,NaN,I've been dealing well with my problems,NaN,I act without stopping to think,NaN,I enjoy new and exciting experiences even if t...,NaN
3,NaN,NaN,Count,Column N %,Count,Column N %,Count,Column N %,Count,Column N %,...,Count,Column N %,Count,Column N %,Count,Column N %,Count,Column N %,Count,Column N %
4,S5. Which category do you fall under?,Total,4805,1,4805,1,3005,1,302,1,...,472,1,3072,1,3379,1,488,1,2265,1



CLEANED TABLE 1: shape (14, 107)


,0,1,2,3,4,5,6,7,8,9,...,97,98,99,100,101,102,103,104,105,106
0,NaN,NaN,Total,Sample,NaN,NaN,NaN,S2. Age Group,NaN,NaN,...,E2. Psychographics (T2B),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,Total,Total,Local Youths,Non-Local Youths,Adults,Total,13 – 14 years,15 – 19 years,...,Total,I feel that studying is pointless for me,I get on badly with the teachers,I am content with my life,I feel lonely,I feel fearful for no apparent reason,I've been feeling optimistic about the future,I've been dealing well with my problems,I act without stopping to think,I enjoy new and exciting experiences even if t...
2,NaN,NaN,(A),(A),(B),(C),(D),(A),(B),(C),...,(A),(B),(C),(D),(E),(F),(G),(H),(I),(J)
3,S5. Which category do you fall under?,Secondary School/ IP Junior High Student,.,NaN,A C,NaN,.a,NaN,A C,A,...,D G H,A D E F G H I J,A D E G H I J,NaN,NaN,D G H,NaN,NaN,G H,D G H
4,NaN,JC/MI/ IB/ IP Senior High Student,.,NaN,A,NaN,.a,D,.a,A D,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN



CLEANED TABLE 2: shape (8, 180)


...

,0,1,2,3,4,5,6,7,8,9,...,81,82,83,84,85,86,87,88,89,90
0,NaN,NaN,Total,S2. Age Group,NaN,NaN,NaN,NaN,NaN,NaN,...,E2. Psychographics (T2B),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,Total,Total,30-34 years,35-39 years,40-44 years,45-49 years,50-54 years,55-59 years,...,Total,I feel that studying is pointless for me,I get on badly with the teachers,I am content with my life,I feel lonely,I feel fearful for no apparent reason,I've been feeling optimistic about the future,I've been dealing well with my problems,I act without stopping to think,I enjoy new and exciting experiences even if t...
2,NaN,NaN,(A),(A),(B),(C),(D),(E),(F),(G),...,(A),(B),(C),(D),(E),(F),(G),(H),(I),(J)
3,B1_Stance,Conservative,.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,".a,b",".a,b",A,NaN,NaN,A,A J,NaN,NaN
4,NaN,Neutral,.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,D G H,".a,b",".a,b",NaN,NaN,NaN,NaN,NaN,NaN,NaN



CLEANED TABLE 4: shape (26, 168)


,0,1,2,3,4,5,6,7,8,9,...,158,159,160,161,162,163,164,165,166,167
0,Custom Table,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,Total,NaN,S5. Education/ Status vs S6. In-Risk Youth,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,Total,NaN,Total,NaN,Sec,NaN,JC,NaN,...,I feel fearful for no apparent reason,NaN,I've been feeling optimistic about the future,NaN,I've been dealing well with my problems,NaN,I act without stopping to think,NaN,I enjoy new and exciting experiences even if t...,NaN
3,NaN,NaN,Count,Column N %,Count,Column N %,Count,Column N %,Count,Column N %,...,Count,Column N %,Count,Column N %,Count,Column N %,Count,Column N %,Count,Column N %
4,$A1.Score,Total,3005.123,1,3005.123,1,613.395,1,93.03,1,...,370.5214,1,1803.3554,1,2010.1728,1,340.0788,1,1672.397,1



CLEANED TABLE 5: shape (24, 85)


,0,1,2,3,4,5,6,7,8,9,...,75,76,77,78,79,80,81,82,83,84
0,NaN,NaN,Total,S5. Education/ Status vs S6. In-Risk Youth,NaN,NaN,NaN,NaN,NaN,NaN,...,E2. Psychographics (T2B),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,Total,Total,Sec,JC,Poly,ITE,Local Uni,PEI,...,Total,I feel that studying is pointless for me,I get on badly with the teachers,I am content with my life,I feel lonely,I feel fearful for no apparent reason,I've been feeling optimistic about the future,I've been dealing well with my problems,I act without stopping to think,I enjoy new and exciting experiences even if t...
2,NaN,NaN,(A),(A),(B),(C),(D),(E),(F),(G),...,(A),(B),(C),(D),(E),(F),(G),(H),(I),(J)
3,$A1.Score,Consuming drugs can be addictive.,.,B,NaN,NaN,B,NaN,B,NaN,...,NaN,NaN,NaN,A B,NaN,NaN,A B,A B,NaN,A B
4,NaN,Consuming drugs will harm one’s health.,.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,A,NaN,NaN,A C E F I,A,NaN,NaN


### Drop Irrelevant Rows and Apply Dynamic Header Detection

In [8]:
import pandas as pd
import numpy as np
from IPython.display import display
import re

# --- REQUIRED HELPERS (add these so the cell never fails again) ---

def _normalize_cell(x):
    """Return cleaned string or None."""
    if x is None or (isinstance(x, float) and np.isnan(x)):
        return None
    s = str(x).strip()
    if s == "" or s.lower().startswith("unnamed"):
        return None
    return s

def _first_non_null_in_row(row):
    """Return the first non-null, non-empty string in a row list."""
    for v in row:
        if pd.notna(v) and str(v).strip() != "":
            return v
    return None

def _make_unique_columns(cols):
    """Ensure unique column names by appending suffixes."""
    out = []
    counts = {}
    for c in cols:
        base = str(c)
        if base in counts:
            counts[base] += 1
            out_name = f"{base}_{counts[base]}"
        else:
            counts[base] = 0
            out_name = base
        out.append(out_name)
    return out

# --- MAIN PROCESSING BLOCK ---

processed_after_rowdrop_and_header = []
print("\nApplying content-based row drop (contains 'S2' or 'Sample') and header-row replacement:\n")

def row_contains_forbidden_values(row, forbidden=("s2", "sample")):
    """Return True if any cell contains forbidden text."""
    for v in row:
        if pd.isna(v):
            continue
        txt = str(v).lower()
        for word in forbidden:
            if word in txt:
                return True
    return False

for i, t in enumerate(cleaned_tables):
    df = t.copy()

    # 1) DROP ROWS with forbidden keywords
    mask = df.apply(lambda row: row_contains_forbidden_values(row), axis=1)
    before_rows = df.shape[0]
    df = df.loc[~mask].reset_index(drop=True)
    after_rows = df.shape[0]

    # 2) FIND HEADER ROW (first non-null cell == "Total")
    header_row_idx = None
    max_search = min(12, df.shape[0])

    for r in range(max_search):
        first_val = _first_non_null_in_row(df.iloc[r].tolist())
        if first_val is None:
            continue
        norm = _normalize_cell(first_val)
        if norm is not None and norm.lower() == "total":
            header_row_idx = r
            break

    # 3) APPLY HEADER ROW AS COLUMNS
    if header_row_idx is not None:
        new_cols = df.iloc[header_row_idx].fillna("").astype(str).tolist()
        new_cols = [c.strip() for c in new_cols]
        new_cols = _make_unique_columns(new_cols)

        df = df.iloc[header_row_idx + 1:].reset_index(drop=True)
        df.columns = new_cols
        hdr_msg = f"Header row at index {header_row_idx} used as columns."
    else:
        hdr_msg = "No 'Total' header row found; columns unchanged."

    processed_after_rowdrop_and_header.append(df)

    print(f"\nTABLE {i}: rows before={before_rows}, after drop={after_rows}. {hdr_msg}")
    print(f"Shape after processing: {df.shape}")
    display(df.head(5))

# overwrite cleaned_tables
cleaned_tables = processed_after_rowdrop_and_header



Applying content-based row drop (contains 'S2' or 'Sample') and header-row replacement:


TABLE 0: rows before=16, after drop=15. Header row at index 1 used as columns.
Shape after processing: (13, 212)


,,_1,Total,_2,Total_1,_3,Local Youths,_4,Non-Local Youths,_5,...,I feel fearful for no apparent reason,_102,I've been feeling optimistic about the future,_103,I've been dealing well with my problems,_104,I act without stopping to think,_105,I enjoy new and exciting experiences even if they are unusual,_106
0,NaN,NaN,Count,Column N %,Count,Column N %,Count,Column N %,Count,Column N %,...,Count,Column N %,Count,Column N %,Count,Column N %,Count,Column N %,Count,Column N %
1,S5. Which category do you fall under?,Total,4805,1,4805,1,3005,1,302,1,...,472,1,3072,1,3379,1,488,1,2265,1
2,NaN,Secondary School/ IP Junior High Student,658,0.137,658,0.137,624,0.208,34,0.113,...,82,0.173,343,0.112,390,0.116,82,0.168,337,0.149
3,NaN,JC/MI/ IB/ IP Senior High Student,101,0.021,101,0.021,95,0.032,6,0.02,...,10,0.022,60,0.019,69,0.02,12,0.025,54,0.024
4,NaN,Polytechnic Student,305,0.063,305,0.063,296,0.098,9,0.03,...,47,0.1,175,0.057,204,0.06,35,0.072,183,0.081



TABLE 1: rows before=14, after drop=13. Header row at index 0 used as columns.
Shape after processing: (12, 107)


,,_1,Total,Total_1,Local Youths,Non-Local Youths,Adults,Total_2,13 – 14 years,15 – 19 years,...,Total_16,I feel that studying is pointless for me,I get on badly with the teachers,I am content with my life,I feel lonely,I feel fearful for no apparent reason,I've been feeling optimistic about the future,I've been dealing well with my problems,I act without stopping to think,I enjoy new and exciting experiences even if they are unusual
0,NaN,NaN,(A),(A),(B),(C),(D),(A),(B),(C),...,(A),(B),(C),(D),(E),(F),(G),(H),(I),(J)
1,S5. Which category do you fall under?,Secondary School/ IP Junior High Student,.,NaN,A C,NaN,.a,NaN,A C,A,...,D G H,A D E F G H I J,A D E G H I J,NaN,NaN,D G H,NaN,NaN,G H,D G H
2,NaN,JC/MI/ IB/ IP Senior High Student,.,NaN,A,NaN,.a,D,.a,A D,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,Polytechnic Student,.,C,A C,NaN,.a,E,.a,A D E,...,NaN,NaN,NaN,NaN,NaN,A D G H,NaN,NaN,NaN,A D G H
4,NaN,ITE Student,.,NaN,A C,NaN,.a,D E,.a,A D E,...,D H,NaN,NaN,NaN,NaN,D G H,NaN,NaN,NaN,NaN



TABLE 2: rows before=8, after drop=7. Header row at index 1 used as columns.
Shape after processing: (5, 180)


...

,,_1,Total,Total_1,30-34 years,35-39 years,40-44 years,45-49 years,50-54 years,55-59 years,...,Total_14,I feel that studying is pointless for me,I get on badly with the teachers,I am content with my life,I feel lonely,I feel fearful for no apparent reason,I've been feeling optimistic about the future,I've been dealing well with my problems,I act without stopping to think,I enjoy new and exciting experiences even if they are unusual
0,NaN,NaN,(A),(A),(B),(C),(D),(E),(F),(G),...,(A),(B),(C),(D),(E),(F),(G),(H),(I),(J)
1,B1_Stance,Conservative,.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,".a,b",".a,b",A,NaN,NaN,A,A J,NaN,NaN
2,NaN,Neutral,.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,D G H,".a,b",".a,b",NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,Liberal,.,NaN,NaN,NaN,.a,NaN,NaN,.a,...,H,".a,b",".a,b",NaN,A D G H,.a,NaN,NaN,H,D H



TABLE 4: rows before=26, after drop=25. Header row at index 1 used as columns.
Shape after processing: (23, 168)


,,_1,Total,_2,Total_1,_3,Sec,_4,JC,_5,...,I feel fearful for no apparent reason,_80,I've been feeling optimistic about the future,_81,I've been dealing well with my problems,_82,I act without stopping to think,_83,I enjoy new and exciting experiences even if they are unusual,_84
0,NaN,NaN,Count,Column N %,Count,Column N %,Count,Column N %,Count,Column N %,...,Count,Column N %,Count,Column N %,Count,Column N %,Count,Column N %,Count,Column N %
1,$A1.Score,Total,3005.123,1,3005.123,1,613.395,1,93.03,1,...,370.5214,1,1803.3554,1,2010.1728,1,340.0788,1,1672.397,1
2,NaN,Consuming drugs can be addictive.,2842.3636,0.945839,2842.3636,0.945839,556.1448,0.906667,88.6886,0.953333,...,352.0813,0.950232,1735.338,0.962283,1923.9204,0.957092,322.8728,0.949406,1603.3934,0.95874
3,NaN,Consuming drugs will harm one’s health.,2837.9808,0.944381,2837.9808,0.944381,580.6806,0.946667,86.5179,0.93,...,342.9147,0.925492,1740.0578,0.9649,1926.0516,0.958152,315.5831,0.927971,1597.6966,0.955333
4,NaN,Consuming drugs affects one’s life negatively.,2773.5828,0.922952,2773.5828,0.922952,560.2341,0.913333,84.9674,0.913333,...,340.1677,0.918078,1715.7353,0.951413,1894.2273,0.942321,318.7864,0.93739,1578.0146,0.943565



TABLE 5: rows before=24, after drop=23. Header row at index 0 used as columns.
Shape after processing: (22, 85)


,,_1,Total,Total_1,Sec,JC,Poly,ITE,Local Uni,PEI,...,Total_14,I feel that studying is pointless for me,I get on badly with the teachers,I am content with my life,I feel lonely,I feel fearful for no apparent reason,I've been feeling optimistic about the future,I've been dealing well with my problems,I act without stopping to think,I enjoy new and exciting experiences even if they are unusual
0,NaN,NaN,(A),(A),(B),(C),(D),(E),(F),(G),...,(A),(B),(C),(D),(E),(F),(G),(H),(I),(J)
1,$A1.Score,Consuming drugs can be addictive.,.,B,NaN,NaN,B,NaN,B,NaN,...,NaN,NaN,NaN,A B,NaN,NaN,A B,A B,NaN,A B
2,NaN,Consuming drugs will harm one’s health.,.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,A,NaN,NaN,A C E F I,A,NaN,NaN
3,NaN,Consuming drugs affects one’s life negatively.,.,NaN,NaN,NaN,J,NaN,NaN,NaN,...,NaN,NaN,NaN,A C E,NaN,NaN,A C E,A C E,NaN,A C E
4,NaN,Consuming drugs affects families negatively.,.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,C,NaN,NaN,A C E H I,NaN,C,A C,A C,NaN,A C


### **Detect Title Row & Unmerge Title Row**

In [9]:
import pandas as pd
import numpy as np
from IPython.display import display

# --- Needed helper functions ---
def _normalize_cell(x):
    if x is None or (isinstance(x, float) and np.isnan(x)):
        return None
    s = str(x).strip()
    if s == "" or s.lower().startswith("unnamed"):
        return None
    return s

def _first_non_null_in_row(row):
    """Return the first non-null, non-empty string value in a row."""
    for v in row:
        if pd.notna(v) and str(v).strip() != "":
            return v
    return None

def _make_unique_columns(cols):
    """Ensure all column names are unique."""
    out = []
    counts = {}
    for c in cols:
        base = str(c)
        if base in counts:
            counts[base] += 1
            out_name = f"{base}_{counts[base]}"
        else:
            counts[base] = 0
            out_name = base
        out.append(out_name)
    return out

def row_contains_forbidden_values(row, forbidden=("s2", "sample")):
    """Return True if any cell contains forbidden keyword(s)."""
    for v in row:
        if pd.isna(v):
            continue
        txt = str(v).lower()
        for word in forbidden:
            if word in txt:   # substring match
                return True
    return False

# --- Main processing step ---
processed_after_rowdrop_and_header = []
print("\nApplying content-based row drop (contains 'S2' or 'Sample') and header-row replacement:\n")

for i, t in enumerate(cleaned_tables):
    df = t.copy()

    # 1) DROP rows containing forbidden text
    mask = df.apply(lambda row: row_contains_forbidden_values(row), axis=1)
    before_rows = df.shape[0]
    df = df.loc[~mask].reset_index(drop=True)
    after_rows = df.shape[0]

    # 2) Detect header row whose first non-null cell == 'Total'
    header_row_idx = None
    max_search = min(12, df.shape[0])

    for r in range(max_search):
        first_val = _first_non_null_in_row(df.iloc[r].tolist())
        if first_val is None:
            continue
        norm = _normalize_cell(first_val)
        if norm is not None and norm.lower() == "total":
            header_row_idx = r
            break

    # 3) Apply header if found
    if header_row_idx is not None:
        new_cols = df.iloc[header_row_idx].fillna("").astype(str).tolist()
        new_cols = [c.strip() for c in new_cols]
        new_cols = _make_unique_columns(new_cols)
        df = df.iloc[header_row_idx+1:].reset_index(drop=True)
        df.columns = new_cols
        hdr_msg = f"Header row at index {header_row_idx} used as columns."
    else:
        hdr_msg = "No 'Total' header row found; columns unchanged."

    processed_after_rowdrop_and_header.append(df)

    print(f"\nTABLE {i}: rows before={before_rows}, after drop={after_rows}. {hdr_msg}")
    print(f"Shape after processing: {df.shape}")
    display(df.head(5))

# overwrite cleaned_tables if desired
cleaned_tables = processed_after_rowdrop_and_header



Applying content-based row drop (contains 'S2' or 'Sample') and header-row replacement:


TABLE 0: rows before=13, after drop=13. No 'Total' header row found; columns unchanged.
Shape after processing: (13, 212)


,,_1,Total,_2,Total_1,_3,Local Youths,_4,Non-Local Youths,_5,...,I feel fearful for no apparent reason,_102,I've been feeling optimistic about the future,_103,I've been dealing well with my problems,_104,I act without stopping to think,_105,I enjoy new and exciting experiences even if they are unusual,_106
0,NaN,NaN,Count,Column N %,Count,Column N %,Count,Column N %,Count,Column N %,...,Count,Column N %,Count,Column N %,Count,Column N %,Count,Column N %,Count,Column N %
1,S5. Which category do you fall under?,Total,4805,1,4805,1,3005,1,302,1,...,472,1,3072,1,3379,1,488,1,2265,1
2,NaN,Secondary School/ IP Junior High Student,658,0.137,658,0.137,624,0.208,34,0.113,...,82,0.173,343,0.112,390,0.116,82,0.168,337,0.149
3,NaN,JC/MI/ IB/ IP Senior High Student,101,0.021,101,0.021,95,0.032,6,0.02,...,10,0.022,60,0.019,69,0.02,12,0.025,54,0.024
4,NaN,Polytechnic Student,305,0.063,305,0.063,296,0.098,9,0.03,...,47,0.1,175,0.057,204,0.06,35,0.072,183,0.081



TABLE 1: rows before=12, after drop=12. No 'Total' header row found; columns unchanged.
Shape after processing: (12, 107)


,,_1,Total,Total_1,Local Youths,Non-Local Youths,Adults,Total_2,13 – 14 years,15 – 19 years,...,Total_16,I feel that studying is pointless for me,I get on badly with the teachers,I am content with my life,I feel lonely,I feel fearful for no apparent reason,I've been feeling optimistic about the future,I've been dealing well with my problems,I act without stopping to think,I enjoy new and exciting experiences even if they are unusual
0,NaN,NaN,(A),(A),(B),(C),(D),(A),(B),(C),...,(A),(B),(C),(D),(E),(F),(G),(H),(I),(J)
1,S5. Which category do you fall under?,Secondary School/ IP Junior High Student,.,NaN,A C,NaN,.a,NaN,A C,A,...,D G H,A D E F G H I J,A D E G H I J,NaN,NaN,D G H,NaN,NaN,G H,D G H
2,NaN,JC/MI/ IB/ IP Senior High Student,.,NaN,A,NaN,.a,D,.a,A D,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,Polytechnic Student,.,C,A C,NaN,.a,E,.a,A D E,...,NaN,NaN,NaN,NaN,NaN,A D G H,NaN,NaN,NaN,A D G H
4,NaN,ITE Student,.,NaN,A C,NaN,.a,D E,.a,A D E,...,D H,NaN,NaN,NaN,NaN,D G H,NaN,NaN,NaN,NaN



TABLE 2: rows before=5, after drop=5. No 'Total' header row found; columns unchanged.
Shape after processing: (5, 180)


...

,,_1,Total,Total_1,30-34 years,35-39 years,40-44 years,45-49 years,50-54 years,55-59 years,...,Total_14,I feel that studying is pointless for me,I get on badly with the teachers,I am content with my life,I feel lonely,I feel fearful for no apparent reason,I've been feeling optimistic about the future,I've been dealing well with my problems,I act without stopping to think,I enjoy new and exciting experiences even if they are unusual
0,NaN,NaN,(A),(A),(B),(C),(D),(E),(F),(G),...,(A),(B),(C),(D),(E),(F),(G),(H),(I),(J)
1,B1_Stance,Conservative,.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,".a,b",".a,b",A,NaN,NaN,A,A J,NaN,NaN
2,NaN,Neutral,.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,D G H,".a,b",".a,b",NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,Liberal,.,NaN,NaN,NaN,.a,NaN,NaN,.a,...,H,".a,b",".a,b",NaN,A D G H,.a,NaN,NaN,H,D H



TABLE 4: rows before=23, after drop=23. No 'Total' header row found; columns unchanged.
Shape after processing: (23, 168)


,,_1,Total,_2,Total_1,_3,Sec,_4,JC,_5,...,I feel fearful for no apparent reason,_80,I've been feeling optimistic about the future,_81,I've been dealing well with my problems,_82,I act without stopping to think,_83,I enjoy new and exciting experiences even if they are unusual,_84
0,NaN,NaN,Count,Column N %,Count,Column N %,Count,Column N %,Count,Column N %,...,Count,Column N %,Count,Column N %,Count,Column N %,Count,Column N %,Count,Column N %
1,$A1.Score,Total,3005.123,1,3005.123,1,613.395,1,93.03,1,...,370.5214,1,1803.3554,1,2010.1728,1,340.0788,1,1672.397,1
2,NaN,Consuming drugs can be addictive.,2842.3636,0.945839,2842.3636,0.945839,556.1448,0.906667,88.6886,0.953333,...,352.0813,0.950232,1735.338,0.962283,1923.9204,0.957092,322.8728,0.949406,1603.3934,0.95874
3,NaN,Consuming drugs will harm one’s health.,2837.9808,0.944381,2837.9808,0.944381,580.6806,0.946667,86.5179,0.93,...,342.9147,0.925492,1740.0578,0.9649,1926.0516,0.958152,315.5831,0.927971,1597.6966,0.955333
4,NaN,Consuming drugs affects one’s life negatively.,2773.5828,0.922952,2773.5828,0.922952,560.2341,0.913333,84.9674,0.913333,...,340.1677,0.918078,1715.7353,0.951413,1894.2273,0.942321,318.7864,0.93739,1578.0146,0.943565



TABLE 5: rows before=22, after drop=22. No 'Total' header row found; columns unchanged.
Shape after processing: (22, 85)


,,_1,Total,Total_1,Sec,JC,Poly,ITE,Local Uni,PEI,...,Total_14,I feel that studying is pointless for me,I get on badly with the teachers,I am content with my life,I feel lonely,I feel fearful for no apparent reason,I've been feeling optimistic about the future,I've been dealing well with my problems,I act without stopping to think,I enjoy new and exciting experiences even if they are unusual
0,NaN,NaN,(A),(A),(B),(C),(D),(E),(F),(G),...,(A),(B),(C),(D),(E),(F),(G),(H),(I),(J)
1,$A1.Score,Consuming drugs can be addictive.,.,B,NaN,NaN,B,NaN,B,NaN,...,NaN,NaN,NaN,A B,NaN,NaN,A B,A B,NaN,A B
2,NaN,Consuming drugs will harm one’s health.,.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,A,NaN,NaN,A C E F I,A,NaN,NaN
3,NaN,Consuming drugs affects one’s life negatively.,.,NaN,NaN,NaN,J,NaN,NaN,NaN,...,NaN,NaN,NaN,A C E,NaN,NaN,A C E,A C E,NaN,A C E
4,NaN,Consuming drugs affects families negatively.,.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,C,NaN,NaN,A C E H I,NaN,C,A C,A C,NaN,A C


### Extract Table Titles and Assign Unique Names

In [10]:
import re
from collections import defaultdict

# helper reused from earlier or re-declare if needed
def safe_name(s):
    if s is None:
        return ""
    t = str(s).strip()
    if t == "" or t.lower() in {"nan", "none", "nonetype"}:
        return ""
    t = re.sub(r'\s+', ' ', t.replace('\n', ' ').strip())
    t = re.sub(r'[\\/:\*\?"<>\|\[\]\(\)]', '', t)
    if len(t) > 60:
        t = t[:57].rstrip() + "..."
    return t

# helper: proportion of cells containing letters
def letter_proportion(df):
    total = 0
    letters = 0
    for val in df.to_numpy().ravel():
        total += 1
        if pd.isna(val):
            continue
        if re.search(r'[A-Za-z]', str(val)):
            letters += 1
    return letters / total if total > 0 else 0.0

# optional fallback heuristic: use is_count_pct_table if available in the namespace
use_count_pct_fallback = 'is_count_pct_table' in globals()

# 1) collect titles from first cell of second row (index 1, col 0)
table_titles = {}
for i, df in enumerate(cleaned_tables):
    title = None
    try:
        if df.shape[0] > 1 and df.shape[1] > 0:
            raw = df.iloc[1, 0]
            title = _normalize_cell(raw)
    except Exception:
        title = None
    # fallback to generic if no title
    table_titles[i] = title or f"table_{i}"

# 2) group indices by title
groups = defaultdict(list)
for idx, ttitle in table_titles.items():
    groups[ttitle].append(idx)

# 3) assign names, marking SigTest tables where duplicates exist
table_names = {}
name_counts = {}

for title, indices in groups.items():
    base = safe_name(title) or f"table_{indices[0]}"
    # if only one table with this title -> assign unique base name
    if len(indices) == 1:
        prev = name_counts.get(base, 0)
        if prev == 0:
            table_names[indices[0]] = base
            name_counts[base] = 1
        else:
            table_names[indices[0]] = f"{base}_{prev}"
            name_counts[base] = prev + 1
        continue

    # multiple tables share same title -> find candidate with highest letter proportion
    scores = []
    for idx in indices:
        prop = letter_proportion(cleaned_tables[idx])
        scores.append((prop, idx))
    scores.sort(reverse=True)  # highest letter proportion first

    # best candidate is likely SigTest (mostly-letter)
    best_prop, best_idx = scores[0]
    # tie handling: if tie and fallback available, use is_count_pct_table to choose SigTest instead
    tied = [s for s in scores if abs(s[0] - best_prop) < 1e-9]
    if len(tied) > 1 and use_count_pct_fallback:
        # pick the one with is_count_pct_table==True as the SigTest, else keep best_idx
        sig_candidate = None
        for _, idx in tied:
            try:
                if is_count_pct_table(cleaned_tables[idx]):
                    sig_candidate = idx
                    break
            except Exception:
                pass
        if sig_candidate is not None:
            best_idx = sig_candidate

    # now assign names: main base name to first non-sig table (choose lowest index that's not best_idx),
    # and assign SigTest name to best_idx. If more than 2 tables share title, assign base names to others.
    # Ensure names are unique using name_counts
    # assign base names to non-best tables
    for idx in sorted(indices):
        if idx == best_idx:
            continue
        prev = name_counts.get(base, 0)
        if prev == 0:
            assigned = base
            name_counts[base] = 1
        else:
            assigned = f"{base}_{prev}"
            name_counts[base] = prev + 1
        table_names[idx] = assigned

    # assign SigTest name for best_idx (ensure uniqueness)
    sig_base = f"{base}_SigTestTable"
    prev_sig = name_counts.get(sig_base, 0)
    if prev_sig == 0:
        assigned_sig = sig_base
        name_counts[sig_base] = 1
    else:
        assigned_sig = f"{sig_base}_{prev_sig}"
        name_counts[sig_base] = prev_sig + 1
    table_names[best_idx] = assigned_sig

# 4) Sanity: any leftover indices not assigned? (shouldn't happen) assign defaults
for i in range(len(cleaned_tables)):
    if i not in table_names:
        raw = table_titles.get(i) or f"table_{i}"
        s = safe_name(raw) or f"table_{i}"
        prev = name_counts.get(s, 0)
        if prev == 0:
            table_names[i] = s
            name_counts[s] = 1
        else:
            table_names[i] = f"{s}_{prev}"
            name_counts[s] = prev + 1

# 5) build dictionary name -> dataframe
tables_by_name = { table_names[i]: cleaned_tables[i] for i in range(len(cleaned_tables)) }

# 6) print summary
print("Title extraction (from first cell of second row) and assigned names:")
for i in range(len(cleaned_tables)):
    t = table_titles.get(i)
    mark = "(SigTest)" if re.search(r'_SigTestTable(?:_\d+)?$', table_names[i]) else ""
    print(f" - [{i}] title={repr(t)} -> name={table_names[i]} {mark}")

# example access
print("\nExample table access by name (first key):")
if tables_by_name:
    k = next(iter(tables_by_name.keys()))
    print("First key:", k)
    display(tables_by_name[k].head(5))
else:
    print("No tables available.")


Title extraction (from first cell of second row) and assigned names:
 - [0] title='S5. Which category do you fall under?' -> name=S5. Which category do you fall under 
 - [1] title='S5. Which category do you fall under?' -> name=S5. Which category do you fall under_SigTestTable (SigTest)
 - [2] title='B1_Stance' -> name=B1_Stance 
 - [3] title='B1_Stance' -> name=B1_Stance_SigTestTable (SigTest)
 - [4] title='$A1.Score' -> name=$A1.Score 
 - [5] title='$A1.Score' -> name=$A1.Score_SigTestTable (SigTest)

Example table access by name (first key):
First key: S5. Which category do you fall under


,,_1,Total,_2,Total_1,_3,Local Youths,_4,Non-Local Youths,_5,...,I feel fearful for no apparent reason,_102,I've been feeling optimistic about the future,_103,I've been dealing well with my problems,_104,I act without stopping to think,_105,I enjoy new and exciting experiences even if they are unusual,_106
0,NaN,NaN,Count,Column N %,Count,Column N %,Count,Column N %,Count,Column N %,...,Count,Column N %,Count,Column N %,Count,Column N %,Count,Column N %,Count,Column N %
1,S5. Which category do you fall under?,Total,4805,1,4805,1,3005,1,302,1,...,472,1,3072,1,3379,1,488,1,2265,1
2,NaN,Secondary School/ IP Junior High Student,658,0.137,658,0.137,624,0.208,34,0.113,...,82,0.173,343,0.112,390,0.116,82,0.168,337,0.149
3,NaN,JC/MI/ IB/ IP Senior High Student,101,0.021,101,0.021,95,0.032,6,0.02,...,10,0.022,60,0.019,69,0.02,12,0.025,54,0.024
4,NaN,Polytechnic Student,305,0.063,305,0.063,296,0.098,9,0.03,...,47,0.1,175,0.057,204,0.06,35,0.072,183,0.081


### Construct Table Name Dictionary and Index Mapping

In [11]:
import re
from IPython.display import display

def _get_raw_title_from_cleaned_df(df):
    """Return raw title from first cell of second row if present; else empty string."""
    try:
        if df.shape[0] > 1 and df.shape[1] > 0:
            return _normalize_cell(df.iloc[1, 0]) or ""
    except Exception:
        pass
    return ""

# Build table_name_dict from cleaned_tables (the latest cleaned state)
table_name_dict = {}

# Ensure table_names exists or create a fallback mapping
have_table_names = 'table_names' in globals() and isinstance(table_names, dict)

for idx, df in enumerate(cleaned_tables):
    # prefer an existing raw_titles entry if available, otherwise extract from cleaned df
    if 'raw_titles' in globals() and isinstance(raw_titles, dict) and raw_titles.get(idx):
        raw_title_val = raw_titles[idx]
    else:
        raw_title_val = _get_raw_title_from_cleaned_df(df)

    # prefer existing assigned name if available, otherwise fallback to safe_name(raw_title) or generic
    if have_table_names and idx in table_names:
        assigned_name = table_names[idx]
    else:
        assigned_name = safe_name(raw_title_val) or f"table_{idx}"

    is_sig = bool(re.search(r'_SigTestTable(?:_\d+)?$', assigned_name))

    table_name_dict[idx] = {
        'name': assigned_name,
        'is_sigtest': is_sig,
        'raw_title': raw_title_val or '',
        # store a copy to avoid accidental in-place modifications later
        'dataframe': df.copy()
    }

# Quick summary printout + display heads
print("Table Name Dictionary (based on cleaned_tables):")
print("="*60)
for idx, info in table_name_dict.items():
    print(f"Table {idx}:")
    print(f"  Name: {info['name']}")
    print(f"  Is SigTest: {info['is_sigtest']}")
    print(f"  Raw Title: {info['raw_title']!r}")
    print(f"  Shape: {info['dataframe'].shape}")
    display(info['dataframe'].head(5))
    print()

# Also build a name -> dataframe mapping for easy lookup
tables_by_name = { info['name']: info['dataframe'] for info in table_name_dict.values() }

# Example access
print("Example key lookup (first name):")
if tables_by_name:
    first_key = next(iter(tables_by_name.keys()))
    print("First table name:", first_key)
    display(tables_by_name[first_key].head(3))
else:
    print("No tables found in cleaned_tables.")


Table Name Dictionary (based on cleaned_tables):
Table 0:
  Name: S5. Which category do you fall under
  Is SigTest: False
  Raw Title: 'S5. Which category do you fall under?'
  Shape: (13, 212)


,,_1,Total,_2,Total_1,_3,Local Youths,_4,Non-Local Youths,_5,...,I feel fearful for no apparent reason,_102,I've been feeling optimistic about the future,_103,I've been dealing well with my problems,_104,I act without stopping to think,_105,I enjoy new and exciting experiences even if they are unusual,_106
0,NaN,NaN,Count,Column N %,Count,Column N %,Count,Column N %,Count,Column N %,...,Count,Column N %,Count,Column N %,Count,Column N %,Count,Column N %,Count,Column N %
1,S5. Which category do you fall under?,Total,4805,1,4805,1,3005,1,302,1,...,472,1,3072,1,3379,1,488,1,2265,1
2,NaN,Secondary School/ IP Junior High Student,658,0.137,658,0.137,624,0.208,34,0.113,...,82,0.173,343,0.112,390,0.116,82,0.168,337,0.149
3,NaN,JC/MI/ IB/ IP Senior High Student,101,0.021,101,0.021,95,0.032,6,0.02,...,10,0.022,60,0.019,69,0.02,12,0.025,54,0.024
4,NaN,Polytechnic Student,305,0.063,305,0.063,296,0.098,9,0.03,...,47,0.1,175,0.057,204,0.06,35,0.072,183,0.081



Table 1:
  Name: S5. Which category do you fall under_SigTestTable
  Is SigTest: True
  Raw Title: 'S5. Which category do you fall under?'
  Shape: (12, 107)


,,_1,Total,Total_1,Local Youths,Non-Local Youths,Adults,Total_2,13 – 14 years,15 – 19 years,...,Total_16,I feel that studying is pointless for me,I get on badly with the teachers,I am content with my life,I feel lonely,I feel fearful for no apparent reason,I've been feeling optimistic about the future,I've been dealing well with my problems,I act without stopping to think,I enjoy new and exciting experiences even if they are unusual
0,NaN,NaN,(A),(A),(B),(C),(D),(A),(B),(C),...,(A),(B),(C),(D),(E),(F),(G),(H),(I),(J)
1,S5. Which category do you fall under?,Secondary School/ IP Junior High Student,.,NaN,A C,NaN,.a,NaN,A C,A,...,D G H,A D E F G H I J,A D E G H I J,NaN,NaN,D G H,NaN,NaN,G H,D G H
2,NaN,JC/MI/ IB/ IP Senior High Student,.,NaN,A,NaN,.a,D,.a,A D,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,Polytechnic Student,.,C,A C,NaN,.a,E,.a,A D E,...,NaN,NaN,NaN,NaN,NaN,A D G H,NaN,NaN,NaN,A D G H
4,NaN,ITE Student,.,NaN,A C,NaN,.a,D E,.a,A D E,...,D H,NaN,NaN,NaN,NaN,D G H,NaN,NaN,NaN,NaN



Table 2:
  Name: B1_Stance
  Is SigTest: False
  Raw Title: 'B1_Stance'
  Shape: (5, 180)


...

,,_1,Total,_2,Total_1,_3,Sec,_4,JC,_5,...,I feel fearful for no apparent reason,_80,I've been feeling optimistic about the future,_81,I've been dealing well with my problems,_82,I act without stopping to think,_83,I enjoy new and exciting experiences even if they are unusual,_84
0,NaN,NaN,Count,Column N %,Count,Column N %,Count,Column N %,Count,Column N %,...,Count,Column N %,Count,Column N %,Count,Column N %,Count,Column N %,Count,Column N %
1,$A1.Score,Total,3005.123,1,3005.123,1,613.395,1,93.03,1,...,370.5214,1,1803.3554,1,2010.1728,1,340.0788,1,1672.397,1
2,NaN,Consuming drugs can be addictive.,2842.3636,0.945839,2842.3636,0.945839,556.1448,0.906667,88.6886,0.953333,...,352.0813,0.950232,1735.338,0.962283,1923.9204,0.957092,322.8728,0.949406,1603.3934,0.95874
3,NaN,Consuming drugs will harm one’s health.,2837.9808,0.944381,2837.9808,0.944381,580.6806,0.946667,86.5179,0.93,...,342.9147,0.925492,1740.0578,0.9649,1926.0516,0.958152,315.5831,0.927971,1597.6966,0.955333
4,NaN,Consuming drugs affects one’s life negatively.,2773.5828,0.922952,2773.5828,0.922952,560.2341,0.913333,84.9674,0.913333,...,340.1677,0.918078,1715.7353,0.951413,1894.2273,0.942321,318.7864,0.93739,1578.0146,0.943565



Table 5:
  Name: $A1.Score_SigTestTable
  Is SigTest: True
  Raw Title: '$A1.Score'
  Shape: (22, 85)


,,_1,Total,Total_1,Sec,JC,Poly,ITE,Local Uni,PEI,...,Total_14,I feel that studying is pointless for me,I get on badly with the teachers,I am content with my life,I feel lonely,I feel fearful for no apparent reason,I've been feeling optimistic about the future,I've been dealing well with my problems,I act without stopping to think,I enjoy new and exciting experiences even if they are unusual
0,NaN,NaN,(A),(A),(B),(C),(D),(E),(F),(G),...,(A),(B),(C),(D),(E),(F),(G),(H),(I),(J)
1,$A1.Score,Consuming drugs can be addictive.,.,B,NaN,NaN,B,NaN,B,NaN,...,NaN,NaN,NaN,A B,NaN,NaN,A B,A B,NaN,A B
2,NaN,Consuming drugs will harm one’s health.,.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,A,NaN,NaN,A C E F I,A,NaN,NaN
3,NaN,Consuming drugs affects one’s life negatively.,.,NaN,NaN,NaN,J,NaN,NaN,NaN,...,NaN,NaN,NaN,A C E,NaN,NaN,A C E,A C E,NaN,A C E
4,NaN,Consuming drugs affects families negatively.,.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,C,NaN,NaN,A C E H I,NaN,C,A C,A C,NaN,A C



Example key lookup (first name):
First table name: S5. Which category do you fall under


,,_1,Total,_2,Total_1,_3,Local Youths,_4,Non-Local Youths,_5,...,I feel fearful for no apparent reason,_102,I've been feeling optimistic about the future,_103,I've been dealing well with my problems,_104,I act without stopping to think,_105,I enjoy new and exciting experiences even if they are unusual,_106
0,NaN,NaN,Count,Column N %,Count,Column N %,Count,Column N %,Count,Column N %,...,Count,Column N %,Count,Column N %,Count,Column N %,Count,Column N %,Count,Column N %
1,S5. Which category do you fall under?,Total,4805,1,4805,1,3005,1,302,1,...,472,1,3072,1,3379,1,488,1,2265,1
2,NaN,Secondary School/ IP Junior High Student,658,0.137,658,0.137,624,0.208,34,0.113,...,82,0.173,343,0.112,390,0.116,82,0.168,337,0.149


In [12]:
import pandas as pd

# Drop column index 0 from cleaned_tables
new_cleaned = []
for i, df in enumerate(cleaned_tables):
    df2 = df.copy()
    if df2.shape[1] > 0:             # only drop if column 0 exists
        df2.drop(df2.columns[0], axis=1, inplace=True)
    new_cleaned.append(df2)

cleaned_tables = new_cleaned  # overwrite

# Update table_name_dict with the dropped-column version
for idx, info in table_name_dict.items():
    df = info['dataframe']
    df2 = df.copy()
    if df2.shape[1] > 0:
        df2.drop(df2.columns[0], axis=1, inplace=True)
    table_name_dict[idx]['dataframe'] = df2

print("Column 0 dropped from cleaned_tables and table_name_dict.")


Column 0 dropped from cleaned_tables and table_name_dict.


In [13]:
import pandas as pd

# New dictionary that will store the mapping
sigtest_row0_column_mapping = {}

for idx, info in table_name_dict.items():
    
    if not info["is_sigtest"]:
        continue   # Only process SigTest tables
    
    df = info["dataframe"]
    table_name = info["name"]
    
    # Require that row 0 exists
    if df.shape[0] == 0:
        sigtest_row0_column_mapping[table_name] = {}
        continue
    
    row0 = df.iloc[0]
    
    mapping = {}
    
    for col in df.columns:
        val = row0[col]
        val_str = str(val).strip() if pd.notna(val) else "NaN"
        
        mapping[col] = val_str
    
    sigtest_row0_column_mapping[table_name] = mapping

# Pretty print the mapping
for tname, mp in sigtest_row0_column_mapping.items():
    print("\n" + "="*80)
    print("SigTest Table:", tname)
    print("="*80)
    for col, meaning in mp.items():
        print(f"{col:25} → {meaning}")



SigTest Table: S5. Which category do you fall under_SigTestTable
_1                        → NaN
Total                     → (A)
Total_1                   → (A)
Local Youths              → (B)
Non-Local Youths          → (C)
Adults                    → (D)
Total_2                   → (A)
13 – 14 years             → (B)
15 – 19 years             → (C)
20 – 24 years             → (D)
25 – 29 years             → (E)
30 – 34 years             → (F)
35 – 39 years             → (G)
40 – 44 years             → (H)
45 – 49 years             → (I)
50 – 54 years             → (J)
55 – 59 years             → (K)
60 – 64 years             → (L)
65 – 69 years             → (M)
Above 69 years            → (N)
Total_3                   → (A)
Male                      → (B)
Female                    → (C)
Total_4                   → (A)
Singapore Citizen         → (B)
Singapore Permanent Resident (PR) → (C)
Others                    → (D)
Total_5                   → (A)
Chinese                   → (B

In [14]:
import re
import pandas as pd
from IPython.display import display

def _sanitize_suffix(s):
    """Make a short safe suffix from a cell value for use in column names.

    Keeps letters, numbers, underscore, dash, percent sign, dot, and parentheses.
    Collapses whitespace to underscores and truncates long strings.
    """
    if s is None:
        return ""
    t = str(s).strip()
    if t == "":
        return ""
    # collapse whitespace into single underscore
    t = re.sub(r'\s+', '_', t)
    # allow letters, digits, underscore, dash, percent, dot, parentheses
    t = re.sub(r'[^\w\-%\.\(\)]', '', t)
    # trim long suffixes
    if len(t) > 40:
        t = t[:37] + "..."
    return t

# New dict to hold the updated dataframes (copy originals)
updated_tables_by_name = {}

# Iterate over table_name_dict and update only non-SigTest tables
for idx, info in table_name_dict.items():
    table_name = info['name']
    df = info['dataframe'].copy()

    # If SigTest, keep unchanged (but still copy)
    if info.get("is_sigtest", False):
        updated_tables_by_name[table_name] = df.copy()
        continue

    # If empty or no columns, just store as-is
    if df.shape[1] == 0:
        updated_tables_by_name[table_name] = df
        table_name_dict[idx]['dataframe'] = df
        continue

    # read row0 (main header row) if present; otherwise, use empty placeholders
    row0 = df.iloc[0] if df.shape[0] > 0 else pd.Series([None] * df.shape[1], index=df.columns)

    orig_cols = list(df.columns)
    new_cols = []
    for i, col in enumerate(orig_cols):
        base = col
        # If column looks like "_1", "_2", ... then take the left (previous) column as base when possible
        if re.fullmatch(r'_+\d+', str(col)) and i > 0:
            base = orig_cols[i-1]

        # get suffix from row0 cell
        cell_val = row0.iloc[i] if i < len(row0) else None
        suffix = _sanitize_suffix(cell_val)

        if suffix:
            new_name = f"{base}_{suffix}"
        else:
            new_name = str(base)

        new_cols.append(new_name)

    # make unique (append suffix _1, _2 if duplicates)
    seen = {}
    unique_cols = []
    for name in new_cols:
        if name not in seen:
            seen[name] = 0
            unique_cols.append(name)
        else:
            seen[name] += 1
            unique_cols.append(f"{name}_{seen[name]}")

    # assign new column names
    df.columns = unique_cols

    # store updated df under its assigned name
    updated_tables_by_name[table_name] = df

    # also update table_name_dict so its 'dataframe' reflects the change
    table_name_dict[idx]['dataframe'] = df

# rebuild tables_by_name from updated mapping (so downstream code uses the new names)
tables_by_name = updated_tables_by_name

# quick verification: print a small summary and display heads for first few non-sigtest tables
print("Updated column names for non-SigTest tables (showing first 10 columns):\n")
count = 0
for name, df in tables_by_name.items():
    # limit output so it doesn't flood the notebook
    if count >= 8:
        break
    print(f"Table: {name}  — shape: {df.shape}")
    display(df.iloc[:, :10].head(3))
    print("-" * 60)
    count += 1

# `table_name_dict` and `tables_by_name` now contain the updated dataframes.


Updated column names for non-SigTest tables (showing first 10 columns):

Table: S5. Which category do you fall under  — shape: (13, 211)


,_1_nan,Total_Count,Total_Column_N_%,Total_1_Count,Total_1_Column_N_%,Local Youths_Count,Local Youths_Column_N_%,Non-Local Youths_Count,Non-Local Youths_Column_N_%,Adults_Count
0,NaN,Count,Column N %,Count,Column N %,Count,Column N %,Count,Column N %,Count
1,Total,4805,1,4805,1,3005,1,302,1,1498
2,Secondary School/ IP Junior High Student,658,0.137,658,0.137,624,0.208,34,0.113,0


------------------------------------------------------------
Table: S5. Which category do you fall under_SigTestTable  — shape: (12, 106)


,_1,Total,Total_1,Local Youths,Non-Local Youths,Adults,Total_2,13 – 14 years,15 – 19 years,20 – 24 years
0,NaN,(A),(A),(B),(C),(D),(A),(B),(C),(D)
1,Secondary School/ IP Junior High Student,.,NaN,A C,NaN,.a,NaN,A C,A,.a
2,JC/MI/ IB/ IP Senior High Student,.,NaN,A,NaN,.a,D,.a,A D,NaN


------------------------------------------------------------
Table: B1_Stance  — shape: (5, 179)


...

------------------------------------------------------------
Table: $A1.Score  — shape: (23, 167)


,_1_nan,Total_Count,Total_Column_N_%,Total_1_Count,Total_1_Column_N_%,Sec_Count,Sec_Column_N_%,JC_Count,JC_Column_N_%,Poly_Count
0,NaN,Count,Column N %,Count,Column N %,Count,Column N %,Count,Column N %,Count
1,Total,3005.123,1,3005.123,1,613.395,1,93.03,1,292.775
2,Consuming drugs can be addictive.,2842.3636,0.945839,2842.3636,0.945839,556.1448,0.906667,88.6886,0.953333,285.2465


------------------------------------------------------------
Table: $A1.Score_SigTestTable  — shape: (22, 84)


,_1,Total,Total_1,Sec,JC,Poly,ITE,Local Uni,PEI,NSF
0,NaN,(A),(A),(B),(C),(D),(E),(F),(G),(H)
1,Consuming drugs can be addictive.,.,B,NaN,NaN,B,NaN,B,NaN,NaN
2,Consuming drugs will harm one’s health.,.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


------------------------------------------------------------


In [15]:
# Drop row 0 for non-SigTest tables and update table_name_dict + tables_by_name

for idx, info in table_name_dict.items():
    if info["is_sigtest"]:
        # leave SigTest tables untouched
        continue

    df = info["dataframe"].copy()

    if df.shape[0] > 0:
        df = df.iloc[1:].reset_index(drop=True)

    # update dictionary
    table_name_dict[idx]["dataframe"] = df
    tables_by_name[info["name"]] = df


In [16]:
print("\n=== DISPLAYING NON-SIGTEST TABLES (updated) ===\n")

for idx, info in table_name_dict.items():
    if info["is_sigtest"]:
        continue  # skip SigTest tables

    name = info["name"]
    df = info["dataframe"]

    print(f"\n{name}  —  shape: {df.shape}")
    print("-" * 80)
    display(df.head(5))   # show first 10 rows



=== DISPLAYING NON-SIGTEST TABLES (updated) ===


S5. Which category do you fall under  —  shape: (12, 211)
--------------------------------------------------------------------------------


,_1_nan,Total_Count,Total_Column_N_%,Total_1_Count,Total_1_Column_N_%,Local Youths_Count,Local Youths_Column_N_%,Non-Local Youths_Count,Non-Local Youths_Column_N_%,Adults_Count,...,I feel fearful for no apparent reason_Count,I feel fearful for no apparent reason_Column_N_%,I've been feeling optimistic about the future_Count,I've been feeling optimistic about the future_Column_N_%,I've been dealing well with my problems_Count,I've been dealing well with my problems_Column_N_%,I act without stopping to think_Count,I act without stopping to think_Column_N_%,I enjoy new and exciting experiences even if they are unusual_Count,I enjoy new and exciting experiences even if they are unusual_Column_N_%
0,Total,4805,1,4805,1,3005,1,302,1,1498,...,472,1,3072,1,3379,1,488,1,2265,1
1,Secondary School/ IP Junior High Student,658,0.137,658,0.137,624,0.208,34,0.113,0,...,82,0.173,343,0.112,390,0.116,82,0.168,337,0.149
2,JC/MI/ IB/ IP Senior High Student,101,0.021,101,0.021,95,0.032,6,0.02,0,...,10,0.022,60,0.019,69,0.02,12,0.025,54,0.024
3,Polytechnic Student,305,0.063,305,0.063,296,0.098,9,0.03,0,...,47,0.1,175,0.057,204,0.06,35,0.072,183,0.081
4,ITE Student,111,0.023,111,0.023,109,0.036,2,0.007,0,...,19,0.041,57,0.018,59,0.018,16,0.033,52,0.023



B1_Stance  —  shape: (4, 179)
--------------------------------------------------------------------------------


,_1_nan,Total_Count,Total_Column_N_%,Total_1_Count,Total_1_Column_N_%,30-34 years_Count,30-34 years_Column_N_%,35-39 years_Count,35-39 years_Column_N_%,40-44 years_Count,...,I feel fearful for no apparent reason_Count,I feel fearful for no apparent reason_Column_N_%,I've been feeling optimistic about the future_Count,I've been feeling optimistic about the future_Column_N_%,I've been dealing well with my problems_Count,I've been dealing well with my problems_Column_N_%,I act without stopping to think_Count,I act without stopping to think_Column_N_%,I enjoy new and exciting experiences even if they are unusual_Count,I enjoy new and exciting experiences even if they are unusual_Column_N_%
0,Total,1498,1,1498,1,166,1,163,1,161,...,71,1,1064,1,1183,1,110,1,462,1
1,Conservative,1449,0.967,1449,0.967,156,0.937,154,0.945,158,...,69,0.974,1041,0.979,1161,0.982,105,0.961,444,0.962
2,Neutral,42,0.028,42,0.028,9,0.055,8,0.05,3,...,2,0.026,18,0.017,19,0.016,2,0.018,11,0.025
3,Liberal,7,0.005,7,0.005,1,0.008,1,0.005,0,...,0,0,4,0.004,3,0.002,2,0.021,6,0.013



$A1.Score  —  shape: (22, 167)
--------------------------------------------------------------------------------


,_1_nan,Total_Count,Total_Column_N_%,Total_1_Count,Total_1_Column_N_%,Sec_Count,Sec_Column_N_%,JC_Count,JC_Column_N_%,Poly_Count,...,I feel fearful for no apparent reason_Count,I feel fearful for no apparent reason_Column_N_%,I've been feeling optimistic about the future_Count,I've been feeling optimistic about the future_Column_N_%,I've been dealing well with my problems_Count,I've been dealing well with my problems_Column_N_%,I act without stopping to think_Count,I act without stopping to think_Column_N_%,I enjoy new and exciting experiences even if they are unusual_Count,I enjoy new and exciting experiences even if they are unusual_Column_N_%
0,Total,3005.123,1,3005.123,1,613.395,1,93.03,1,292.775,...,370.5214,1,1803.3554,1,2010.1728,1,340.0788,1,1672.397,1
1,Consuming drugs can be addictive.,2842.3636,0.945839,2842.3636,0.945839,556.1448,0.906667,88.6886,0.953333,285.2465,...,352.0813,0.950232,1735.338,0.962283,1923.9204,0.957092,322.8728,0.949406,1603.3934,0.95874
2,Consuming drugs will harm one’s health.,2837.9808,0.944381,2837.9808,0.944381,580.6806,0.946667,86.5179,0.93,282.737,...,342.9147,0.925492,1740.0578,0.9649,1926.0516,0.958152,315.5831,0.927971,1597.6966,0.955333
3,Consuming drugs affects one’s life negatively.,2773.5828,0.922952,2773.5828,0.922952,560.2341,0.913333,84.9674,0.913333,281.9005,...,340.1677,0.918078,1715.7353,0.951413,1894.2273,0.942321,318.7864,0.93739,1578.0146,0.943565
4,Consuming drugs affects families negatively.,2859.0097,0.951379,2859.0097,0.951379,572.502,0.933333,86.2078,0.926667,284.41,...,353.2777,0.953461,1748.6381,0.969658,1934.4309,0.962321,319.817,0.94042,1619.7082,0.968495


In [17]:
import re
from collections import defaultdict
import pandas as pd
from IPython.display import display

# Result container
total_column_capital_cells = {}

if 'table_name_dict' not in globals():
    raise RuntimeError("table_name_dict not found — run the earlier cell that builds it first.")

# helper: extract contiguous uppercase-letter tokens from a string
def extract_upper_tokens(s):
    if s is None:
        return []
    s_str = str(s)
    # find runs of uppercase letters
    tokens = re.findall(r'[A-Z]+', s_str)
    return tokens

# scanning
for idx, info in table_name_dict.items():
    if not info.get('is_sigtest', False):
        continue

    table_name = info.get('name', f"table_{idx}")
    df = info.get('dataframe')
    if df is None or df.shape[0] == 0 or df.shape[1] == 0:
        # empty table
        total_column_capital_cells[table_name] = {}
        continue

    # find columns whose name contains 'total' (case-insensitive)
    total_cols = [col for col in df.columns if isinstance(col, str) and 'total' in col.lower()]

    if not total_cols:
        total_column_capital_cells[table_name] = {}
        continue

    col_matches = {}

    for col in total_cols:
        matches = []
        for r_idx in range(df.shape[0]):
            val = df.iloc[r_idx][col]
            if pd.isna(val):
                continue
            s = str(val).strip()

            # match rule: has at least one uppercase A-Z and no lowercase a-z
            if re.search(r'[A-Z]', s) and not re.search(r'[a-z]', s):
                tokens = extract_upper_tokens(s)  # e.g. ["A"], ["ACE"], ["A","B"]
                if tokens:
                    matches.append({
                        'row_index': r_idx,
                        'raw_value': s,
                        'extracted_tokens': tokens,
                        'extracted_joined': "/".join(tokens)
                    })
        col_matches[col] = matches

    total_column_capital_cells[table_name] = col_matches

# Print a readable summary
print("Summary of capital-letter cells found in 'total' columns of SigTest tables:\n")
for tname, cols in total_column_capital_cells.items():
    print("="*80)
    print("Table:", tname)
    if not cols:
        print("  (no 'total' columns or no matches found)")
        continue
    for colname, matches in cols.items():
        print(f"  Column: {colname!r} — {len(matches)} match(es)")
        for m in matches:
            print(f"    row {m['row_index']:>3}: {m['raw_value']!r}  -> tokens: {m['extracted_joined']}")
    print()

# Example: the full mapping is in variable `total_column_capital_cells`
# You can inspect a specific table like:
# total_column_capital_cells['MyTableName']['Total']


Summary of capital-letter cells found in 'total' columns of SigTest tables:

Table: S5. Which category do you fall under_SigTestTable
  Column: 'Total' — 1 match(es)
    row   0: '(A)'  -> tokens: A
  Column: 'Total_1' — 6 match(es)
    row   0: '(A)'  -> tokens: A
    row   3: 'C'  -> tokens: C
    row   6: 'B'  -> tokens: B
    row   8: 'B C'  -> tokens: B/C
    row   9: 'B'  -> tokens: B
    row  10: 'D'  -> tokens: D
  Column: 'Total_2' — 11 match(es)
    row   0: '(A)'  -> tokens: A
    row   2: 'D'  -> tokens: D
    row   3: 'E'  -> tokens: E
    row   4: 'D E'  -> tokens: D/E
    row   5: 'C'  -> tokens: C
    row   6: 'B E'  -> tokens: B/E
    row   7: 'E'  -> tokens: E
    row   8: 'C D N'  -> tokens: C/D/N
    row   9: 'C D E'  -> tokens: C/D/E
    row  10: 'E F G H I J K'  -> tokens: E/F/G/H/I/J/K
    row  11: 'E'  -> tokens: E
  Column: 'Total_3' — 4 match(es)
    row   0: '(A)'  -> tokens: A
    row   5: 'B'  -> tokens: B
    row   9: 'B'  -> tokens: B
    row  11: 'B'  ->

In [18]:
# --- Add row_name (value in column 0) to each token match in total_column_capital_cells ---

for tname, cols in total_column_capital_cells.items():

    # find dataframe for this table name
    df = None
    for idx, info in table_name_dict.items():
        if info["name"] == tname:
            df = info["dataframe"]
            break

    if df is None:
        continue  # safety guard

    # requires column 0 to exist
    if df.shape[1] == 0:
        continue

    for colname, matches in cols.items():
        for m in matches:
            r = m["row_index"]

            # check row within bounds
            if r < df.shape[0]:
                try:
                    row_name = df.iloc[r, 0]    # column 0 contains row label
                except:
                    row_name = None
            else:
                row_name = None

            # clean formatting
            if pd.isna(row_name):
                row_name = None
            else:
                row_name = str(row_name).strip()

            # store it
            m["row_name"] = row_name

print("\n=== Updated Summary with Row Names Included ===\n")
for tname, cols in total_column_capital_cells.items():
    print("="*80)
    print("TABLE:", tname)

    for colname, matches in cols.items():
        if not matches:
            print(f"  Column {colname}: (no matches)")
            continue
        
        print(f"\n  Column {colname}:")
        for m in matches:
            print(f"    row {m['row_index']:>3} ({m['row_name']}): {m['raw_value']!r} -> tokens {m['extracted_joined']}")



=== Updated Summary with Row Names Included ===

TABLE: S5. Which category do you fall under_SigTestTable

  Column Total:
    row   0 (None): '(A)' -> tokens A

  Column Total_1:
    row   0 (None): '(A)' -> tokens A
    row   3 (Polytechnic Student): 'C' -> tokens C
    row   6 (Private Education Institute Student): 'B' -> tokens B
    row   8 (Currently holding a full-time/ part-time job): 'B C' -> tokens B/C
    row   9 (Currently not schooling or working): 'B' -> tokens B
    row  10 (Net Not Working): 'D' -> tokens D

  Column Total_2:
    row   0 (None): '(A)' -> tokens A
    row   2 (JC/MI/ IB/ IP Senior High Student): 'D' -> tokens D
    row   3 (Polytechnic Student): 'E' -> tokens E
    row   4 (ITE Student): 'D E' -> tokens D/E
    row   5 (Local University Student (i.e., NUS/NTU/ SMU/ SUTD/ SIT/ SUSS)): 'C' -> tokens C
    row   6 (Private Education Institute Student): 'B E' -> tokens B/E
    row   7 (Full-time National Servicemen (NSF)): 'E' -> tokens E
    row   8 (Curre

In [19]:
# UPDATED: rebuild variables and create annotated_summary (block-aware mapping per Total_* column)
import re
import pandas as pd
from copy import deepcopy

# ---------- sanity checks ----------
if 'table_name_dict' not in globals():
    raise RuntimeError("table_name_dict not found. Run the earlier cell that builds it first (based on cleaned_tables).")

# ---------- helpers ----------
def extract_upper_tokens(s):
    """Return contiguous uppercase-letter runs found in a cell (e.g. '(A)' -> ['A'], 'B C' -> ['B','C'])."""
    if s is None:
        return []
    s_str = str(s)
    return re.findall(r'[A-Z]+', s_str)

def _strip_parens_and_space(s):
    """Normalize row0 label like '(A)' or ' A ' -> 'A' (uppercase)."""
    if s is None:
        return None
    s = str(s).strip()
    if s == "" or s.lower() in {"nan", "none", "n/a"}:
        return None
    s2 = re.sub(r'[^\w%]', '', s)  # remove punctuation except % if any
    if s2 == "":
        return None
    return s2.upper()

# ---------- 1) Build total_column_capital_cells (if missing) ----------
total_column_capital_cells = {}

for idx, info in table_name_dict.items():
    if not info.get('is_sigtest', False):
        continue
    tname = info.get('name', f"table_{idx}")
    df = info.get('dataframe')
    if df is None or df.shape[0] == 0 or df.shape[1] == 0:
        total_column_capital_cells[tname] = {}
        continue

    # find columns with 'total' in the name (case-insensitive)
    total_cols = [col for col in df.columns if isinstance(col, str) and 'total' in col.lower()]
    col_matches = {}

    for col in total_cols:
        matches = []
        for r_idx in range(df.shape[0]):
            try:
                val = df.iat[r_idx, df.columns.get_loc(col)]
            except Exception:
                val = None
            if pd.isna(val):
                continue
            s = str(val).strip()
            # keep only if there's at least one uppercase A-Z and no lowercase a-z
            if re.search(r'[A-Z]', s) and not re.search(r'[a-z]', s):
                tokens = extract_upper_tokens(s)
                if tokens:
                    matches.append({
                        'row_index': r_idx,
                        'raw_value': s,
                        'extracted_tokens': tokens,
                        'extracted_joined': "/".join(tokens)
                    })
        col_matches[col] = matches

    total_column_capital_cells[tname] = col_matches

print("Built total_column_capital_cells for SigTest tables (keys):", list(total_column_capital_cells.keys()))

# ---------- 2) Remove row-0 matches -> total_column_capital_cells_no_row0 ----------
total_column_capital_cells_no_row0 = deepcopy(total_column_capital_cells)
for tname, cols in list(total_column_capital_cells_no_row0.items()):
    if not isinstance(cols, dict):
        continue
    for colname, matches in list(cols.items()):
        new_matches = [m for m in matches if m.get('row_index') != 0]
        total_column_capital_cells_no_row0[tname][colname] = new_matches

print("Created total_column_capital_cells_no_row0 (row 0 entries removed).")

# ---------- 3) Ensure sigtest_row0_column_mapping exists (rebuild if missing) ----------
sigtest_row0_column_mapping = {}
for idx, info in table_name_dict.items():
    if not info.get('is_sigtest', False):
        continue
    tname = info.get('name', f"table_{idx}")
    df = info.get('dataframe')
    mapping = {}
    if df is None or df.shape[0] == 0:
        sigtest_row0_column_mapping[tname] = {}
        continue
    # row 0 mapping (if present)
    row0 = df.iloc[0] if df.shape[0] > 0 else pd.Series([None]*df.shape[1], index=df.columns)
    for col in df.columns:
        val = row0[col] if col in row0.index else None
        mapping[col] = "" if pd.isna(val) else str(val).strip()
    sigtest_row0_column_mapping[tname] = mapping

print("Built sigtest_row0_column_mapping for SigTest tables (keys):", list(sigtest_row0_column_mapping.keys()))

# ---------- Helper: find block column indices for a given total column ----------
def _get_total_block_range(df, total_col_name):
    """
    Given a sigtest df and a total column name (exact column header), return (start_idx, end_idx)
    where start_idx is index of total_col_name, and end_idx is index of next total_col (exclusive),
    or df.shape[1] if none. If total_col_name not found -> (None, None).
    """
    if df is None or total_col_name is None:
        return None, None
    cols = list(df.columns)
    try:
        start = cols.index(total_col_name)
    except ValueError:
        # try case-insensitive match
        lc = total_col_name.strip().lower()
        start = None
        for i, c in enumerate(cols):
            try:
                if isinstance(c, str) and c.strip().lower() == lc:
                    start = i
                    break
            except Exception:
                continue
        if start is None:
            # try substring
            for i, c in enumerate(cols):
                try:
                    if isinstance(c, str) and lc in c.lower():
                        start = i
                        break
                except Exception:
                    continue
    if start is None:
        return None, None

    # find next total_* column after start
    end = len(cols)
    for j in range(start + 1, len(cols)):
        c = cols[j]
        if isinstance(c, str) and 'total' in c.lower():
            end = j
            break
    return start, end

# ---------- 4) Build annotated_summary (block-aware) ----------
annotated_summary = {}

for tname, cols in total_column_capital_cells_no_row0.items():

    # get the row0-defined mapping for this table
    row0_map = sigtest_row0_column_mapping.get(tname, {})

    # retrieve dataframe for this table
    df = None
    for idx, info in table_name_dict.items():
        if info.get("name") == tname:
            df = info.get("dataframe")
            break

    annotated_summary[tname] = {}

    # precompute total columns order (for fallback)
    total_cols_in_df = [col for col in (df.columns if df is not None else []) if isinstance(col, str) and 'total' in col.lower()]

    for total_col, matches in cols.items():
        enriched_rows = []

        # figure block range for this total_col
        start_idx, end_idx = _get_total_block_range(df, total_col)

        for m in matches:
            raw = m.get("raw_value")
            row_idx = m.get("row_index")

            # extract row_name from column 0
            row_name = None
            if df is not None and df.shape[1] > 0 and isinstance(row_idx, int) and 0 <= row_idx < df.shape[0]:
                val = df.iloc[row_idx, 0]
                row_name = None if pd.isna(val) else str(val).strip()

            tokens = m.get("extracted_tokens", [])    # e.g ["C"], ["B","C"], ["E","F","G"]

            # flatten multi-letter tokens (ACE -> A,C,E)
            token_list = []
            for tok in tokens:
                tok = str(tok)
                if len(tok) > 1:
                    token_list.extend(list(tok))
                else:
                    token_list.append(tok)

            # match each letter token to the correct column name, but restrict to block
            for tok in token_list:
                tok = tok.upper()

                matched_colname = None

                # 1) If we have a block (start_idx defined), search only within that block
                if df is not None and start_idx is not None:
                    cols_in_block = list(df.columns)[start_idx:end_idx]
                    # search left-to-right inside the block for a column whose row0 mapping contains the letter
                    for colname in cols_in_block:
                        meaning = row0_map.get(colname, "")
                        letters_in_header = re.findall(r"[A-Z]", str(meaning))
                        if tok in letters_in_header:
                            matched_colname = colname
                            break

                # 2) If none found within block (or block not defined), fall back to scanning full row0_map
                if matched_colname is None:
                    for colname, meaning in row0_map.items():
                        letters_in_header = re.findall(r"[A-Z]", str(meaning))
                        if tok in letters_in_header:
                            matched_colname = colname
                            break

                # 3) As last resort, try to find by direct normalized match on column header containing token letter in parentheses
                if matched_colname is None and df is not None:
                    # example: header might contain "(C)" somewhere; prefer that header inside the block first
                    cols_to_search = list(df.columns)[start_idx:end_idx] if (df is not None and start_idx is not None) else list(df.columns)
                    for colname in cols_to_search:
                        row0_val = row0_map.get(colname, "")
                        parsed = _strip_parens_and_space(row0_val)
                        if parsed == tok:
                            matched_colname = colname
                            break

                enriched_rows.append({
                    "row_index": row_idx,
                    "row_name": row_name,
                    "raw_value": raw,
                    "token": tok,
                    "column_name": matched_colname,
                    "total_block_start_col": total_col,
                    "total_block_range": (start_idx, end_idx)
                })

        annotated_summary[tname][total_col] = enriched_rows

# ---------- 5) Print summary of annotated_summary for quick verification ----------
print("\n=== Annotated Summary (with mapped column names & row names) ===\n")
for tname, cols in annotated_summary.items():
    print("="*80)
    print("TABLE:", tname)
    print("="*80)

    for total_col, rows in cols.items():
        print(f"\n  Total Column: {total_col}")
        if not rows:
            print("    (no matches)")
            continue

        for entry in rows:
            print(
                f"    row {entry['row_index']:>3} ({entry['row_name']}) : "
                f"{entry['raw_value']!r} -> token {entry['token']} -> Column Name: {entry['column_name']} (block {entry['total_block_start_col']}, cols {entry['total_block_range']})"
            )

# expose names back to notebook globals (so downstream cells can use them)
globals()['total_column_capital_cells'] = total_column_capital_cells
globals()['total_column_capital_cells_no_row0'] = total_column_capital_cells_no_row0
globals()['sigtest_row0_column_mapping'] = sigtest_row0_column_mapping
globals()['annotated_summary'] = annotated_summary

print("\nDone. Rebuilt variables: total_column_capital_cells_no_row0, sigtest_row0_column_mapping, annotated_summary.")


Built total_column_capital_cells for SigTest tables (keys): ['S5. Which category do you fall under_SigTestTable', 'B1_Stance_SigTestTable', '$A1.Score_SigTestTable']
Created total_column_capital_cells_no_row0 (row 0 entries removed).
Built sigtest_row0_column_mapping for SigTest tables (keys): ['S5. Which category do you fall under_SigTestTable', 'B1_Stance_SigTestTable', '$A1.Score_SigTestTable']

=== Annotated Summary (with mapped column names & row names) ===

TABLE: S5. Which category do you fall under_SigTestTable

  Total Column: Total
    (no matches)

  Total Column: Total_1
    row   3 (Polytechnic Student) : 'C' -> token C -> Column Name: Non-Local Youths (block Total_1, cols (2, 6))
    row   6 (Private Education Institute Student) : 'B' -> token B -> Column Name: Local Youths (block Total_1, cols (2, 6))
    row   8 (Currently holding a full-time/ part-time job) : 'B C' -> token B -> Column Name: Local Youths (block Total_1, cols (2, 6))
    row   8 (Currently holding a ful

In [20]:
import re
import numpy as np
from IPython.display import display

PAT_TARGET = re.compile(r"_Column_N_%", flags=re.IGNORECASE)
PAT_COUNT = re.compile(r"count", flags=re.IGNORECASE)

print("Shifting row-0 'Count' values into nearest right _Column_N_% column for non-SigTest tables...\n")

changes_summary = {}

for idx, info in table_name_dict.items():
    if info.get("is_sigtest", False):
        continue  # skip SigTest tables

    tname = info.get("name")
    df = info.get("dataframe")

    # safety: ensure df is DataFrame
    if df is None or df.shape[0] == 0:
        print(f"[SKIP] {tname} — empty dataframe")
        continue

    cols = list(df.columns)
    ncols = len(cols)

    # record changes for this table
    changes = []

    # row0 values (use .iat to set/mutate)
    for i, col in enumerate(cols):
        try:
            # source cell value at row 0
            src_val = df.iat[0, i]
        except Exception:
            src_val = None

        # decide if this column is a "Count" column by header or by row0 content
        is_count_header = isinstance(col, str) and bool(PAT_COUNT.search(col))
        is_count_cell = src_val is not None and PAT_COUNT.search(str(src_val))

        if not (is_count_header or is_count_cell):
            continue  # not a candidate

        # find first column to the right whose column name matches _Column_N_%
        target_idx = None
        for j in range(i+1, ncols):
            c = cols[j]
            if isinstance(c, str) and PAT_TARGET.search(c):
                target_idx = j
                break

        if target_idx is None:
            # nothing to shift to; record and continue
            changes.append({
                "src_col_index": i,
                "src_col_name": col,
                "src_value": src_val,
                "target_col_index": None,
                "target_col_name": None,
                "action": "no_target_found"
            })
            continue

        # perform the move: put src_val into target cell (row 0), clear source cell
        try:
            old_target_val = df.iat[0, target_idx]
        except Exception:
            old_target_val = None

        df.iat[0, target_idx] = src_val
        df.iat[0, i] = np.nan  # clear original

        changes.append({
            "src_col_index": i,
            "src_col_name": col,
            "src_value": src_val,
            "target_col_index": target_idx,
            "target_col_name": cols[target_idx],
            "old_target_value": old_target_val,
            "action": "moved"
        })

    # if any changes, update the dictionary and print summary
    if changes:
        table_name_dict[idx]["dataframe"] = df
        changes_summary[tname] = changes
        print("="*90)
        print(f"TABLE: {tname}  (index {idx}) — applied {len(changes)} change(s)")
        for ch in changes:
            if ch["action"] == "moved":
                print(f"  moved row0 from col[{ch['src_col_index']}] '{ch['src_col_name']}' -> "
                      f"col[{ch['target_col_index']}] '{ch['target_col_name']}' (old target: {ch['old_target_value']!r})")
            else:
                print(f"  could not find target for col[{ch['src_col_index']}] '{ch['src_col_name']}' (value: {ch['src_value']!r})")
        print()
    else:
        print(f"[NO CHANGES] {tname}")

print("\nDone shifting values. Displaying updated non-SigTest tables (first 15 rows each):\n")

for idx, info in table_name_dict.items():
    if info.get("is_sigtest", False):
        continue
    tname = info.get("name")
    df = info.get("dataframe")
    print("="*90)
    print(f"TABLE INDEX: {idx}  |  TABLE NAME: {tname}  |  SHAPE: {df.shape}")
    print("-"*90)
    display(df.head(15))
    print("\n")


Shifting row-0 'Count' values into nearest right _Column_N_% column for non-SigTest tables...

TABLE: S5. Which category do you fall under  (index 0) — applied 105 change(s)
  moved row0 from col[1] 'Total_Count' -> col[2] 'Total_Column_N_%' (old target: 1)
  moved row0 from col[3] 'Total_1_Count' -> col[4] 'Total_1_Column_N_%' (old target: 1)
  moved row0 from col[5] 'Local Youths_Count' -> col[6] 'Local Youths_Column_N_%' (old target: 1)
  moved row0 from col[7] 'Non-Local Youths_Count' -> col[8] 'Non-Local Youths_Column_N_%' (old target: 1)
  moved row0 from col[9] 'Adults_Count' -> col[10] 'Adults_Column_N_%' (old target: 1)
  moved row0 from col[11] 'Total_2_Count' -> col[12] 'Total_2_Column_N_%' (old target: 1)
  moved row0 from col[13] '13 – 14 years_Count' -> col[14] '13 – 14 years_Column_N_%' (old target: 1)
  moved row0 from col[15] '15 – 19 years_Count' -> col[16] '15 – 19 years_Column_N_%' (old target: 1)
  moved row0 from col[17] '20 – 24 years_Count' -> col[18] '20 – 24 y

,_1_nan,Total_Count,Total_Column_N_%,Total_1_Count,Total_1_Column_N_%,Local Youths_Count,Local Youths_Column_N_%,Non-Local Youths_Count,Non-Local Youths_Column_N_%,Adults_Count,...,I feel fearful for no apparent reason_Count,I feel fearful for no apparent reason_Column_N_%,I've been feeling optimistic about the future_Count,I've been feeling optimistic about the future_Column_N_%,I've been dealing well with my problems_Count,I've been dealing well with my problems_Column_N_%,I act without stopping to think_Count,I act without stopping to think_Column_N_%,I enjoy new and exciting experiences even if they are unusual_Count,I enjoy new and exciting experiences even if they are unusual_Column_N_%
0,Total,NaN,4805,NaN,4805,NaN,3005,NaN,302,NaN,...,NaN,472,NaN,3072,NaN,3379,NaN,488,NaN,2265
1,Secondary School/ IP Junior High Student,658,0.137,658,0.137,624,0.208,34,0.113,0,...,82,0.173,343,0.112,390,0.116,82,0.168,337,0.149
2,JC/MI/ IB/ IP Senior High Student,101,0.021,101,0.021,95,0.032,6,0.02,0,...,10,0.022,60,0.019,69,0.02,12,0.025,54,0.024
3,Polytechnic Student,305,0.063,305,0.063,296,0.098,9,0.03,0,...,47,0.1,175,0.057,204,0.06,35,0.072,183,0.081
4,ITE Student,111,0.023,111,0.023,109,0.036,2,0.007,0,...,19,0.041,57,0.018,59,0.018,16,0.033,52,0.023
5,"Local University Student (i.e., NUS/NTU/ SMU/ ...",529,0.11,529,0.11,483,0.161,46,0.152,0,...,56,0.12,353,0.115,384,0.114,42,0.085,294,0.13
6,Private Education Institute Student,347,0.072,347,0.072,180,0.06,167,0.553,0,...,48,0.102,210,0.068,208,0.062,45,0.092,154,0.068
7,Full-time National Servicemen (NSF),164,0.034,164,0.034,164,0.055,0,0,0,...,24,0.05,102,0.033,112,0.033,23,0.048,97,0.043
8,Currently holding a full-time/ part-time job,2063,0.429,2063,0.429,1032,0.343,38,0.126,993,...,157,0.333,1403,0.457,1526,0.452,200,0.41,953,0.421
9,Currently not schooling or working,528,0.11,528,0.11,23,0.008,0,0,505,...,28,0.06,370,0.12,426,0.126,33,0.068,141,0.062




TABLE INDEX: 2  |  TABLE NAME: B1_Stance  |  SHAPE: (4, 179)
------------------------------------------------------------------------------------------


,_1_nan,Total_Count,Total_Column_N_%,Total_1_Count,Total_1_Column_N_%,30-34 years_Count,30-34 years_Column_N_%,35-39 years_Count,35-39 years_Column_N_%,40-44 years_Count,...,I feel fearful for no apparent reason_Count,I feel fearful for no apparent reason_Column_N_%,I've been feeling optimistic about the future_Count,I've been feeling optimistic about the future_Column_N_%,I've been dealing well with my problems_Count,I've been dealing well with my problems_Column_N_%,I act without stopping to think_Count,I act without stopping to think_Column_N_%,I enjoy new and exciting experiences even if they are unusual_Count,I enjoy new and exciting experiences even if they are unusual_Column_N_%
0,Total,NaN,1498,NaN,1498,NaN,166,NaN,163,NaN,...,NaN,71,NaN,1064,NaN,1183,NaN,110,NaN,462
1,Conservative,1449,0.967,1449,0.967,156,0.937,154,0.945,158,...,69,0.974,1041,0.979,1161,0.982,105,0.961,444,0.962
2,Neutral,42,0.028,42,0.028,9,0.055,8,0.05,3,...,2,0.026,18,0.017,19,0.016,2,0.018,11,0.025
3,Liberal,7,0.005,7,0.005,1,0.008,1,0.005,0,...,0,0,4,0.004,3,0.002,2,0.021,6,0.013




TABLE INDEX: 4  |  TABLE NAME: $A1.Score  |  SHAPE: (22, 167)
------------------------------------------------------------------------------------------


,_1_nan,Total_Count,Total_Column_N_%,Total_1_Count,Total_1_Column_N_%,Sec_Count,Sec_Column_N_%,JC_Count,JC_Column_N_%,Poly_Count,...,I feel fearful for no apparent reason_Count,I feel fearful for no apparent reason_Column_N_%,I've been feeling optimistic about the future_Count,I've been feeling optimistic about the future_Column_N_%,I've been dealing well with my problems_Count,I've been dealing well with my problems_Column_N_%,I act without stopping to think_Count,I act without stopping to think_Column_N_%,I enjoy new and exciting experiences even if they are unusual_Count,I enjoy new and exciting experiences even if they are unusual_Column_N_%
0,Total,NaN,3005.123,NaN,3005.123,NaN,613.395,NaN,93.03,NaN,...,NaN,370.5214,NaN,1803.3554,NaN,2010.1728,NaN,340.0788,NaN,1672.397
1,Consuming drugs can be addictive.,2842.3636,0.945839,2842.3636,0.945839,556.1448,0.906667,88.6886,0.953333,285.2465,...,352.0813,0.950232,1735.338,0.962283,1923.9204,0.957092,322.8728,0.949406,1603.3934,0.95874
2,Consuming drugs will harm one’s health.,2837.9808,0.944381,2837.9808,0.944381,580.6806,0.946667,86.5179,0.93,282.737,...,342.9147,0.925492,1740.0578,0.9649,1926.0516,0.958152,315.5831,0.927971,1597.6966,0.955333
3,Consuming drugs affects one’s life negatively.,2773.5828,0.922952,2773.5828,0.922952,560.2341,0.913333,84.9674,0.913333,281.9005,...,340.1677,0.918078,1715.7353,0.951413,1894.2273,0.942321,318.7864,0.93739,1578.0146,0.943565
4,Consuming drugs affects families negatively.,2859.0097,0.951379,2859.0097,0.951379,572.502,0.933333,86.2078,0.926667,284.41,...,353.2777,0.953461,1748.6381,0.969658,1934.4309,0.962321,319.817,0.94042,1619.7082,0.968495
5,Consuming drugs affects society negatively.,2715.6565,0.903676,2715.6565,0.903676,547.9662,0.893333,85.8977,0.923333,276.8815,...,333.1349,0.899098,1677.7687,0.930359,1864.1924,0.927379,308.1212,0.906029,1533.3764,0.916873
6,Consuming cannabis can be addictive.,2639.083,0.878195,2639.083,0.878195,526.1566,0.857778,86.828,0.933333,278.5545,...,320.357,0.864611,1620.5736,0.898643,1792.6989,0.891813,303.4962,0.892429,1498.0869,0.895772
7,Consuming cannabis will harm one’s health.,2590.1193,0.861901,2590.1193,0.861901,562.9603,0.917778,84.3472,0.906667,270.1895,...,312.4108,0.843165,1595.6614,0.884829,1764.6459,0.877858,301.4096,0.886293,1440.6455,0.861426
8,Drugs are more harmful than tobacco.,2358.6864,0.784888,2358.6864,0.784888,408.93,0.666667,65.4311,0.703333,235.0565,...,287.8604,0.776906,1460.9411,0.810124,1617.3646,0.80459,264.6042,0.778067,1310.3326,0.783506
9,Cannabis is more harmful than tobacco.,1914.6509,0.637129,1914.6509,0.637129,335.3226,0.546667,57.6786,0.62,195.741,...,250.67,0.676533,1213.2232,0.672759,1332.9319,0.663093,229.1876,0.673925,1063.1531,0.635706


In [21]:
import re
from IPython.display import display

PAT_COUNT = re.compile(r"count", flags=re.IGNORECASE)

print("\n=== Removing columns containing the word 'Count' from NON-SigTest tables ===\n")

for idx, info in table_name_dict.items():
    if info.get("is_sigtest", False):
        continue  # skip SigTest tables

    tname = info["name"]
    df = info["dataframe"]

    # Identify columns that contain "Count" (case-insensitive)
    cols_to_remove = [col for col in df.columns if isinstance(col, str) and PAT_COUNT.search(col)]

    if not cols_to_remove:
        print(f"[NO COUNT COLUMNS] {tname}")
        continue

    print("=" * 90)
    print(f"TABLE: {tname}")
    print(f"Removing columns: {cols_to_remove}")

    # Remove the columns
    df_new = df.drop(columns=cols_to_remove, errors="ignore")

    # Update dictionary
    table_name_dict[idx]["dataframe"] = df_new

print("\n=== Updated NON-SigTest Tables (first 10 rows each) ===\n")

for idx, info in table_name_dict.items():
    if info.get("is_sigtest", False):
        continue
    tname = info["name"]
    df = info["dataframe"]
    print("=" * 90)
    print(f"TABLE INDEX: {idx}  |  NAME: {tname}  |  SHAPE: {df.shape}")
    print("-" * 90)
    display(df.head(10))
    print()



=== Removing columns containing the word 'Count' from NON-SigTest tables ===

TABLE: S5. Which category do you fall under
Removing columns: ['Total_Count', 'Total_1_Count', 'Local Youths_Count', 'Non-Local Youths_Count', 'Adults_Count', 'Total_2_Count', '13 – 14 years_Count', '15 – 19 years_Count', '20 – 24 years_Count', '25 – 29 years_Count', '30 – 34 years_Count', '35 – 39 years_Count', '40 – 44 years_Count', '45 – 49 years_Count', '50 – 54 years_Count', '55 – 59 years_Count', '60 – 64 years_Count', '65 – 69 years_Count', 'Above 69 years_Count', 'Total_3_Count', 'Male_Count', 'Female_Count', 'Total_4_Count', 'Singapore Citizen_Count', 'Singapore Permanent Resident (PR)_Count', 'Others_Count', 'Total_5_Count', 'Chinese_Count', 'Malay_Count', 'Indian_Count', 'Others_1_Count', 'Total_6_Count', 'NET Working_Count', 'NET Non-working_Count', 'Sec_Count', 'JC_Count', 'Poly_Count', 'ITE_Count', 'Local Uni_Count', 'PEI_Count', 'NET Student_Count', 'NSF_Count', 'Total_7_Count', 'In-Risk_Count

,_1_nan,Total_Column_N_%,Total_1_Column_N_%,Local Youths_Column_N_%,Non-Local Youths_Column_N_%,Adults_Column_N_%,Total_2_Column_N_%,13 – 14 years_Column_N_%,15 – 19 years_Column_N_%,20 – 24 years_Column_N_%,...,Total_16_Column_N_%,I feel that studying is pointless for me_Column_N_%,I get on badly with the teachers_Column_N_%,I am content with my life_Column_N_%,I feel lonely_Column_N_%,I feel fearful for no apparent reason_Column_N_%,I've been feeling optimistic about the future_Column_N_%,I've been dealing well with my problems_Column_N_%,I act without stopping to think_Column_N_%,I enjoy new and exciting experiences even if they are unusual_Column_N_%
0,Total,4805,4805,3005,302,1498,4805,318,1082,1157,...,4805,262,154,3438,351,472,3072,3379,488,2265
1,Secondary School/ IP Junior High Student,0.137,0.137,0.208,0.113,0,0.137,0.992,0.317,0,...,0.137,0.266,0.278,0.122,0.167,0.173,0.112,0.116,0.168,0.149
2,JC/MI/ IB/ IP Senior High Student,0.021,0.021,0.032,0.02,0,0.021,0,0.093,0,...,0.021,0.026,0.042,0.019,0.03,0.022,0.019,0.02,0.025,0.024
3,Polytechnic Student,0.063,0.063,0.098,0.03,0,0.063,0,0.225,0.05,...,0.063,0.061,0.061,0.059,0.089,0.1,0.057,0.06,0.072,0.081
4,ITE Student,0.023,0.023,0.036,0.007,0,0.023,0,0.087,0.013,...,0.023,0.041,0.031,0.018,0.036,0.041,0.018,0.018,0.033,0.023
5,"Local University Student (i.e., NUS/NTU/ SMU/ ...",0.11,0.11,0.161,0.152,0,0.11,0,0.057,0.343,...,0.11,0.063,0.075,0.118,0.13,0.12,0.115,0.114,0.085,0.13
6,Private Education Institute Student,0.072,0.072,0.06,0.553,0,0.072,0.008,0.107,0.172,...,0.072,0.098,0.118,0.067,0.105,0.102,0.068,0.062,0.092,0.068
7,Full-time National Servicemen (NSF),0.034,0.034,0.055,0,0,0.034,0,0.043,0.1,...,0.034,0.048,0.051,0.033,0.045,0.05,0.033,0.033,0.048,0.043
8,Currently holding a full-time/ part-time job,0.429,0.429,0.343,0.126,0.663,0.429,0,0.067,0.314,...,0.429,0.383,0.337,0.438,0.349,0.333,0.457,0.452,0.41,0.421
9,Currently not schooling or working,0.11,0.11,0.008,0,0.337,0.11,0,0.004,0.009,...,0.11,0.015,0.006,0.125,0.05,0.06,0.12,0.126,0.068,0.062



TABLE INDEX: 2  |  NAME: B1_Stance  |  SHAPE: (4, 90)
------------------------------------------------------------------------------------------


,_1_nan,Total_Column_N_%,Total_1_Column_N_%,30-34 years_Column_N_%,35-39 years_Column_N_%,40-44 years_Column_N_%,45-49 years_Column_N_%,50-54 years_Column_N_%,55-59 years_Column_N_%,60-64 years_Column_N_%,...,Total_14_Column_N_%,I feel that studying is pointless for me_Column_N_%,I get on badly with the teachers_Column_N_%,I am content with my life_Column_N_%,I feel lonely_Column_N_%,I feel fearful for no apparent reason_Column_N_%,I've been feeling optimistic about the future_Column_N_%,I've been dealing well with my problems_Column_N_%,I act without stopping to think_Column_N_%,I enjoy new and exciting experiences even if they are unusual_Column_N_%
0,Total,1498,1498,166,163,161,156,159,152,152,...,1498,0,0,1223,55,71,1064,1183,110,462
1,Conservative,0.967,0.967,0.937,0.945,0.983,0.964,0.945,0.969,0.976,...,0.967,0,0,0.98,0.932,0.974,0.979,0.982,0.961,0.962
2,Neutral,0.028,0.028,0.055,0.05,0.017,0.026,0.045,0.031,0.015,...,0.028,0,0,0.016,0.028,0.026,0.017,0.016,0.018,0.025
3,Liberal,0.005,0.005,0.008,0.005,0,0.009,0.009,0,0.009,...,0.005,0,0,0.004,0.04,0,0.004,0.002,0.021,0.013



TABLE INDEX: 4  |  NAME: $A1.Score  |  SHAPE: (22, 84)
------------------------------------------------------------------------------------------


,_1_nan,Total_Column_N_%,Total_1_Column_N_%,Sec_Column_N_%,JC_Column_N_%,Poly_Column_N_%,ITE_Column_N_%,Local Uni_Column_N_%,PEI_Column_N_%,NSF_Column_N_%,...,Total_14_Column_N_%,I feel that studying is pointless for me_Column_N_%,I get on badly with the teachers_Column_N_%,I am content with my life_Column_N_%,I feel lonely_Column_N_%,I feel fearful for no apparent reason_Column_N_%,I've been feeling optimistic about the future_Column_N_%,I've been dealing well with my problems_Column_N_%,I act without stopping to think_Column_N_%,I enjoy new and exciting experiences even if they are unusual_Column_N_%
0,Total,3005.123,3005.123,613.395,93.03,292.775,101.738,483.3933,177.3269,162.8,...,3005.123,248.3484,141.1685,2005.1624,268.2008,370.5214,1803.3554,2010.1728,340.0788,1672.397
1,Consuming drugs can be addictive.,0.945839,0.945839,0.906667,0.953333,0.974286,0.92691,0.97164,0.945559,0.948864,...,0.945839,0.911087,0.937623,0.955953,0.924871,0.950232,0.962283,0.957092,0.949406,0.95874
2,Consuming drugs will harm one’s health.,0.944381,0.944381,0.946667,0.93,0.965714,0.913621,0.95746,0.934097,0.923295,...,0.944381,0.955371,0.906337,0.958964,0.920507,0.925492,0.9649,0.958152,0.927971,0.955333
3,Consuming drugs affects one’s life negatively.,0.922952,0.922952,0.913333,0.913333,0.962857,0.887043,0.9291,0.922636,0.931818,...,0.922952,0.943004,0.862262,0.947814,0.885103,0.918078,0.951413,0.942321,0.93739,0.943565
4,Consuming drugs affects families negatively.,0.951379,0.951379,0.933333,0.926667,0.971429,0.92691,0.965968,0.936963,0.946023,...,0.951379,0.942776,0.866032,0.973462,0.933636,0.953461,0.969658,0.962321,0.94042,0.968495
5,Consuming drugs affects society negatively.,0.903676,0.903676,0.893333,0.923333,0.945714,0.830565,0.923428,0.908309,0.909091,...,0.903676,0.902273,0.915155,0.928117,0.871853,0.899098,0.930359,0.927379,0.906029,0.916873
6,Consuming cannabis can be addictive.,0.878195,0.878195,0.857778,0.933333,0.951429,0.86711,0.923428,0.862464,0.892045,...,0.878195,0.879601,0.840209,0.899283,0.869733,0.864611,0.898643,0.891813,0.892429,0.895772
7,Consuming cannabis will harm one’s health.,0.861901,0.861901,0.917778,0.906667,0.922857,0.873754,0.892232,0.839542,0.849432,...,0.861901,0.854903,0.869338,0.881864,0.818498,0.843165,0.884829,0.877858,0.886293,0.861426
8,Drugs are more harmful than tobacco.,0.784888,0.784888,0.666667,0.703333,0.802857,0.780731,0.832676,0.756447,0.852273,...,0.784888,0.759195,0.765039,0.798827,0.725382,0.776906,0.810124,0.80459,0.778067,0.783506
9,Cannabis is more harmful than tobacco.,0.637129,0.637129,0.546667,0.62,0.668571,0.651163,0.662517,0.641834,0.678977,...,0.637129,0.629909,0.685025,0.656262,0.605177,0.676533,0.672759,0.663093,0.673925,0.635706


In [22]:
import re
from IPython.display import display

# Regex to remove the suffix _Column_N_% (case-insensitive)
SUFFIX_PATTERN = re.compile(r"_Column_N_%$", flags=re.IGNORECASE)

print("\n=== Removing suffix '_Column_N_%' from NON-SigTest tables ===\n")

for idx, info in table_name_dict.items():
    if info.get("is_sigtest", False):
        continue  # skip SigTest tables

    tname = info["name"]
    df = info["dataframe"]

    original_cols = list(df.columns)

    # Build new column names with suffix removed
    new_cols = [
        re.sub(SUFFIX_PATTERN, "", col).strip() if isinstance(col, str) else col
        for col in df.columns
    ]

    # Only print/modify if changes were made
    if new_cols != original_cols:
        print("=" * 90)
        print(f"TABLE: {tname}")
        print("Updated column names:")
        print("OLD:", original_cols)
        print("NEW:", new_cols)

        # Apply
        df.columns = new_cols
        table_name_dict[idx]["dataframe"] = df
    else:
        print(f"[NO CHANGE] {tname}")

print("\n=== Updated NON-SigTest Tables (first 10 rows each) ===\n")

for idx, info in table_name_dict.items():
    if info.get("is_sigtest", False):
        continue

    tname = info["name"]
    df = info["dataframe"]

    print("=" * 90)
    print(f"TABLE INDEX: {idx}  |  NAME: {tname}  |  SHAPE: {df.shape}")
    print("-" * 90)
    display(df.head(10))
    print()



=== Removing suffix '_Column_N_%' from NON-SigTest tables ===

TABLE: S5. Which category do you fall under
Updated column names:
OLD: ['_1_nan', 'Total_Column_N_%', 'Total_1_Column_N_%', 'Local Youths_Column_N_%', 'Non-Local Youths_Column_N_%', 'Adults_Column_N_%', 'Total_2_Column_N_%', '13 – 14 years_Column_N_%', '15 – 19 years_Column_N_%', '20 – 24 years_Column_N_%', '25 – 29 years_Column_N_%', '30 – 34 years_Column_N_%', '35 – 39 years_Column_N_%', '40 – 44 years_Column_N_%', '45 – 49 years_Column_N_%', '50 – 54 years_Column_N_%', '55 – 59 years_Column_N_%', '60 – 64 years_Column_N_%', '65 – 69 years_Column_N_%', 'Above 69 years_Column_N_%', 'Total_3_Column_N_%', 'Male_Column_N_%', 'Female_Column_N_%', 'Total_4_Column_N_%', 'Singapore Citizen_Column_N_%', 'Singapore Permanent Resident (PR)_Column_N_%', 'Others_Column_N_%', 'Total_5_Column_N_%', 'Chinese_Column_N_%', 'Malay_Column_N_%', 'Indian_Column_N_%', 'Others_1_Column_N_%', 'Total_6_Column_N_%', 'NET Working_Column_N_%', 'NET 

,_1_nan,Total,Total_1,Local Youths,Non-Local Youths,Adults,Total_2,13 – 14 years,15 – 19 years,20 – 24 years,...,Total_16,I feel that studying is pointless for me,I get on badly with the teachers,I am content with my life,I feel lonely,I feel fearful for no apparent reason,I've been feeling optimistic about the future,I've been dealing well with my problems,I act without stopping to think,I enjoy new and exciting experiences even if they are unusual
0,Total,4805,4805,3005,302,1498,4805,318,1082,1157,...,4805,262,154,3438,351,472,3072,3379,488,2265
1,Secondary School/ IP Junior High Student,0.137,0.137,0.208,0.113,0,0.137,0.992,0.317,0,...,0.137,0.266,0.278,0.122,0.167,0.173,0.112,0.116,0.168,0.149
2,JC/MI/ IB/ IP Senior High Student,0.021,0.021,0.032,0.02,0,0.021,0,0.093,0,...,0.021,0.026,0.042,0.019,0.03,0.022,0.019,0.02,0.025,0.024
3,Polytechnic Student,0.063,0.063,0.098,0.03,0,0.063,0,0.225,0.05,...,0.063,0.061,0.061,0.059,0.089,0.1,0.057,0.06,0.072,0.081
4,ITE Student,0.023,0.023,0.036,0.007,0,0.023,0,0.087,0.013,...,0.023,0.041,0.031,0.018,0.036,0.041,0.018,0.018,0.033,0.023
5,"Local University Student (i.e., NUS/NTU/ SMU/ ...",0.11,0.11,0.161,0.152,0,0.11,0,0.057,0.343,...,0.11,0.063,0.075,0.118,0.13,0.12,0.115,0.114,0.085,0.13
6,Private Education Institute Student,0.072,0.072,0.06,0.553,0,0.072,0.008,0.107,0.172,...,0.072,0.098,0.118,0.067,0.105,0.102,0.068,0.062,0.092,0.068
7,Full-time National Servicemen (NSF),0.034,0.034,0.055,0,0,0.034,0,0.043,0.1,...,0.034,0.048,0.051,0.033,0.045,0.05,0.033,0.033,0.048,0.043
8,Currently holding a full-time/ part-time job,0.429,0.429,0.343,0.126,0.663,0.429,0,0.067,0.314,...,0.429,0.383,0.337,0.438,0.349,0.333,0.457,0.452,0.41,0.421
9,Currently not schooling or working,0.11,0.11,0.008,0,0.337,0.11,0,0.004,0.009,...,0.11,0.015,0.006,0.125,0.05,0.06,0.12,0.126,0.068,0.062



TABLE INDEX: 2  |  NAME: B1_Stance  |  SHAPE: (4, 90)
------------------------------------------------------------------------------------------


,_1_nan,Total,Total_1,30-34 years,35-39 years,40-44 years,45-49 years,50-54 years,55-59 years,60-64 years,...,Total_14,I feel that studying is pointless for me,I get on badly with the teachers,I am content with my life,I feel lonely,I feel fearful for no apparent reason,I've been feeling optimistic about the future,I've been dealing well with my problems,I act without stopping to think,I enjoy new and exciting experiences even if they are unusual
0,Total,1498,1498,166,163,161,156,159,152,152,...,1498,0,0,1223,55,71,1064,1183,110,462
1,Conservative,0.967,0.967,0.937,0.945,0.983,0.964,0.945,0.969,0.976,...,0.967,0,0,0.98,0.932,0.974,0.979,0.982,0.961,0.962
2,Neutral,0.028,0.028,0.055,0.05,0.017,0.026,0.045,0.031,0.015,...,0.028,0,0,0.016,0.028,0.026,0.017,0.016,0.018,0.025
3,Liberal,0.005,0.005,0.008,0.005,0,0.009,0.009,0,0.009,...,0.005,0,0,0.004,0.04,0,0.004,0.002,0.021,0.013



TABLE INDEX: 4  |  NAME: $A1.Score  |  SHAPE: (22, 84)
------------------------------------------------------------------------------------------


,_1_nan,Total,Total_1,Sec,JC,Poly,ITE,Local Uni,PEI,NSF,...,Total_14,I feel that studying is pointless for me,I get on badly with the teachers,I am content with my life,I feel lonely,I feel fearful for no apparent reason,I've been feeling optimistic about the future,I've been dealing well with my problems,I act without stopping to think,I enjoy new and exciting experiences even if they are unusual
0,Total,3005.123,3005.123,613.395,93.03,292.775,101.738,483.3933,177.3269,162.8,...,3005.123,248.3484,141.1685,2005.1624,268.2008,370.5214,1803.3554,2010.1728,340.0788,1672.397
1,Consuming drugs can be addictive.,0.945839,0.945839,0.906667,0.953333,0.974286,0.92691,0.97164,0.945559,0.948864,...,0.945839,0.911087,0.937623,0.955953,0.924871,0.950232,0.962283,0.957092,0.949406,0.95874
2,Consuming drugs will harm one’s health.,0.944381,0.944381,0.946667,0.93,0.965714,0.913621,0.95746,0.934097,0.923295,...,0.944381,0.955371,0.906337,0.958964,0.920507,0.925492,0.9649,0.958152,0.927971,0.955333
3,Consuming drugs affects one’s life negatively.,0.922952,0.922952,0.913333,0.913333,0.962857,0.887043,0.9291,0.922636,0.931818,...,0.922952,0.943004,0.862262,0.947814,0.885103,0.918078,0.951413,0.942321,0.93739,0.943565
4,Consuming drugs affects families negatively.,0.951379,0.951379,0.933333,0.926667,0.971429,0.92691,0.965968,0.936963,0.946023,...,0.951379,0.942776,0.866032,0.973462,0.933636,0.953461,0.969658,0.962321,0.94042,0.968495
5,Consuming drugs affects society negatively.,0.903676,0.903676,0.893333,0.923333,0.945714,0.830565,0.923428,0.908309,0.909091,...,0.903676,0.902273,0.915155,0.928117,0.871853,0.899098,0.930359,0.927379,0.906029,0.916873
6,Consuming cannabis can be addictive.,0.878195,0.878195,0.857778,0.933333,0.951429,0.86711,0.923428,0.862464,0.892045,...,0.878195,0.879601,0.840209,0.899283,0.869733,0.864611,0.898643,0.891813,0.892429,0.895772
7,Consuming cannabis will harm one’s health.,0.861901,0.861901,0.917778,0.906667,0.922857,0.873754,0.892232,0.839542,0.849432,...,0.861901,0.854903,0.869338,0.881864,0.818498,0.843165,0.884829,0.877858,0.886293,0.861426
8,Drugs are more harmful than tobacco.,0.784888,0.784888,0.666667,0.703333,0.802857,0.780731,0.832676,0.756447,0.852273,...,0.784888,0.759195,0.765039,0.798827,0.725382,0.776906,0.810124,0.80459,0.778067,0.783506
9,Cannabis is more harmful than tobacco.,0.637129,0.637129,0.546667,0.62,0.668571,0.651163,0.662517,0.641834,0.678977,...,0.637129,0.629909,0.685025,0.656262,0.605177,0.676533,0.672759,0.663093,0.673925,0.635706


In [23]:
# Cell: Compare mapped cell vs total in the same non-sig table row/column, mark highlights (with sig & non-sig col indexes)
import re
import pandas as pd
from IPython.display import display

# --- helpers (self-contained) ---
def parse_numeric_to_fraction(x):
    """Parse many numeric formats into a fraction 0..1 or None."""
    if x is None:
        return None
    if isinstance(x, (int, float)) and not pd.isna(x):
        v = float(x)
        if 0.0 <= v <= 1.0:
            return v
        if 1.0 < v <= 100.0:
            return v / 100.0
        return v / 100.0
    s = str(x).strip()
    if s == "":
        return None
    s = s.replace("(", "").replace(")", "")
    if "%" in s:
        try:
            num = float(s.replace("%","").replace(",", "").strip())
            return num / 100.0
        except Exception:
            return None
    s2 = s.replace(",", "")
    try:
        v = float(s2)
        if 0.0 <= v <= 1.0:
            return v
        if 1.0 < v <= 100.0:
            return v / 100.0
        return v / 100.0
    except Exception:
        return None

def _normalize_for_match(s):
    """Normalize header/token for fuzzy matching."""
    if s is None:
        return ""
    s = str(s).strip().lower()
    s = re.sub(r'[^\w\s%]', '', s)  # keep letters, numbers, whitespace and percent sign
    s = re.sub(r'\s+', ' ', s).strip()
    return s

def get_df_by_name(name):
    """Return dataframe from table_name_dict by exact or containing match."""
    if name is None:
        return None
    for idx, info in table_name_dict.items():
        if info.get("name") == name:
            return info.get("dataframe")
    for idx, info in table_name_dict.items():
        if name in (info.get("name") or ""):
            return info.get("dataframe")
    return None

def find_total_column(df, total_col_key=None):
    """Heuristic to locate the Total column (returns the column name or None)."""
    if df is None:
        return None
    cols = [c for c in df.columns if isinstance(c, str)]

    if total_col_key:
        key = str(total_col_key).strip().lower()
        # 1) exact match
        for c in cols:
            if c.strip().lower() == key:
                return c
        # 2) substring match
        for c in cols:
            if key in c.lower():
                return c

    # 3) fallback: any column containing 'total'
    for c in cols:
        if "total" in c.lower():
            return c

    return None

def find_sig_col_index(sig_df, total_col_key=None, token=None, sig_col_name=None, max_right_scan=50):
    """
    Find the index (integer position) of the SigTest column that corresponds to the token.
    Strategy:
      1. Direct exact / case-insensitive / normalized match on sig_col_name.
      2. If not found, find total_col in sig_df then scan to the right for:
         - header == token (case-insensitive)
         - header contains token as word
         - normalized header contains normalized token
         - header contains sig_col_name
      3. Fallback: immediate-right column of total_col (if in bounds).
      4. If none found, return None.
    """
    if sig_df is None:
        return None

    cols = list(sig_df.columns)
    ncols = len(cols)

    # 1) try direct matches on sig_col_name
    if sig_col_name:
        # exact
        for i, c in enumerate(cols):
            try:
                if c == sig_col_name:
                    return i
            except Exception:
                pass
        # case-insensitive exact
        for i, c in enumerate(cols):
            try:
                if isinstance(c, str) and c.strip().lower() == str(sig_col_name).strip().lower():
                    return i
            except Exception:
                pass
        # normalized exact
        sig_norm = _normalize_for_match(sig_col_name)
        for i, c in enumerate(cols):
            try:
                if isinstance(c, str) and _normalize_for_match(c) == sig_norm:
                    return i
            except Exception:
                pass

    # 2) locate total_col index in sig_df
    total_col_index = None
    if total_col_key is not None:
        key = str(total_col_key).strip().lower()
        for i, c in enumerate(cols):
            try:
                if isinstance(c, str) and c.strip().lower() == key:
                    total_col_index = i
                    break
            except Exception:
                pass
        if total_col_index is None:
            for i, c in enumerate(cols):
                try:
                    if isinstance(c, str) and key in c.lower():
                        total_col_index = i
                        break
                except Exception:
                    pass

    # 3) if we have a total_col_index, scan to the right for token/sig_col_name matches
    if total_col_index is not None:
        token_norm = _normalize_for_match(token) if token else ""
        sig_norm = _normalize_for_match(sig_col_name) if sig_col_name else ""

        # limit how far we scan to the right for safety
        right_limit = min(ncols, total_col_index + 1 + max_right_scan)

        for j in range(total_col_index + 1, right_limit):
            c = cols[j]
            if not isinstance(c, str):
                continue
            c_raw = c
            c_low = c_raw.strip().lower()
            c_norm = _normalize_for_match(c_raw)

            # exact match on sig_col_name (again) or token exact
            if sig_col_name and c_low == str(sig_col_name).strip().lower():
                return j
            if token and c_low == str(token).strip().lower():
                return j

            # token as separate word in header (e.g., header "A (Male)" matches token "A")
            if token:
                # word boundary match
                try:
                    if re.search(rf'\b{re.escape(str(token).strip().lower())}\b', c_low):
                        return j
                except Exception:
                    pass

            # normalized contains normalized token or sig_col_name
            if token and token_norm and token_norm in c_norm:
                return j
            if sig_col_name and sig_norm and sig_norm in c_norm:
                return j

        # 4) fallback: immediate right column if nothing matched (and in bounds)
        if (total_col_index + 1) < ncols:
            return total_col_index + 1

    # 5) final fallback: try to find token anywhere in header list
    if token:
        token_norm = _normalize_for_match(token)
        for i, c in enumerate(cols):
            try:
                if isinstance(c, str) and token_norm in _normalize_for_match(c):
                    return i
            except Exception:
                pass

    return None

def find_non_sig_column(df, base_colname):
    """
    Matching priority:
      1. Exact (case-sensitive) match
      2. Exact case-insensitive match
      3. Normalized exact match
      4. Normalized substring match
      5. Raw substring fallback
    Returns the first matching column (the actual column name from df.columns) or None.
    """
    if df is None or base_colname is None:
        return None

    base_raw = str(base_colname)
    base_low = base_raw.lower()
    base_norm = _normalize_for_match(base_raw)

    cols = [c for c in df.columns]

    # 1) exact match
    for c in cols:
        if c == base_raw:
            return c

    # 2) case-insensitive exact match
    for c in cols:
        try:
            if isinstance(c, str) and c.strip().lower() == base_low.strip():
                return c
        except Exception:
            continue

    # 3) normalized exact match
    for c in cols:
        try:
            if isinstance(c, str) and _normalize_for_match(c) == base_norm:
                return c
        except Exception:
            continue

    # 4) normalized substring match (base inside column)
    for c in cols:
        try:
            if isinstance(c, str) and base_norm in _normalize_for_match(c):
                return c
        except Exception:
            continue

    # 5) raw substring fallback (case-insensitive)
    for c in cols:
        try:
            if isinstance(c, str) and base_low in c.lower():
                return c
        except Exception:
            continue

    return None

def find_row_index_by_name(df, row_name):
    """Find row index by matching first column value to row_name."""
    if df is None or df.shape[0] == 0 or df.shape[1] == 0 or row_name is None:
        return None
    row0col = df.columns[0]
    target = str(row_name).strip().lower()
    for r in range(df.shape[0]):
        val = df.iloc[r, 0]
        if pd.isna(val):
            continue
        if str(val).strip().lower() == target:
            return r
    return None

# ------------- Main loop: build comparisons -------------
if 'annotated_summary' not in globals():
    raise RuntimeError("annotated_summary not found — run the annotator cell first.")

cross_ref_results = {}
rows = []

for sig_table, totals in annotated_summary.items():
    cross_ref_results[sig_table] = {}
    # load the sig_df (if available) for the find_sig_col_index heuristic
    sig_df = get_df_by_name(sig_table)

    # find corresponding non-sig table info (reuse your original lookup)
    non_sig_info = None
    for idx, info in table_name_dict.items():
        if info.get("is_sigtest", False):
            continue
        # attempt match by name heuristic
        name = info.get("name") or ""
        if sig_table.endswith("_SigTestTable"):
            base = sig_table[:-len("_SigTestTable")]
        else:
            base = sig_table
        if name == base or name.startswith(base) or base in name:
            non_sig_info = info
            break

    for total_col, entries in totals.items():
        out_entries = []
        for e in entries:
            row_name = e.get("row_name")
            token = e.get("token")
            sig_col = e.get("column_name")  # the SigTest column name (base)
            result = {
                "row_index": e.get("row_index"),
                "row_name": row_name,
                "token": token,
                "sig_column_name": sig_col,
                "non_sig_table": None,
                "non_sig_column": None,
                "non_sig_row_index": None,
                "cell_value": None
            }

            if non_sig_info is None:
                out_entries.append(result)
                continue

            # record the non-sig table name and df
            non_sig_name = non_sig_info.get("name")
            result["non_sig_table"] = non_sig_name
            non_sig_df = non_sig_info.get("dataframe")

            # ---------- NEW: find sig column index more robustly ----------
            sig_col_index = None
            # First try direct/header matches for sig_col
            if sig_df is not None:
                sig_col_index = find_sig_col_index(sig_df, total_col_key=total_col, token=token, sig_col_name=sig_col)

            # store sig column index in result (for debugging / downstream)
            result["sig_column_index"] = sig_col_index

            # ---------- find the non-sig column using the UPDATED matching logic ----------
            non_sig_col = None
            if sig_col:
                non_sig_col = find_non_sig_column(non_sig_df, sig_col)

            # if still none, try to find any percent-column-style fallback
            if non_sig_col is None and non_sig_df is not None:
                for c in non_sig_df.columns:
                    try:
                        if isinstance(c, str) and (re.search(r"Column_N_%", c, flags=re.IGNORECASE) or c.strip().endswith('%')):
                            non_sig_col = c
                            break
                    except Exception:
                        continue

            result["non_sig_column"] = non_sig_col

            # find the row index in the non-sig df by matching column 0 to row_name
            non_sig_row_idx = find_row_index_by_name(non_sig_df, row_name)
            result["non_sig_row_index"] = non_sig_row_idx

            # extract the cell value if both column and row found
            if non_sig_col is not None and non_sig_row_idx is not None:
                try:
                    val = non_sig_df.loc[non_sig_row_idx, non_sig_col]
                    result["cell_value"] = val
                except Exception:
                    result["cell_value"] = None

            out_entries.append(result)

        cross_ref_results[sig_table][total_col] = out_entries

# Print a readable summary of cross-references
print("\n=== Cross-reference: SigTest token -> non-SigTest table cell values (UPDATED matching) ===\n")
for sig_table, totals in cross_ref_results.items():
    print("="*100)
    print("SigTest Table:", sig_table)
    for total_col, rows in totals.items():
        print(f"\n  Total Column: {total_col}")
        if not rows:
            print("    (no entries)")
            continue
        for r in rows:
            print(f"    token {r['token']:>2} | sig_col: {r['sig_column_name']!r} | sig_col_index: {r.get('sig_column_index')!s} | non-sig-table: {r['non_sig_table']!r} | non-sig-col: {r['non_sig_column']!r} | row({r['row_index']})/{r['row_name']!r} -> value: {r['cell_value']!r}")
    print()

# cross_ref_results is available for programmatic use



=== Cross-reference: SigTest token -> non-SigTest table cell values (UPDATED matching) ===

SigTest Table: S5. Which category do you fall under_SigTestTable

  Total Column: Total
    (no entries)

  Total Column: Total_1
    token  C | sig_col: 'Non-Local Youths' | sig_col_index: 4 | non-sig-table: 'S5. Which category do you fall under' | non-sig-col: 'Non-Local Youths' | row(3)/'Polytechnic Student' -> value: 0.03
    token  B | sig_col: 'Local Youths' | sig_col_index: 3 | non-sig-table: 'S5. Which category do you fall under' | non-sig-col: 'Local Youths' | row(6)/'Private Education Institute Student' -> value: 0.06
    token  B | sig_col: 'Local Youths' | sig_col_index: 3 | non-sig-table: 'S5. Which category do you fall under' | non-sig-col: 'Local Youths' | row(8)/'Currently holding a full-time/ part-time job' -> value: 0.343
    token  C | sig_col: 'Non-Local Youths' | sig_col_index: 4 | non-sig-table: 'S5. Which category do you fall under' | non-sig-col: 'Non-Local Youths' | row

In [24]:
# Cell: Compare mapped cell vs total in the same non-sig table row/column, mark highlights (with sig & non-sig col indexes)
import re
import pandas as pd
from IPython.display import display

# --- helpers (self-contained) ---
def parse_numeric_to_fraction(x):
    """Parse many numeric formats into a fraction 0..1 or None."""
    if x is None:
        return None
    if isinstance(x, (int, float)) and not pd.isna(x):
        v = float(x)
        if 0.0 <= v <= 1.0:
            return v
        if 1.0 < v <= 100.0:
            return v / 100.0
        return v / 100.0
    s = str(x).strip()
    if s == "":
        return None
    s = s.replace("(", "").replace(")", "")
    if "%" in s:
        try:
            num = float(s.replace("%","").replace(",", "").strip())
            return num / 100.0
        except Exception:
            return None
    s2 = s.replace(",", "")
    try:
        v = float(s2)
        if 0.0 <= v <= 1.0:
            return v
        if 1.0 < v <= 100.0:
            return v / 100.0
        return v / 100.0
    except Exception:
        return None

def get_df_by_name(name):
    """Return dataframe from table_name_dict by exact or containing match."""
    if name is None:
        return None
    for idx, info in table_name_dict.items():
        if info.get("name") == name:
            return info.get("dataframe")
    for idx, info in table_name_dict.items():
        if name in (info.get("name") or ""):
            return info.get("dataframe")
    return None

def find_total_column(df, total_col_key=None):
    """Heuristic to locate the Total column."""
    if df is None:
        return None
    cols = [c for c in df.columns if isinstance(c, str)]

    if total_col_key:
        key = str(total_col_key).strip().lower()
        # 1) exact match
        for c in cols:
            if c.strip().lower() == key:
                return c
        # 2) substring match
        for c in cols:
            if key in c.lower():
                return c

    # 3) fallback: any column containing 'total'
    for c in cols:
        if "total" in c.lower():
            return c

    return None

# ------------- Main loop: build comparisons -------------
if 'cross_ref_results' not in globals():
    raise RuntimeError("cross_ref_results not found — run the mapping cell first.")

rows = []
for sig_table, totals in cross_ref_results.items():

    # Load SigTest dataframe for column index lookup
    sig_df = get_df_by_name(sig_table)

    for total_col, entries in totals.items():
        for ent in entries:

            # ---------- Extract fields ----------
            row_index = ent.get("row_index")
            row_name = ent.get("row_name")
            token = ent.get("token")
            sig_col = ent.get("sig_column_name")
            non_sig_table_name = ent.get("non_sig_table")
            mapped_col = ent.get("non_sig_column")
            mapped_row_idx = ent.get("non_sig_row_index")
            mapped_cell_raw = ent.get("cell_value")

            # ---------- Load non-sig table ----------
            non_sig_df = get_df_by_name(non_sig_table_name)

            # ---------- Column index resolution ----------
            # SigTest column index - NEW heuristic:
            sig_col_index = None
            # 1) direct exact match (fast path)
            if sig_df is not None and sig_col in (sig_df.columns if sig_col else []):
                sig_col_index = list(sig_df.columns).index(sig_col)
            # 2) case-insensitive match by header
            elif sig_df is not None and isinstance(sig_col, str):
                for i, c in enumerate(sig_df.columns):
                    try:
                        if isinstance(c, str) and c.lower() == sig_col.lower():
                            sig_col_index = i
                            break
                    except Exception:
                        continue

            # 3) NEW: If still not found, locate the exact total column in sig_df
            #    then search to the right from that column for sig_col or token matches.
            if sig_col_index is None and sig_df is not None:
                total_col_index = None
                # try exact match of total_col in sig_df columns (case-insensitive)
                if total_col is not None:
                    for i, c in enumerate(sig_df.columns):
                        try:
                            if isinstance(c, str) and str(c).strip().lower() == str(total_col).strip().lower():
                                total_col_index = i
                                break
                        except Exception:
                            continue
                # fallback: substring match for total_col
                if total_col_index is None and total_col is not None:
                    key = str(total_col).strip().lower()
                    for i, c in enumerate(sig_df.columns):
                        try:
                            if isinstance(c, str) and key in c.lower():
                                total_col_index = i
                                break
                        except Exception:
                            continue

                # If we found a total_col_index, search to the right for a column matching sig_col or token
                if total_col_index is not None:
                    ncols = len(sig_df.columns)
                    for j in range(total_col_index + 1, ncols):
                        c = sig_df.columns[j]
                        try:
                            if sig_col and isinstance(c, str) and c.strip().lower() == str(sig_col).strip().lower():
                                sig_col_index = j
                                break
                            if sig_col and isinstance(c, str) and str(sig_col).strip().lower() in c.strip().lower():
                                sig_col_index = j
                                break
                            # if sig_col not present or still not matched, try matching token
                            if token and isinstance(c, str) and str(token).strip().lower() in c.strip().lower():
                                sig_col_index = j
                                break
                        except Exception:
                            continue
                    # last-resort: pick the immediate right column if nothing matched (guarded by bounds)
                    if sig_col_index is None and (total_col_index + 1) < ncols:
                        sig_col_index = total_col_index + 1

            # Non-SigTest column index
            non_sig_col_index = None
            if non_sig_df is not None and mapped_col in (non_sig_df.columns if mapped_col else []):
                non_sig_col_index = list(non_sig_df.columns).index(mapped_col)
            elif non_sig_df is not None and isinstance(mapped_col, str):
                for i, c in enumerate(non_sig_df.columns):
                    try:
                        if isinstance(c, str) and c.lower() == mapped_col.lower():
                            non_sig_col_index = i
                            break
                    except Exception:
                        continue

            # ---------- Determine TOTAL column ----------
            total_col_name = find_total_column(non_sig_df, total_col)

            # ---------- Parse numeric values ----------
            total_raw = None
            total_num = None
            cell_num = parse_numeric_to_fraction(mapped_cell_raw)

            if (
                non_sig_df is not None
                and total_col_name is not None
                and mapped_row_idx is not None
                and not pd.isna(mapped_row_idx)
            ):
                try:
                    total_raw = non_sig_df.loc[int(mapped_row_idx), total_col_name]
                    total_num = parse_numeric_to_fraction(total_raw)
                except Exception:
                    total_raw = None
                    total_num = None

            # ---------- Compute abs_diff + highlight ----------
            abs_diff = None
            highlight = False
            if cell_num is not None and total_num is not None:
                abs_diff = abs(cell_num - total_num)
                highlight = abs_diff >= 0.05

            # ---------- Record row ----------
            rows.append({
                "sig_table": sig_table,
                "total_col_key": total_col,
                "token": token,
                "sig_column_name": sig_col,
                "sig_column_index": sig_col_index,          # ⭐ NEW HEURISTIC
                "sig_row_index": row_index,
                "sig_row_name": row_name,

                "non_sig_table": non_sig_table_name,
                "mapped_column": mapped_col,
                "non_sig_column_index": non_sig_col_index,  # ⭐ NEW
                "mapped_row_index": mapped_row_idx,

                "mapped_cell_raw": mapped_cell_raw,
                "mapped_cell_fraction": cell_num,

                "total_column_used": total_col_name,
                "total_raw": total_raw,
                "total_fraction": total_num,

                "abs_diff": abs_diff,
                "highlight": highlight
            })

# ---------- Build DataFrame ----------
cross_ref_comparison_df = pd.DataFrame(rows)

print("Total comparisons:", len(cross_ref_comparison_df))

if not cross_ref_comparison_df.empty:

    # --- reorder/select useful columns ---
    display_cols = [
        "sig_table","token","sig_column_name","sig_column_index","sig_row_index","sig_row_name",
        "non_sig_table","mapped_column","non_sig_column_index","mapped_row_index",
        "mapped_cell_raw","total_column_used","total_raw",
        "mapped_cell_fraction","total_fraction","abs_diff","highlight"
    ]
    display_cols = [c for c in display_cols if c in cross_ref_comparison_df.columns]

    df_sorted = cross_ref_comparison_df.sort_values(
        by=["highlight", "abs_diff", "sig_row_index"],
        ascending=[False, False, True],
        na_position="last"
    )


    display(df_sorted[display_cols].head(200))

    # ---------- Print summary of highlighted ----------
    highlighted = df_sorted[df_sorted["highlight"] == True]

    if not highlighted.empty:
        print("\n=== CELLS THAT SHOULD BE HIGHLIGHTED (abs diff ≥ 5 percentage points) ===\n")
        for _, r in highlighted.iterrows():
            def pct(x): return f"{x*100:.2f}%" if x is not None else "N/A"
            print(
                f"- {r['non_sig_table']}  |  row {r['mapped_row_index']}  |  col '{r['mapped_column']}' (index {r['non_sig_column_index']})"
                f"\n    cell = {r['mapped_cell_raw']!r} ({pct(r['mapped_cell_fraction'])})"
                f"\n    total @ '{r['total_column_used']}' = {r['total_raw']!r} ({pct(r['total_fraction'])})"
                f"\n    abs_diff = {pct(r['abs_diff'])}  => SHOULD HIGHLIGHT"
                f"\n"
            )
    else:
        print("\nNo highlighted values found (abs diff < 5%).")

cross_ref_comparison_df  # keep in namespace


Total comparisons: 220


,sig_table,token,sig_column_name,sig_column_index,sig_row_index,sig_row_name,non_sig_table,mapped_column,non_sig_column_index,mapped_row_index,mapped_cell_raw,total_column_used,total_raw,mapped_cell_fraction,total_fraction,abs_diff,highlight
21,S5. Which category do you fall under_SigTestTable,F,30 – 34 years,11,10,Net Not Working,S5. Which category do you fall under,30 – 34 years,11,10,0.089000,Total_2,0.571000,0.089000,0.571000,0.482000,True
23,S5. Which category do you fall under_SigTestTable,H,40 – 44 years,13,10,Net Not Working,S5. Which category do you fall under,40 – 44 years,13,10,0.096000,Total_2,0.571000,0.096000,0.571000,0.475000,True
22,S5. Which category do you fall under_SigTestTable,G,35 – 39 years,12,10,Net Not Working,S5. Which category do you fall under,35 – 39 years,12,10,0.097000,Total_2,0.571000,0.097000,0.571000,0.474000,True
183,$A1.Score_SigTestTable,G,Sikhism,38,11,Drug-taking is fine as long as it doesn’t harm...,$A1.Score,Sikhism,38,11,0.381650,Total_7,0.832333,0.381650,0.832333,0.450682,True
185,$A1.Score_SigTestTable,J,Sikhism & Others,41,11,Drug-taking is fine as long as it doesn’t harm...,$A1.Score,Sikhism & Others,41,11,0.381650,Total_7,0.832333,0.381650,0.832333,0.450682,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124,B1_Stance_SigTestTable,H,I've been dealing well with my problems,87,2,Neutral,B1_Stance,I've been dealing well with my problems,87,2,0.016000,Total_14,0.028000,0.016000,0.028000,0.012000,False
65,S5. Which category do you fall under_SigTestTable,I,NET HDB,77,5,"Local University Student (i.e., NUS/NTU/ SMU/ ...",S5. Which category do you fall under,NET HDB,77,5,0.098000,Total_11,0.110000,0.098000,0.110000,0.012000,False
1,S5. Which category do you fall under_SigTestTable,B,Local Youths,3,6,Private Education Institute Student,S5. Which category do you fall under,Local Youths,3,6,0.060000,Total_1,0.072000,0.060000,0.072000,0.012000,False
146,$A1.Score_SigTestTable,B,Male,18,4,Consuming drugs affects families negatively.,$A1.Score,Male,18,4,0.939500,Total_3,0.951379,0.939500,0.951379,0.011879,False



=== CELLS THAT SHOULD BE HIGHLIGHTED (abs diff ≥ 5 percentage points) ===

- S5. Which category do you fall under  |  row 10  |  col '30 – 34 years' (index 11)
    cell = 0.089 (8.90%)
    total @ 'Total_2' = 0.571 (57.10%)
    abs_diff = 48.20%  => SHOULD HIGHLIGHT

- S5. Which category do you fall under  |  row 10  |  col '40 – 44 years' (index 13)
    cell = 0.096 (9.60%)
    total @ 'Total_2' = 0.571 (57.10%)
    abs_diff = 47.50%  => SHOULD HIGHLIGHT

- S5. Which category do you fall under  |  row 10  |  col '35 – 39 years' (index 12)
    cell = 0.097 (9.70%)
    total @ 'Total_2' = 0.571 (57.10%)
    abs_diff = 47.40%  => SHOULD HIGHLIGHT

- $A1.Score  |  row 11  |  col 'Sikhism' (index 38)
    cell = 0.3816501034972311 (38.17%)
    total @ 'Total_7' = 0.8323325201663646 (83.23%)
    abs_diff = 45.07%  => SHOULD HIGHLIGHT

- $A1.Score  |  row 11  |  col 'Sikhism & Others' (index 41)
    cell = 0.3816501034972311 (38.17%)
    total @ 'Total_7' = 0.8323325201663646 (83.23%)
    ab

,sig_table,total_col_key,token,sig_column_name,sig_column_index,sig_row_index,sig_row_name,non_sig_table,mapped_column,non_sig_column_index,mapped_row_index,mapped_cell_raw,mapped_cell_fraction,total_column_used,total_raw,total_fraction,abs_diff,highlight
0,S5. Which category do you fall under_SigTestTable,Total_1,C,Non-Local Youths,4,3,Polytechnic Student,S5. Which category do you fall under,Non-Local Youths,4,3,0.030000,0.030000,Total_1,0.063000,0.063000,0.033000,False
1,S5. Which category do you fall under_SigTestTable,Total_1,B,Local Youths,3,6,Private Education Institute Student,S5. Which category do you fall under,Local Youths,3,6,0.060000,0.060000,Total_1,0.072000,0.072000,0.012000,False
2,S5. Which category do you fall under_SigTestTable,Total_1,B,Local Youths,3,8,Currently holding a full-time/ part-time job,S5. Which category do you fall under,Local Youths,3,8,0.343000,0.343000,Total_1,0.429000,0.429000,0.086000,True
3,S5. Which category do you fall under_SigTestTable,Total_1,C,Non-Local Youths,4,8,Currently holding a full-time/ part-time job,S5. Which category do you fall under,Non-Local Youths,4,8,0.126000,0.126000,Total_1,0.429000,0.429000,0.303000,True
4,S5. Which category do you fall under_SigTestTable,Total_1,B,Local Youths,3,9,Currently not schooling or working,S5. Which category do you fall under,Local Youths,3,9,0.008000,0.008000,Total_1,0.110000,0.110000,0.102000,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
215,$A1.Score_SigTestTable,Total_14,C,I get on badly with the teachers,76,15,I wouldn’t mind trying drugs if it was legal.,$A1.Score,I get on badly with the teachers,76,15,0.610635,0.610635,Total_14,0.826999,0.826999,0.216365,True
216,$A1.Score_SigTestTable,Total_14,E,I feel lonely,78,15,I wouldn’t mind trying drugs if it was legal.,$A1.Score,I feel lonely,78,15,0.694762,0.694762,Total_14,0.826999,0.826999,0.132237,True
217,$A1.Score_SigTestTable,Total_14,F,I feel fearful for no apparent reason,79,15,I wouldn’t mind trying drugs if it was legal.,$A1.Score,I feel fearful for no apparent reason,79,15,0.749769,0.749769,Total_14,0.826999,0.826999,0.077230,True
218,$A1.Score_SigTestTable,Total_14,I,I act without stopping to think,82,15,I wouldn’t mind trying drugs if it was legal.,$A1.Score,I act without stopping to think,82,15,0.736309,0.736309,Total_14,0.826999,0.826999,0.090690,True


In [25]:
# Replace first cell (row0, col0) with "Base" AND clear first column header ("") 
# for all NON-SigTest tables and update table_name_dict

import pandas as pd

# Safety checks
if 'table_name_dict' not in globals():
    raise RuntimeError("table_name_dict not found. Run the cell that builds table_name_dict first.")

original_first_cells = {}   # backup: {table_name: original row0-col0 value}
original_first_colnames = {}  # backup: {table_name: original header of col0}
modified_tables = []

for idx, info in list(table_name_dict.items()):
    try:
        is_sig = bool(info.get("is_sigtest", False))
    except Exception:
        is_sig = False

    if is_sig:
        # skip SigTest tables
        continue

    tname = info.get("name", f"table_{idx}")
    df = info.get("dataframe")

    if df is None:
        print(f"[SKIP] {tname}: no dataframe found.")
        continue
    if not isinstance(df, pd.DataFrame):
        print(f"[SKIP] {tname}: dataframe object is not a pandas DataFrame (type={type(df)}).")
        continue
    if df.shape[0] == 0 or df.shape[1] == 0:
        print(f"[SKIP] {tname}: empty dataframe (shape={df.shape}).")
        continue

    # backup original first cell
    try:
        orig_cell = df.iat[0, 0]
    except Exception:
        try:
            orig_cell = df.iloc[0, 0]
        except Exception:
            orig_cell = None

    original_first_cells[tname] = orig_cell

    # backup original first column name
    try:
        orig_colname = df.columns[0]
    except Exception:
        orig_colname = None

    original_first_colnames[tname] = orig_colname

    # ------- UPDATE row0-col0 value -------
    if orig_cell != "Base":
        try:
            df.iat[0, 0] = "Base"
        except Exception:
            try:
                df.iloc[0, 0] = "Base"
            except Exception as exc:
                print(f"[ERROR] {tname}: failed to set first cell to 'Base': {exc}")
                continue

    # ------- UPDATE first column header -------
    if orig_colname != "":
        new_cols = list(df.columns)
        new_cols[0] = ""  # empty header
        df.columns = new_cols

    # write back
    table_name_dict[idx]["dataframe"] = df
    modified_tables.append(tname)
    print(f"[MODIFIED] {tname}: first cell was {orig_cell!r} -> 'Base', first header was {orig_colname!r} -> ''")

print("\nSummary:")
print(f"  Tables modified: {len(modified_tables)}")
if modified_tables:
    for t in modified_tables[:50]:
        print("   -", t)
else:
    print("   (none)")

# expose backups
globals()['original_first_cells_non_sig'] = original_first_cells
globals()['original_first_colnames_non_sig'] = original_first_colnames
print("\nBackups saved: original_first_cells_non_sig, original_first_colnames_non_sig")


[MODIFIED] S5. Which category do you fall under: first cell was 'Total' -> 'Base', first header was '_1_nan' -> ''
[MODIFIED] B1_Stance: first cell was 'Total' -> 'Base', first header was '_1_nan' -> ''
[MODIFIED] $A1.Score: first cell was 'Total' -> 'Base', first header was '_1_nan' -> ''

Summary:
  Tables modified: 3
   - S5. Which category do you fall under
   - B1_Stance
   - $A1.Score

Backups saved: original_first_cells_non_sig, original_first_colnames_non_sig


In [ ]:
# Cell: Export highlighted cells -> CSV + Excel with visual highlights
import os
from pathlib import Path
import pandas as pd
import numpy as np
import re

# Output directory
OUT_DIR = Path.cwd() / "highlighted_outputs"
OUT_DIR.mkdir(parents=True, exist_ok=True)

# Files to write
CSV_OUT = OUT_DIR / "highlighted_cells.csv"

# Basic safety checks
if 'cross_ref_comparison_df' not in globals():
    raise RuntimeError("cross_ref_comparison_df not found — run the comparison cell first.")
if 'table_name_dict' not in globals():
    raise RuntimeError("table_name_dict not found — need this to locate DataFrames.")

df_comparisons = cross_ref_comparison_df.copy()

# Ensure the highlight boolean column exists
if "highlight" not in df_comparisons.columns:
    raise RuntimeError("'highlight' column missing from cross_ref_comparison_df.")

# Filter highlighted cells (these drive the CSV and cell-level highlights; we will still export all tables)
highlights = df_comparisons[df_comparisons["highlight"] == True].reset_index(drop=True)
if not highlights.empty:
    highlights_by_table = {k: g for k, g in highlights.groupby("non_sig_table")}
else:
    highlights_by_table = {}

# -------------------------
# Helpers
# -------------------------
def _safe_filename(s):
    """Return a Windows-safe filename string for s."""
    if s is None:
        return "table"
    name = str(s).strip()
    # remove forbidden characters and control whitespace
    name = re.sub(r'[<>:"/\\|?*\n\r\t]', '', name)
    name = re.sub(r'\s+', '_', name)
    name = name.strip("._")
    if len(name) > 120:
        name = name[:120].rstrip("_")
    if name == "":
        name = "table"
    # simple device-name guard
    if name.upper() in {"CON","PRN","AUX","NUL","COM1","COM2","LPT1","LPT2"}:
        name = f"table_{name}"
    return name

def parse_numeric_to_fraction_for_write(x):
    """Try to parse a cell to a float fraction between 0.0 and 1.0; return None if not parseable."""
    if x is None:
        return None
    if isinstance(x, (int, float, np.number)) and not pd.isna(x):
        try:
            v = float(x)
        except Exception:
            return None
        # assume already fraction or percent-like
        if 0.0 <= v <= 1.0:
            return v
        if 1.0 < v <= 100.0:
            return v / 100.0
        return v / 100.0
    s = str(x).strip()
    if s == "" or s.lower() in {"nan","none"}:
        return None
    s = s.replace("(", "").replace(")", "")
    if "%" in s:
        try:
            return float(s.replace("%", "").replace(",", "").strip()) / 100.0
        except Exception:
            return None
    s2 = s.replace(",", "")
    try:
        v = float(s2)
        if 0.0 <= v <= 1.0:
            return v
        if 1.0 < v <= 100.0:
            return v / 100.0
        return v / 100.0
    except Exception:
        return None

# CSV accumulator (rows for highlighted entries only)
csv_rows = []

# Track which files were written (sanitized names) for debugging/logging
written_files = []

# Loop over all tables in table_name_dict and export each one
for tbl_idx, info in table_name_dict.items():
    table_name = info.get("name") or f"table_{tbl_idx}"
    non_sig_df = info.get("dataframe")
    if non_sig_df is None:
        print(f"Skipping table {table_name}: missing dataframe.")
        continue
    if non_sig_df.empty:
        print(f"Skipping table {table_name}: dataframe is empty.")
        continue

    # Determine highlight group for this table (may be None)
    group = highlights_by_table.get(table_name)

    # If there are highlight rows, append them to csv_rows (same fields you used earlier)
    if group is not None and not group.empty:
        for _, r in group.iterrows():
            csv_rows.append({
                "non_sig_table": table_name,
                "non_sig_table_index": tbl_idx,
                "mapped_row_index": r.get("mapped_row_index"),
                "non_sig_column_index": r.get("non_sig_column_index"),
                "mapped_column_name": r.get("mapped_column"),
                "mapped_cell_raw": r.get("mapped_cell_raw"),
                "total_column_used": r.get("total_column_used"),
                "total_raw": r.get("total_raw"),
                "mapped_cell_fraction": r.get("mapped_cell_fraction"),
                "total_fraction": r.get("total_fraction"),
                "abs_diff": r.get("abs_diff"),
                "token": r.get("token"),
                "sig_table": r.get("sig_table"),
                "sig_column_name": r.get("sig_column_name"),
                "sig_column_index": r.get("sig_column_index"),
                "sig_row_name": r.get("sig_row_name"),
            })

    # Build safe filename and sanitize dataframe for writing
    safe_name = _safe_filename(table_name)
    excel_path = OUT_DIR / f"{safe_name}__red_highlighted.xlsx"

    # Replace +/-inf with NaN and make a copy to avoid warnings
    safe_df_for_write = non_sig_df.replace([np.inf, -np.inf], np.nan).copy()

    try:
        # Open ExcelWriter with engine_kwargs forwarding options to xlsxwriter
        with pd.ExcelWriter(excel_path, engine="xlsxwriter",
                           engine_kwargs={"options": {"nan_inf_to_errors": True}}) as writer:

            # write the whole df first (we will overwrite headers and apply formatting)
            safe_df_for_write.to_excel(writer, sheet_name="data", index=False)
            workbook = writer.book
            worksheet = writer.sheets["data"]

            # header format (#D9E1F2)
            header_fmt = workbook.add_format({
                "bg_color": "#D9E1F2",
                "bold": True,
                "align": "center"
            })

            # highlight format (soft peach)
            highlight_fmt = workbook.add_format({"bg_color": "#FBE2D5"})

            # percentage format for data cells (one decimal place)
            percent_fmt = workbook.add_format({"num_format": "0.0%"})

            # combined format for highlighted + percentage cells
            highlight_percent_fmt = workbook.add_format({
                "bg_color": "#FBE2D5",
                "num_format": "0.0%"
            })

            # Step 1: rewrite headers with header_fmt (overwrite default)
            for col_idx, _ in enumerate(safe_df_for_write.columns):
                worksheet.write(0, col_idx, safe_df_for_write.columns[col_idx], header_fmt)

            # Step 2: write all cells, applying percent_fmt to numeric cells EXCEPT Base rows
            for df_row_idx in range(safe_df_for_write.shape[0]):
                first_cell = safe_df_for_write.iat[df_row_idx, 0] if safe_df_for_write.shape[1] > 0 else None
                is_base_row = False
                try:
                    is_base_row = (str(first_cell).strip().lower() == "base")
                except Exception:
                    is_base_row = False

                excel_row = df_row_idx + 1  # header row is 0
                for col_idx in range(safe_df_for_write.shape[1]):
                    val = safe_df_for_write.iat[df_row_idx, col_idx]
                    numeric_val = None
                    if not is_base_row:
                        numeric_val = parse_numeric_to_fraction_for_write(val)

                    if numeric_val is not None:
                        try:
                            # guard: only call write_number if numeric_val is finite
                            if isinstance(numeric_val, (int, float, np.number)) and np.isfinite(numeric_val):
                                worksheet.write_number(excel_row, col_idx, numeric_val, percent_fmt)
                            else:
                                worksheet.write(excel_row, col_idx, val)
                        except Exception:
                            worksheet.write(excel_row, col_idx, val)
                    else:
                        worksheet.write(excel_row, col_idx, val)

            # Step 3: apply highlight formatting (only if highlight rows exist for this table)
            if group is not None and not group.empty:
                for _, r in group.iterrows():
                    # you used mapped_row_index & non_sig_column_index earlier; support both if present
                    row_idx = r.get("mapped_row_index") if "mapped_row_index" in r.index else r.get("non_sig_row_index")
                    col_idx = r.get("non_sig_column_index") if "non_sig_column_index" in r.index else r.get("non_sig_cell_col")
                    if pd.isna(row_idx) or col_idx is None:
                        continue
                    try:
                        df_row_idx = int(row_idx)
                        excel_row = df_row_idx + 1
                        excel_col = int(col_idx)
                    except Exception:
                        continue

                    # bounds check
                    if df_row_idx < 0 or df_row_idx >= safe_df_for_write.shape[0] or excel_col < 0 or excel_col >= safe_df_for_write.shape[1]:
                        continue

                    raw_val = safe_df_for_write.iat[df_row_idx, excel_col]
                    # detect Base row
                    try:
                        first_cell = safe_df_for_write.iat[df_row_idx, 0] if safe_df_for_write.shape[1] > 0 else None
                        is_base_row = (str(first_cell).strip().lower() == "base")
                    except Exception:
                        is_base_row = False

                    numeric_val = None
                    if not is_base_row:
                        numeric_val = parse_numeric_to_fraction_for_write(raw_val)

                    try:
                        if is_base_row:
                            worksheet.write(excel_row, excel_col, raw_val, highlight_fmt)
                        else:
                            if numeric_val is not None and isinstance(numeric_val, (int, float, np.number)) and np.isfinite(numeric_val):
                                worksheet.write_number(excel_row, excel_col, numeric_val, highlight_percent_fmt)
                            else:
                                worksheet.write(excel_row, excel_col, raw_val, highlight_fmt)
                    except Exception as exc:
                        print(f"Warning: failed to highlight cell in {table_name} at row {row_idx}, col {col_idx}: {exc}")

        written_files.append(str(excel_path))
        print(f"Wrote highlighted Excel: {excel_path}")

    except PermissionError:
        print(f"PermissionError: cannot write {excel_path} — file may be open or blocked. Skipping.")
        continue
    except Exception as exc:
        print(f"Failed writing Excel for table {table_name}: {exc}")
        continue

# write master CSV summary (highlight rows only)
try:
    csv_df = pd.DataFrame(csv_rows)
    csv_df.to_csv(CSV_OUT, index=False)
    print(f"Wrote CSV summary of highlighted cells: {CSV_OUT}")
except Exception as exc:
    print(f"Failed writing CSV summary: {exc}")

print("\nExport complete. Files written:", len(written_files))
print("Output directory:", OUT_DIR)


Wrote highlighted Excel: c:\Users\jiali\survey-highlighter\HighlighterTool\highlighted_outputs\S5._Which_category_do_you_fall_under_highlighted.xlsx
Wrote highlighted Excel: c:\Users\jiali\survey-highlighter\HighlighterTool\highlighted_outputs\S5._Which_category_do_you_fall_under_SigTestTable_highlighted.xlsx


C:\Users\jiali\AppData\Local\Temp\ipykernel_56144\139066387.py:137: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  safe_df_for_write = non_sig_df.replace([np.inf, -np.inf], np.nan).copy()


Wrote highlighted Excel: c:\Users\jiali\survey-highlighter\HighlighterTool\highlighted_outputs\B1_Stance_highlighted.xlsx
Wrote highlighted Excel: c:\Users\jiali\survey-highlighter\HighlighterTool\highlighted_outputs\B1_Stance_SigTestTable_highlighted.xlsx
Wrote highlighted Excel: c:\Users\jiali\survey-highlighter\HighlighterTool\highlighted_outputs\$A1.Score_highlighted.xlsx
Wrote highlighted Excel: c:\Users\jiali\survey-highlighter\HighlighterTool\highlighted_outputs\$A1.Score_SigTestTable_highlighted.xlsx
Wrote CSV summary of highlighted cells: c:\Users\jiali\survey-highlighter\HighlighterTool\highlighted_outputs\highlighted_cells.csv

Export complete. Files written: 6
Output directory: c:\Users\jiali\survey-highlighter\HighlighterTool\highlighted_outputs


### Green Highlights 

In [27]:
# Cell: Locate all cells in SigTest tables that contain uppercase "A" 
#       but NOT in columns whose header contains the word "total"

import re
import pandas as pd

# safety check
if 'table_name_dict' not in globals():
    raise RuntimeError("table_name_dict not found — run the earlier cell that builds it first.")

# --- helper: extract uppercase token runs ---
def extract_upper_tokens(s):
    if s is None:
        return []
    s_str = str(s)
    return re.findall(r"[A-Z]+", s_str)

# --- OUTPUT STRUCTURE ---
# { table_name : { column_name : [ { row_index, row_name, raw_value, tokens }, ... ] } }
cells_with_A = {}

# --- SCAN ALL SIGTEST TABLES ---
for idx, info in table_name_dict.items():

    if not info.get("is_sigtest", False):
        continue  # skip non-sig tables

    tname = info.get("name", f"table_{idx}")
    df = info.get("dataframe")

    if df is None or df.shape[0] == 0 or df.shape[1] == 0:
        cells_with_A[tname] = {}
        continue

    result_cols = {}

    for colname in df.columns:

        # skip any column whose name contains "total"
        if isinstance(colname, str) and "total" in colname.lower():
            continue

        matches = []

        for r_idx in range(df.shape[0]):
            val = df.iloc[r_idx, df.columns.get_loc(colname)]
            if pd.isna(val):
                continue

            s = str(val).strip()

            # extract uppercase tokens (e.g., A, ACE, AB)
            tokens = extract_upper_tokens(s)  
            if not tokens:
                continue

            # does ANY token contain letter "A"?
            contains_A = any("A" in tok for tok in tokens)
            if not contains_A:
                continue

            # get row_name from first column
            row_name = None
            try:
                rn = df.iloc[r_idx, 0]
                row_name = None if pd.isna(rn) else str(rn).strip()
            except Exception:
                row_name = None

            matches.append({
                "row_index": r_idx,
                "row_name": row_name,
                "raw_value": s,
                "tokens": tokens
            })

        if matches:
            result_cols[colname] = matches

    cells_with_A[tname] = result_cols

# --- PRINT SUMMARY ---
print("\n=== Cells Containing Letter 'A' (Ignoring TOTAL Columns) ===\n")
for tname, cols in cells_with_A.items():
    print("="*90)
    print("TABLE:", tname)
    if not cols:
        print("  (no A-found entries outside Total columns)")
        continue

    for colname, matches in cols.items():
        print(f"\n  Column: {colname} — {len(matches)} match(es)")
        for m in matches:
            print(
                f"    row {m['row_index']:>3} ({m['row_name']}) "
                f"raw={m['raw_value']!r} → tokens={m['tokens']}"
            )

cells_with_A  # keeps the dictionary available



=== Cells Containing Letter 'A' (Ignoring TOTAL Columns) ===

TABLE: S5. Which category do you fall under_SigTestTable

  Column: Local Youths — 8 match(es)
    row   1 (Secondary School/ IP Junior High Student) raw='A C' → tokens=['A', 'C']
    row   2 (JC/MI/ IB/ IP Senior High Student) raw='A' → tokens=['A']
    row   3 (Polytechnic Student) raw='A C' → tokens=['A', 'C']
    row   4 (ITE Student) raw='A C' → tokens=['A', 'C']
    row   5 (Local University Student (i.e., NUS/NTU/ SMU/ SUTD/ SIT/ SUSS)) raw='A' → tokens=['A']
    row   7 (Full-time National Servicemen (NSF)) raw='A' → tokens=['A']
    row  10 (Net Not Working) raw='A D' → tokens=['A', 'D']
    row  11 (Net Student) raw='A' → tokens=['A']

  Column: Non-Local Youths — 4 match(es)
    row   5 (Local University Student (i.e., NUS/NTU/ SMU/ SUTD/ SIT/ SUSS)) raw='A' → tokens=['A']
    row   6 (Private Education Institute Student) raw='A B' → tokens=['A', 'B']
    row  10 (Net Not Working) raw='A B D' → tokens=['A', 'B', 

{'S5. Which category do you fall under_SigTestTable': {'Local Youths': [{'row_index': 1,
    'row_name': 'Secondary School/ IP Junior High Student',
    'raw_value': 'A C',
    'tokens': ['A', 'C']},
   {'row_index': 2,
    'row_name': 'JC/MI/ IB/ IP Senior High Student',
    'raw_value': 'A',
    'tokens': ['A']},
   {'row_index': 3,
    'row_name': 'Polytechnic Student',
    'raw_value': 'A C',
    'tokens': ['A', 'C']},
   {'row_index': 4,
    'row_name': 'ITE Student',
    'raw_value': 'A C',
    'tokens': ['A', 'C']},
   {'row_index': 5,
    'row_name': 'Local University Student (i.e., NUS/NTU/ SMU/ SUTD/ SIT/ SUSS)',
    'raw_value': 'A',
    'tokens': ['A']},
   {'row_index': 7,
    'row_name': 'Full-time National Servicemen (NSF)',
    'raw_value': 'A',
    'tokens': ['A']},
   {'row_index': 10,
    'row_name': 'Net Not Working',
    'raw_value': 'A D',
    'tokens': ['A', 'D']},
   {'row_index': 11,
    'row_name': 'Net Student',
    'raw_value': 'A',
    'tokens': ['A']}],
  

In [28]:
# Cell: Compare A-cells (SigTest) vs Total_1 (Non-SigTest) using SAME logic as main comparison cell
import re
import pandas as pd
from IPython.display import display

# ---------- Safety ----------
if 'cells_with_A' not in globals():
    raise RuntimeError("cells_with_A not found — run the A-scan cell first.")
if 'table_name_dict' not in globals():
    raise RuntimeError("table_name_dict not found — build it first.")

# ---------- Helpers (identical style to main comparison cell) ----------
def parse_numeric_to_fraction(x):
    if x is None: return None
    if isinstance(x,(int,float)) and not pd.isna(x):
        v=float(x)
        if 0<=v<=1: return v
        if 1<v<=100: return v/100
        return v/100
    s=str(x).strip()
    if s=="" or s.lower() in {"nan","none"}: return None
    s=s.replace("(","").replace(")","")
    if "%" in s:
        try: return float(s.replace("%","").replace(",",""))/100
        except: return None
    try:
        v=float(s.replace(",",""))
        if 0<=v<=1: return v
        if 1<v<=100: return v/100
        return v/100
    except:
        return None

def get_df_by_name(name):
    if name is None: return None
    for idx,info in table_name_dict.items():
        if info.get("name")==name: return info.get("dataframe")
    for idx,info in table_name_dict.items():
        if name in (info.get("name") or ""):
            return info.get("dataframe")
    return None

def find_total_column(df, key="Total_1"):
    # identical pattern to your main logic
    if df is None: return None
    kl = str(key).strip().lower()
    for c in df.columns:
        if isinstance(c,str) and c.strip().lower()==kl:
            return c
    # substring
    for c in df.columns:
        if isinstance(c,str) and kl in c.lower():
            return c
    # fallback: any "total" col
    for c in df.columns:
        if isinstance(c,str) and "total" in c.lower():
            return c
    return None

def match_non_sig_table(sig_table):
    base=sig_table.replace("_SigTestTable","")
    base=base.strip()
    # exact
    for idx,info in table_name_dict.items():
        if info.get("is_sigtest"): continue
        name=info.get("name") or ""
        if name==base or name.startswith(base):
            return info
    # contains
    for idx,info in table_name_dict.items():
        if info.get("is_sigtest"): continue
        name=info.get("name") or ""
        if base in name: return info
    return None

def find_non_sig_column(df, sig_col):
    """Match SigTest column name -> non-sig column name (same logic style)."""
    if df is None or sig_col is None:
        return None
    sig_low=str(sig_col).strip().lower()

    # 1. exact match
    for c in df.columns:
        if isinstance(c,str) and c==sig_col:
            return c
    # 2. case-insensitive
    for c in df.columns:
        if isinstance(c,str) and c.strip().lower()==sig_low:
            return c
    # 3. substring
    for c in df.columns:
        if isinstance(c,str) and sig_low in c.lower():
            return c

    return None

# ---------- Main comparison ----------
rows=[]

for sig_table, colmap in cells_with_A.items():

    # Load SigTest DF
    sig_df=get_df_by_name(sig_table)

    # Map to non-sig DF
    non_sig_info=match_non_sig_table(sig_table)
    if non_sig_info:
        non_sig_table=non_sig_info["name"]
        non_sig_df=non_sig_info["dataframe"]
    else:
        non_sig_table=None
        non_sig_df=None

    # Locate Total_1 col IN THE NON-SIG table
    non_sig_total1_col=find_total_column(non_sig_df, "Total_1")

    # Process each SigTest column where 'A' was found
    for sig_col, matchlist in colmap.items():
        for ent in matchlist:

            sig_row_idx=ent["row_index"]
            sig_row_name=ent["row_name"]
            sig_raw=ent["raw_value"]
            sig_fraction=parse_numeric_to_fraction(sig_raw)

            rec={
                "sig_table":sig_table,
                "sig_col":sig_col,
                "sig_row_index":sig_row_idx,
                "sig_row_name":sig_row_name,
                "sig_raw":sig_raw,
                "sig_fraction":sig_fraction,
                "non_sig_table":non_sig_table,
                "non_sig_total1_col":non_sig_total1_col,
                "non_sig_row_index":None,
                "non_sig_cell_col":None,
                "non_sig_cell_raw":None,
                "non_sig_total1_raw":None,
                "abs_diff":None,
                "highlight":False,
                "reason":None
            }

            # --- Find row in non-sig table ---
            ns_row_idx=None
            if non_sig_df is not None and sig_row_name:
                target=str(sig_row_name).strip().lower()
                for r in range(non_sig_df.shape[0]):
                    val=str(non_sig_df.iloc[r,0]).strip().lower() if not pd.isna(non_sig_df.iloc[r,0]) else None
                    if val and val==target:
                        ns_row_idx=r
                        break

            rec["non_sig_row_index"]=ns_row_idx
            if ns_row_idx is None:
                rec["reason"]="row_not_found"
                rows.append(rec)
                continue

            # --- Find non-sig cell column matching Sig column ---
            ns_cell_col=find_non_sig_column(non_sig_df,sig_col)
            rec["non_sig_cell_col"]=ns_cell_col
            if ns_cell_col is None:
                rec["reason"]="non_sig_cell_col_not_found"
                rows.append(rec)
                continue

            # --- Read non-sig cell value
            try:
                ns_cell_raw=non_sig_df.loc[ns_row_idx, ns_cell_col]
            except:
                ns_cell_raw=None
            rec["non_sig_cell_raw"]=ns_cell_raw
            ns_cell_fraction=parse_numeric_to_fraction(ns_cell_raw)

            # --- Read non-sig Total_1 value
            if non_sig_total1_col:
                try:
                    ns_total1_raw=non_sig_df.loc[ns_row_idx, non_sig_total1_col]
                except:
                    ns_total1_raw=None
                ns_total1_fraction=parse_numeric_to_fraction(ns_total1_raw)
            else:
                ns_total1_raw=None
                ns_total1_fraction=None

            rec["non_sig_total1_raw"]=ns_total1_raw

            # --- Compute abs_diff (same logic style as main cell)
            abs_diff=None
            highlight=False
            if ns_cell_fraction is not None and ns_total1_fraction is not None:
                abs_diff=abs(ns_cell_fraction - ns_total1_fraction)
                highlight=abs_diff>=0.05
                rec["reason"]="ok"
            else:
                rec["reason"]="value_unparsable"

            rec["abs_diff"]=abs_diff
            rec["highlight"]=highlight

            rows.append(rec)

# ---------- Build DataFrame ----------
sigA_vs_nonSig_total1_df=pd.DataFrame(rows)

print("Total comparisons:", len(sigA_vs_nonSig_total1_df))

if not sigA_vs_nonSig_total1_df.empty:
    df_sorted=sigA_vs_nonSig_total1_df.sort_values(
        by=["highlight","abs_diff","sig_row_index"],
        ascending=[False,False,True],
        na_position="last"
    )
    display(df_sorted.head(200))

    print("\n=== A-cell mismatches (abs_diff ≥ 5%) ===\n")
    for _,r in df_sorted[df_sorted["highlight"]].iterrows():
        def pct(x): return f"{x*100:.2f}%" if x is not None else "N/A"
        print(
            f"- {r['non_sig_table']} | row {r['non_sig_row_index']} | col '{r['non_sig_cell_col']}'"
            f"\n    cell = {r['non_sig_cell_raw']!r}"
            f"\n    Total_1 = {r['non_sig_total1_raw']!r}"
            f"\n    abs_diff = {pct(r['abs_diff'])}"
            "\n"
        )

globals()['sigA_vs_nonSig_total1_df']=sigA_vs_nonSig_total1_df
print("Done — results stored in sigA_vs_nonSig_total1_df")


Total comparisons: 291


,sig_table,sig_col,sig_row_index,sig_row_name,sig_raw,sig_fraction,non_sig_table,non_sig_total1_col,non_sig_row_index,non_sig_cell_col,non_sig_cell_raw,non_sig_total1_raw,abs_diff,highlight,reason
67,S5. Which category do you fall under_SigTestTable,Local Uni,5,"Local University Student (i.e., NUS/NTU/ SMU/ ...",A C J,None,S5. Which category do you fall under,Total_1,5,Local Uni,0.998000,0.110000,0.888000,True,ok
14,S5. Which category do you fall under_SigTestTable,13 – 14 years,1,Secondary School/ IP Junior High Student,A C,None,S5. Which category do you fall under,Total_1,1,13 – 14 years,0.992000,0.137000,0.855000,True,ok
38,S5. Which category do you fall under_SigTestTable,Above 69 years,9,Currently not schooling or working,A C D E F G H I J K L M,None,S5. Which category do you fall under,Total_1,9,Above 69 years,0.825000,0.110000,0.715000,True,ok
111,S5. Which category do you fall under_SigTestTable,25 years old and above,9,Currently not schooling or working,A B C D E,None,S5. Which category do you fall under,Total_1,9,25 years old and above,0.593000,0.110000,0.483000,True,ok
28,S5. Which category do you fall under_SigTestTable,30 – 34 years,8,Currently holding a full-time/ part-time job,A C D K L M N,None,S5. Which category do you fall under,Total_1,8,30 – 34 years,0.911000,0.429000,0.482000,True,ok
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
254,$A1.Score_SigTestTable,I've been feeling optimistic about the future,6,Consuming cannabis can be addictive.,A,None,$A1.Score,Total_1,6,I've been feeling optimistic about the future,0.898643,0.878195,0.020449,False,ok
86,S5. Which category do you fall under_SigTestTable,NET HDB,8,Currently holding a full-time/ part-time job,A J L,None,S5. Which category do you fall under,Total_1,8,NET HDB,0.449000,0.429000,0.020000,False,ok
149,S5. Which category do you fall under_SigTestTable,I enjoy new and exciting experiences even if t...,5,"Local University Student (i.e., NUS/NTU/ SMU/ ...",A B,None,S5. Which category do you fall under,Total_1,5,I enjoy new and exciting experiences even if t...,0.130000,0.110000,0.020000,False,ok
132,S5. Which category do you fall under_SigTestTable,NET Digital & Social Media,6,Private Education Institute Student,A B,None,S5. Which category do you fall under,Total_1,6,NET Digital & Social Media,0.092000,0.072000,0.020000,False,ok



=== A-cell mismatches (abs_diff ≥ 5%) ===

- S5. Which category do you fall under | row 5 | col 'Local Uni'
    cell = 0.998
    Total_1 = 0.11
    abs_diff = 88.80%

- S5. Which category do you fall under | row 1 | col '13 – 14 years'
    cell = 0.992
    Total_1 = 0.137
    abs_diff = 85.50%

- S5. Which category do you fall under | row 9 | col 'Above 69 years'
    cell = 0.825
    Total_1 = 0.11
    abs_diff = 71.50%

- S5. Which category do you fall under | row 9 | col '25 years old and above'
    cell = 0.593
    Total_1 = 0.11
    abs_diff = 48.30%

- S5. Which category do you fall under | row 8 | col '30 – 34 years'
    cell = 0.911
    Total_1 = 0.429
    abs_diff = 48.20%

- S5. Which category do you fall under | row 6 | col 'Non-Local Youths'
    cell = 0.553
    Total_1 = 0.072
    abs_diff = 48.10%

- S5. Which category do you fall under | row 6 | col 'Others'
    cell = 0.553
    Total_1 = 0.072
    abs_diff = 48.10%

- S5. Which category do you fall under | row 8 | col '

In [ ]:
# Export A-vs-Total_1 highlighted cells -> CSV + Excel (green highlight #DAF2D0)
# Exports ALL tables in table_name_dict; applies green highlights where present,
# then overlays red highlights into the SAME green file (both -> gold #FFD966).
import os
from pathlib import Path
import pandas as pd
import numpy as np
import re

# Output directory
OUT_DIR = Path.cwd() / "highlighted_outputs"
OUT_DIR.mkdir(parents=True, exist_ok=True)

# CSV filename changed from sigA_highlighted_cells.csv -> sigGreen_highlighted_cells.csv
CSV_OUT = OUT_DIR / "sigGreen_highlighted_cells.csv"

# Safety checks
if 'sigA_vs_nonSig_total1_df' not in globals():
    raise RuntimeError("sigA_vs_nonSig_total1_df not found — run the A-vs-Total_1 comparison cell first.")
if 'table_name_dict' not in globals():
    raise RuntimeError("table_name_dict not found — build it first.")

results_df = sigA_vs_nonSig_total1_df.copy()

# Keep only the flagged highlights (we'll export all tables but only these contribute to CSV/highlighting)
highlights = results_df[results_df.get("highlight") == True].reset_index(drop=True)

# Build a lookup: non_sig_table -> DataFrame of highlight rows (may be empty dict)
if not highlights.empty:
    highlights_by_table = {k: g for k, g in highlights.groupby("non_sig_table")}
else:
    highlights_by_table = {}

# load red highlights CSV (this is the file that contains the "red" highlights you want overlaid)
red_csv_path = OUT_DIR / "highlighted_cells.csv"
if red_csv_path.exists():
    try:
        df_red_global = pd.read_csv(red_csv_path)
    except Exception:
        df_red_global = pd.DataFrame()
else:
    df_red_global = pd.DataFrame()

# helper: parse numeric to fraction (same as used earlier)
def parse_numeric_to_fraction_for_write(x):
    if x is None:
        return None
    if isinstance(x, (int, float, np.number)) and not pd.isna(x):
        v = float(x)
        if 0.0 <= v <= 1.0:
            return v
        if 1.0 < v <= 100.0:
            return v / 100.0
        return v / 100.0
    s = str(x).strip()
    if s == "" or s.lower() in {"nan","none"}:
        return None
    s = s.replace("(", "").replace(")", "")
    if "%" in s:
        try:
            return float(s.replace("%", "").replace(",", "").strip()) / 100.0
        except Exception:
            return None
    s2 = s.replace(",", "")
    try:
        v = float(s2)
        if 0.0 <= v <= 1.0:
            return v
        if 1.0 < v <= 100.0:
            return v / 100.0
        return v / 100.0
    except Exception:
        return None

# -------------------------
# helper: safe filename for Windows
# -------------------------
def _safe_filename(s):
    if s is None:
        return "table"
    name = str(s).strip()
    # remove forbidden characters and replace whitespace with underscore
    name = re.sub(r'[<>:"/\\|?*\n\r\t]', '', name)
    name = re.sub(r'\s+', '_', name)
    # remove leading/trailing dots/underscores
    name = name.strip("._")
    # collapse long names
    if len(name) > 120:
        name = name[:120].rstrip("_")
    # avoid reserved device names (simple guard)
    if name.upper() in {"CON","PRN","AUX","NUL","COM1","COM2","LPT1","LPT2"}:
        name = f"table_{name}"
    # If name becomes empty, fallback
    if name == "":
        name = "table"
    return name

# We'll collect CSV rows only for highlighted rows
csv_rows = []

# Track written files
written_files = []

# Loop over all tables in table_name_dict and export each one
for tbl_idx, info in table_name_dict.items():
    non_sig_table_name = info.get("name") or f"table_{tbl_idx}"
    non_sig_df = info.get("dataframe")
    if non_sig_df is None:
        print(f"Skipping table {non_sig_table_name}: missing dataframe.")
        continue
    if non_sig_df.empty:
        print(f"Skipping table {non_sig_table_name}: dataframe is empty.")
        continue

    # find highlights for this table (may be None)
    group = highlights_by_table.get(non_sig_table_name)

    # If there are highlight rows, append them to csv_rows
    if group is not None and not group.empty:
        for _, r in group.iterrows():
            csv_rows.append({
                "non_sig_table": non_sig_table_name,
                "non_sig_table_index": tbl_idx,
                "non_sig_row_index": r.get("non_sig_row_index"),
                "non_sig_cell_col": r.get("non_sig_cell_col"),
                "non_sig_cell_raw": r.get("non_sig_cell_raw"),
                "non_sig_total1_raw": r.get("non_sig_total1_raw"),
                "abs_diff": r.get("abs_diff"),
                "sig_table": r.get("sig_table"),
                "sig_col": r.get("sig_col"),
                "sig_row_name": r.get("sig_row_name"),
            })

    # Build safe excel filename for green output (preserve your original behavior)
    safe_name = _safe_filename(non_sig_table_name)
    excel_path = OUT_DIR / f"{safe_name}_Highlights.xlsx"

    # Create a copy/sanitized df to write to excel to avoid inf issues
    # Using .copy() avoids the pandas downcasting FutureWarning
    safe_df_for_write = non_sig_df.replace([np.inf, -np.inf], np.nan).copy()

    # We'll track the set of green coordinates written (row_index, excel_col_index) for overlap detection
    green_coords = set()

    # Write workbook with nan/inf handling and permission error handling (GREEN file)
    try:
        # Pass xlsxwriter Workbook options via engine_kwargs so it works on older pandas builds
        with pd.ExcelWriter(excel_path, engine="xlsxwriter",
                        engine_kwargs={"options": {"nan_inf_to_errors": True}}) as writer:
            # write the dataframe (we will overwrite header and formats manually)
            safe_df_for_write.to_excel(writer, sheet_name="data", index=False)
            workbook = writer.book
            worksheet = writer.sheets["data"]


            # Header format (soft header color)
            header_fmt = workbook.add_format({
                "bg_color": "#D9E1F2",
                "bold": True,
                "align": "center"
            })

            # ---- GREEN HIGHLIGHT FORMATS (requested name) ----
            green_highlight_fmt = workbook.add_format({"bg_color": "#DAF2D0"})
            green_highlight_percent_fmt = workbook.add_format({
                "bg_color": "#DAF2D0",
                "num_format": "0.0%"
            })

            # Percent format for data cells (one decimal)
            percent_fmt = workbook.add_format({"num_format": "0.0%"})

            # 1) rewrite headers with header_fmt (overwrite default)
            for col_idx, col_name in enumerate(safe_df_for_write.columns):
                worksheet.write(0, col_idx, col_name, header_fmt)

            # 2) write all cells, applying percent_fmt to numeric cells except Base rows
            for df_row_idx in range(safe_df_for_write.shape[0]):
                # check if this row is the 'Base' row (first column equals 'Base')
                first_cell = safe_df_for_write.iat[df_row_idx, 0] if safe_df_for_write.shape[1] > 0 else None
                is_base_row = False
                try:
                    is_base_row = (str(first_cell).strip().lower() == "base")
                except Exception:
                    is_base_row = False

                excel_row = df_row_idx + 1  # because header is row 0
                for col_idx in range(safe_df_for_write.shape[1]):
                    val = safe_df_for_write.iat[df_row_idx, col_idx]
                    # if it's numeric / parseable as numeric fraction and not Base row -> write as percentage
                    numeric_val = None
                    if not is_base_row:
                        numeric_val = parse_numeric_to_fraction_for_write(val)
                    # write numeric fraction with percent_fmt (if numeric_val is not None)
                    if numeric_val is not None:
                        try:
                            worksheet.write_number(excel_row, col_idx, numeric_val, percent_fmt)
                        except Exception:
                            worksheet.write(excel_row, col_idx, val)
                    else:
                        worksheet.write(excel_row, col_idx, val)

            # 3) if there are highlighted entries for this table, apply green highlight formatting
            if group is not None and not group.empty:
                for _, r in group.iterrows():
                    ns_row_idx = r.get("non_sig_row_index")
                    ns_cell_col = r.get("non_sig_cell_col")
                    if pd.isna(ns_row_idx) or ns_cell_col is None:
                        continue
                    try:
                        ns_row_idx = int(ns_row_idx)
                    except Exception:
                        continue

                    # find column index in dataframe (your code expects column name here)
                    try:
                        excel_col = list(safe_df_for_write.columns).index(ns_cell_col)
                    except Exception:
                        # column not found in df: skip
                        continue

                    # bounds check
                    if ns_row_idx < 0 or ns_row_idx >= safe_df_for_write.shape[0] or excel_col < 0 or excel_col >= safe_df_for_write.shape[1]:
                        continue

                    excel_row = ns_row_idx + 1

                    # determine whether the row is Base
                    try:
                        first_cell = safe_df_for_write.iat[ns_row_idx, 0] if safe_df_for_write.shape[1] > 0 else None
                        is_base_row = (str(first_cell).strip().lower() == "base")
                    except Exception:
                        is_base_row = False

                    # pick appropriate format: green_highlight_percent_fmt if numeric and not Base, otherwise green_highlight_fmt
                    raw_val = safe_df_for_write.iat[ns_row_idx, excel_col]
                    numeric_val = None
                    if not is_base_row:
                        numeric_val = parse_numeric_to_fraction_for_write(raw_val)

                    try:
                        if is_base_row:
                            # highlight without percent format
                            worksheet.write(excel_row, excel_col, raw_val, green_highlight_fmt)
                        else:
                            if numeric_val is not None:
                                worksheet.write_number(excel_row, excel_col, numeric_val, green_highlight_percent_fmt)
                            else:
                                worksheet.write(excel_row, excel_col, raw_val, green_highlight_fmt)
                        # record the green coord for later overlap detection with red
                        green_coords.add((ns_row_idx, excel_col))
                    except Exception as exc:
                        print(f"Warning: failed to highlight cell {non_sig_table_name} row {ns_row_idx} col {ns_cell_col}: {exc}")

            # -----------------------
            # Now overlay red highlights into the SAME workbook/sheet
            # -----------------------
            # red / both formats
            red_highlight_fmt = workbook.add_format({"bg_color": "#FBE2D5"})
            red_highlight_percent_fmt = workbook.add_format({"bg_color": "#FBE2D5", "num_format": "0.0%"})
            both_highlight_fmt = workbook.add_format({"bg_color": "#FFD966"})
            both_highlight_percent_fmt = workbook.add_format({"bg_color": "#FFD966", "num_format": "0.0%"})

            # Now apply red highlights for this table using the global red CSV (df_red_global)
            if not df_red_global.empty:
                red_rows_for_table = df_red_global[df_red_global.get("non_sig_table") == non_sig_table_name]
            else:
                red_rows_for_table = pd.DataFrame()

            if not red_rows_for_table.empty:
                for _, rr in red_rows_for_table.iterrows():
                    # resolve row/col in same style you used for green
                    rr_row_idx = rr.get("mapped_row_index") if "mapped_row_index" in rr.index else rr.get("non_sig_row_index")
                    rr_col_val = rr.get("non_sig_column_index") if "non_sig_column_index" in rr.index else rr.get("non_sig_cell_col")
                    if pd.isna(rr_row_idx) or rr_col_val is None:
                        continue
                    try:
                        rr_row_idx = int(rr_row_idx)
                    except Exception:
                        continue

                    # rr_col_val might be numeric index or column name; support both
                    try:
                        if isinstance(rr_col_val, (int, float)) or str(rr_col_val).strip().isdigit():
                            rr_col_idx = int(rr_col_val)
                        else:
                            rr_col_idx = list(safe_df_for_write.columns).index(rr_col_val)
                    except Exception:
                        # column not found: skip
                        continue

                    # bounds check
                    if rr_row_idx < 0 or rr_row_idx >= safe_df_for_write.shape[0] or rr_col_idx < 0 or rr_col_idx >= safe_df_for_write.shape[1]:
                        continue

                    rr_excel_row = rr_row_idx + 1
                    rr_excel_col = rr_col_idx

                    # detect Base row
                    try:
                        first_cell = safe_df_for_write.iat[rr_row_idx, 0] if safe_df_for_write.shape[1] > 0 else None
                        rr_is_base = (str(first_cell).strip().lower() == "base")
                    except Exception:
                        rr_is_base = False

                    rr_raw_val = safe_df_for_write.iat[rr_row_idx, rr_excel_col]
                    rr_numeric_val = None
                    if not rr_is_base:
                        rr_numeric_val = parse_numeric_to_fraction_for_write(rr_raw_val)

                    # if this exact coord was green earlier, mark BOTH; otherwise mark RED
                    if (rr_row_idx, rr_excel_col) in green_coords:
                        # BOTH
                        try:
                            if rr_is_base:
                                worksheet.write(rr_excel_row, rr_excel_col, rr_raw_val, both_highlight_fmt)
                            else:
                                if rr_numeric_val is not None and isinstance(rr_numeric_val, (int, float, np.number)) and np.isfinite(rr_numeric_val):
                                    worksheet.write_number(rr_excel_row, rr_excel_col, rr_numeric_val, both_highlight_percent_fmt)
                                else:
                                    worksheet.write(rr_excel_row, rr_excel_col, rr_raw_val, both_highlight_fmt)
                        except Exception as exc:
                            print(f"Warning: failed to write BOTH highlight for {non_sig_table_name} row {rr_row_idx} col {rr_col_val}: {exc}")
                    else:
                        # RED
                        try:
                            if rr_is_base:
                                worksheet.write(rr_excel_row, rr_excel_col, rr_raw_val, red_highlight_fmt)
                            else:
                                if rr_numeric_val is not None and isinstance(rr_numeric_val, (int, float, np.number)) and np.isfinite(rr_numeric_val):
                                    worksheet.write_number(rr_excel_row, rr_excel_col, rr_numeric_val, red_highlight_percent_fmt)
                                else:
                                    worksheet.write(rr_excel_row, rr_excel_col, rr_raw_val, red_highlight_fmt)
                        except Exception as exc:
                            print(f"Warning: failed to write RED highlight for {non_sig_table_name} row {rr_row_idx} col {rr_col_val}: {exc}")

        print(f"Wrote highlighted Excel (green + red overlays): {excel_path}")
        written_files.append(str(excel_path))

    except PermissionError:
        print(f"PermissionError: cannot write {excel_path} — file may be open or blocked. Skipping.")
        continue
    except Exception as exc:
        print(f"Failed writing Excel for table {non_sig_table_name}: {exc}")
        continue

# write CSV summary (only contains rows for highlighted entries, as before)
try:
    csv_df = pd.DataFrame(csv_rows)
    csv_df.to_csv(CSV_OUT, index=False)
    print(f"Wrote CSV summary of highlighted cells: {CSV_OUT}")
except Exception as exc:
    print(f"Failed writing CSV summary: {exc}")

print("\nExport complete. Files in:", OUT_DIR)
print("Files written:", len(written_files))


C:\Users\jiali\AppData\Local\Temp\ipykernel_56144\962599261.py:140: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  safe_df_for_write = non_sig_df.replace([np.inf, -np.inf], np.nan).copy()


Wrote highlighted Excel (green + red overlays): c:\Users\jiali\survey-highlighter\HighlighterTool\highlighted_outputs\S5._Which_category_do_you_fall_under_Green_highlighted.xlsx
Wrote highlighted Excel (green + red overlays): c:\Users\jiali\survey-highlighter\HighlighterTool\highlighted_outputs\S5._Which_category_do_you_fall_under_SigTestTable_Green_highlighted.xlsx
Wrote highlighted Excel (green + red overlays): c:\Users\jiali\survey-highlighter\HighlighterTool\highlighted_outputs\B1_Stance_Green_highlighted.xlsx
Wrote highlighted Excel (green + red overlays): c:\Users\jiali\survey-highlighter\HighlighterTool\highlighted_outputs\B1_Stance_SigTestTable_Green_highlighted.xlsx
Wrote highlighted Excel (green + red overlays): c:\Users\jiali\survey-highlighter\HighlighterTool\highlighted_outputs\$A1.Score_Green_highlighted.xlsx
Wrote highlighted Excel (green + red overlays): c:\Users\jiali\survey-highlighter\HighlighterTool\highlighted_outputs\$A1.Score_SigTestTable_Green_highlighted.xlsx
W

the total column i referenced to is the total column in front. 

## The previous code

In [30]:
# # Export A-vs-Total_1 highlighted cells -> CSV + Excel (green highlight #DAF2D0)
# # Exports ALL tables in table_name_dict; applies green highlights where present.
# import os
# from pathlib import Path
# import pandas as pd
# import numpy as np
# import re

# # Output directory
# OUT_DIR = Path.cwd() / "highlighted_outputs"
# OUT_DIR.mkdir(parents=True, exist_ok=True)

# # CSV filename changed from sigA_highlighted_cells.csv -> sigGreen_highlighted_cells.csv
# CSV_OUT = OUT_DIR / "sigGreen_highlighted_cells.csv"

# # Safety checks
# if 'sigA_vs_nonSig_total1_df' not in globals():
#     raise RuntimeError("sigA_vs_nonSig_total1_df not found — run the A-vs-Total_1 comparison cell first.")
# if 'table_name_dict' not in globals():
#     raise RuntimeError("table_name_dict not found — build it first.")

# results_df = sigA_vs_nonSig_total1_df.copy()

# # Keep only the flagged highlights (we'll export all tables but only these contribute to CSV/highlighting)
# highlights = results_df[results_df.get("highlight") == True].reset_index(drop=True)

# # Build a lookup: non_sig_table -> DataFrame of highlight rows (may be empty dict)
# if not highlights.empty:
#     highlights_by_table = {k: g for k, g in highlights.groupby("non_sig_table")}
# else:
#     highlights_by_table = {}

# # helper: parse numeric to fraction (same as used earlier)
# def parse_numeric_to_fraction_for_write(x):
#     if x is None:
#         return None
#     if isinstance(x, (int, float, np.number)) and not pd.isna(x):
#         v = float(x)
#         if 0.0 <= v <= 1.0:
#             return v
#         if 1.0 < v <= 100.0:
#             return v / 100.0
#         return v / 100.0
#     s = str(x).strip()
#     if s == "" or s.lower() in {"nan","none"}:
#         return None
#     s = s.replace("(", "").replace(")", "")
#     if "%" in s:
#         try:
#             return float(s.replace("%", "").replace(",", "").strip()) / 100.0
#         except Exception:
#             return None
#     s2 = s.replace(",", "")
#     try:
#         v = float(s2)
#         if 0.0 <= v <= 1.0:
#             return v
#         if 1.0 < v <= 100.0:
#             return v / 100.0
#         return v / 100.0
#     except Exception:
#         return None

# # -------------------------
# # helper: safe filename for Windows
# # -------------------------
# def _safe_filename(s):
#     if s is None:
#         return "table"
#     name = str(s).strip()
#     # remove forbidden characters and replace whitespace with underscore
#     name = re.sub(r'[<>:"/\\|?*\n\r\t]', '', name)
#     name = re.sub(r'\s+', '_', name)
#     # remove leading/trailing dots/underscores
#     name = name.strip("._")
#     # collapse long names
#     if len(name) > 120:
#         name = name[:120].rstrip("_")
#     # avoid reserved device names (simple guard)
#     if name.upper() in {"CON","PRN","AUX","NUL","COM1","COM2","LPT1","LPT2"}:
#         name = f"table_{name}"
#     # If name becomes empty, fallback
#     if name == "":
#         name = "table"
#     return name

# # We'll collect CSV rows only for highlighted rows
# csv_rows = []

# # Loop over all tables in table_name_dict and export each one
# for tbl_idx, info in table_name_dict.items():
#     non_sig_table_name = info.get("name") or f"table_{tbl_idx}"
#     non_sig_df = info.get("dataframe")
#     if non_sig_df is None:
#         print(f"Skipping table {non_sig_table_name}: missing dataframe.")
#         continue
#     if non_sig_df.empty:
#         print(f"Skipping table {non_sig_table_name}: dataframe is empty.")
#         continue

#     # find highlights for this table (may be None)
#     group = highlights_by_table.get(non_sig_table_name)

#     # If there are highlight rows, append them to csv_rows
#     if group is not None and not group.empty:
#         for _, r in group.iterrows():
#             csv_rows.append({
#                 "non_sig_table": non_sig_table_name,
#                 "non_sig_table_index": tbl_idx,
#                 "non_sig_row_index": r.get("non_sig_row_index"),
#                 "non_sig_cell_col": r.get("non_sig_cell_col"),
#                 "non_sig_cell_raw": r.get("non_sig_cell_raw"),
#                 "non_sig_total1_raw": r.get("non_sig_total1_raw"),
#                 "abs_diff": r.get("abs_diff"),
#                 "sig_table": r.get("sig_table"),
#                 "sig_col": r.get("sig_col"),
#                 "sig_row_name": r.get("sig_row_name"),
#             })

#     # Build safe excel filename
#     safe_name = _safe_filename(non_sig_table_name)
#     excel_path = OUT_DIR / f"{safe_name}_Green_highlighted.xlsx"

#     # Create a copy/sanitized df to write to excel to avoid inf issues
#     # Using .copy() avoids the pandas downcasting FutureWarning
#     safe_df_for_write = non_sig_df.replace([np.inf, -np.inf], np.nan).copy()

#     # Write workbook with nan/inf handling and permission error handling
#     try:
#         # Pass xlsxwriter Workbook options via engine_kwargs so it works on older pandas builds
#         with pd.ExcelWriter(excel_path, engine="xlsxwriter",
#                         engine_kwargs={"options": {"nan_inf_to_errors": True}}) as writer:
#             # write the dataframe (we will overwrite header and formats manually)
#             safe_df_for_write.to_excel(writer, sheet_name="data", index=False)
#             workbook = writer.book
#             worksheet = writer.sheets["data"]


#             # Header format (soft header color)
#             header_fmt = workbook.add_format({
#                 "bg_color": "#D9E1F2",
#                 "bold": True,
#                 "align": "center"
#             })

#             # ---- GREEN HIGHLIGHT FORMATS (requested name) ----
#             green_highlight_fmt = workbook.add_format({"bg_color": "#DAF2D0"})
#             green_highlight_percent_fmt = workbook.add_format({
#                 "bg_color": "#DAF2D0",
#                 "num_format": "0.0%"
#             })

#             # Percent format for data cells (one decimal)
#             percent_fmt = workbook.add_format({"num_format": "0.0%"})

#             # 1) rewrite headers with header_fmt (overwrite default)
#             for col_idx, col_name in enumerate(safe_df_for_write.columns):
#                 worksheet.write(0, col_idx, col_name, header_fmt)

#             # 2) write all cells, applying percent_fmt to numeric cells except Base rows
#             for df_row_idx in range(safe_df_for_write.shape[0]):
#                 # check if this row is the 'Base' row (first column equals 'Base')
#                 first_cell = safe_df_for_write.iat[df_row_idx, 0] if safe_df_for_write.shape[1] > 0 else None
#                 is_base_row = False
#                 try:
#                     is_base_row = (str(first_cell).strip().lower() == "base")
#                 except Exception:
#                     is_base_row = False

#                 excel_row = df_row_idx + 1  # because header is row 0
#                 for col_idx in range(safe_df_for_write.shape[1]):
#                     val = safe_df_for_write.iat[df_row_idx, col_idx]
#                     # if it's numeric / parseable as numeric fraction and not Base row -> write as percentage
#                     numeric_val = None
#                     if not is_base_row:
#                         numeric_val = parse_numeric_to_fraction_for_write(val)
#                     # write numeric fraction with percent_fmt (if numeric_val is not None)
#                     if numeric_val is not None:
#                         try:
#                             worksheet.write_number(excel_row, col_idx, numeric_val, percent_fmt)
#                         except Exception:
#                             worksheet.write(excel_row, col_idx, val)
#                     else:
#                         worksheet.write(excel_row, col_idx, val)

#             # 3) if there are highlighted entries for this table, apply green highlight formatting
#             if group is not None and not group.empty:
#                 for _, r in group.iterrows():
#                     ns_row_idx = r.get("non_sig_row_index")
#                     ns_cell_col = r.get("non_sig_cell_col")
#                     if pd.isna(ns_row_idx) or ns_cell_col is None:
#                         continue
#                     try:
#                         ns_row_idx = int(ns_row_idx)
#                     except Exception:
#                         continue

#                     # find column index in dataframe
#                     try:
#                         excel_col = list(safe_df_for_write.columns).index(ns_cell_col)
#                     except Exception:
#                         # column not found in df: skip
#                         continue

#                     # bounds check
#                     if ns_row_idx < 0 or ns_row_idx >= safe_df_for_write.shape[0] or excel_col < 0 or excel_col >= safe_df_for_write.shape[1]:
#                         continue

#                     excel_row = ns_row_idx + 1

#                     # determine whether the row is Base
#                     try:
#                         first_cell = safe_df_for_write.iat[ns_row_idx, 0] if safe_df_for_write.shape[1] > 0 else None
#                         is_base_row = (str(first_cell).strip().lower() == "base")
#                     except Exception:
#                         is_base_row = False

#                     # pick appropriate format: green_highlight_percent_fmt if numeric and not Base, otherwise green_highlight_fmt
#                     raw_val = safe_df_for_write.iat[ns_row_idx, excel_col]
#                     numeric_val = None
#                     if not is_base_row:
#                         numeric_val = parse_numeric_to_fraction_for_write(raw_val)

#                     try:
#                         if is_base_row:
#                             # highlight without percent format
#                             worksheet.write(excel_row, excel_col, raw_val, green_highlight_fmt)
#                         else:
#                             if numeric_val is not None:
#                                 worksheet.write_number(excel_row, excel_col, numeric_val, green_highlight_percent_fmt)
#                             else:
#                                 worksheet.write(excel_row, excel_col, raw_val, green_highlight_fmt)
#                     except Exception as exc:
#                         print(f"Warning: failed to highlight cell {non_sig_table_name} row {ns_row_idx} col {ns_cell_col}: {exc}")

#         print(f"Wrote highlighted Excel: {excel_path}")

#     except PermissionError:
#         print(f"PermissionError: cannot write {excel_path} — file may be open or blocked. Skipping.")
#         continue
#     except Exception as exc:
#         print(f"Failed writing Excel for table {non_sig_table_name}: {exc}")
#         continue

# # write CSV summary (only contains rows for highlighted entries, as before)
# try:
#     csv_df = pd.DataFrame(csv_rows)
#     csv_df.to_csv(CSV_OUT, index=False)
#     print(f"Wrote CSV summary of highlighted cells: {CSV_OUT}")
# except Exception as exc:
#     print(f"Failed writing CSV summary: {exc}")

# print("\nExport complete. Files in:", OUT_DIR)
